In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")
from datetime import datetime
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib notebook
%matplotlib inline
import seaborn as sns
import pickle
from tqdm import tqdm
import math
import os
from sklearn import metrics
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.multioutput import RegressorChain
from sklearn.model_selection import RandomizedSearchCV
from datetime import datetime
from numpy import mean
from numpy import std
from numpy import absolute
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from sklearn.multioutput import MultiOutputRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
import pickle

## Loading 2019 data

In [2]:
file= 'E:\CIT\MSc_AI\Research project_NIMBUS\Rework\corrected_data.xlsx'
#data = pd.read_excel(file,infer_datetime_format=True,index_col=0,sheet_name='data_2019')
data = pd.read_excel(file,sheet_name='data_2019')
data['applicable_date'] = pd.to_datetime(data['applicable_date'],format='%Y-%m-%d %H:%M:%S')
print(data.shape)
data.head()

(8304, 16)


,applicable_date,elec_prices,rain_d,temp_d,wdsp_d,wddir_d,rain_c,temp_c,wdsp_c,wddir_c,rain_g,temp_g,wdsp_g,wddir_g,daily_oil_price,daily_natural_gas
0,2019-01-01 00:00:00,70.28,0.0,8.8,4.626,250,0.0,8.5,2.056,240,0.0,9.9,2.056,260,59.41,2.61649
1,2019-01-01 01:00:00,65.18,0.0,8.3,4.626,250,0.0,8.5,1.542,270,0.0,9.3,2.056,240,59.41,2.61649
2,2019-01-01 02:00:00,60.91,0.0,8.4,4.112,240,0.0,8.4,1.542,280,0.0,9.2,2.570,240,59.41,2.61649
3,2019-01-01 03:00:00,65.16,0.0,8.6,4.112,250,0.0,8.5,2.570,260,0.0,9.1,2.570,250,59.41,2.61649
4,2019-01-01 04:00:00,65.16,0.0,8.4,5.140,250,0.0,8.5,2.570,270,0.0,8.8,2.056,250,59.41,2.61649


In [3]:
data['applicable_date'].min(), data['applicable_date'].max(), (data['applicable_date'].max() - data['applicable_date'].min())

(Timestamp('2019-01-01 00:00:00'),
 Timestamp('2019-12-12 23:00:00'),
 Timedelta('345 days 23:00:00'))

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8304 entries, 0 to 8303
Data columns (total 16 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   applicable_date    8304 non-null   datetime64[ns]
 1   elec_prices        8304 non-null   float64       
 2   rain_d             8304 non-null   float64       
 3   temp_d             8304 non-null   float64       
 4   wdsp_d             8304 non-null   float64       
 5   wddir_d            8304 non-null   int64         
 6   rain_c             8304 non-null   float64       
 7   temp_c             8304 non-null   float64       
 8   wdsp_c             8304 non-null   float64       
 9   wddir_c            8304 non-null   int64         
 10  rain_g             8304 non-null   float64       
 11  temp_g             8304 non-null   float64       
 12  wdsp_g             8304 non-null   float64       
 13  wddir_g            8304 non-null   int64         
 14  daily_oi

In [5]:
# Basic stats:
data.elec_prices.describe()

count    8304.000000
mean       50.572471
std        23.430899
min       -11.860000
25%        38.417500
50%        47.394000
75%        59.000000
max       365.040000
Name: elec_prices, dtype: float64

## Loading 2018-2019 data for testing

In [6]:
file= 'E:\CIT\MSc_AI\Research project_NIMBUS\Rework\corrected_data.xlsx'
data_1819 = pd.read_excel(file,sheet_name='data_2018_2019')
#data_1819 = pd.read_csv(file,infer_datetime_format=True,index_col=0)
data_1819['applicable_date'] = pd.to_datetime(data_1819['applicable_date'],format='%Y-%m-%d %H:%M:%S')
print(data_1819.shape)
data_1819.head()

(10514, 16)


,applicable_date,elec_prices,rain_d,temp_d,wdsp_d,wddir_d,rain_c,temp_c,wdsp_c,wddir_c,rain_g,temp_g,wdsp_g,wddir_g,daily_oil_price,daily_natural_gas
0,2018-09-30 22:00:00,71.267,0.0,6.4,5.140,290,0.0,8.2,3.084,340,0.0,5.7,1.028,300,78.89,2.78616
1,2018-09-30 23:00:00,67.212,0.0,5.4,5.140,280,0.0,8.1,3.598,320,0.0,3.9,1.542,290,78.89,2.78616
2,2018-10-01 00:00:00,60.500,0.0,5.2,5.140,280,0.0,7.7,4.112,320,0.0,6.2,1.542,270,84.94,2.79509
3,2018-10-01 01:00:00,63.682,0.0,5.2,5.654,280,0.0,7.4,3.084,320,0.0,7.2,2.056,270,84.94,2.79509
4,2018-10-01 02:00:00,71.617,0.0,4.8,4.626,270,0.0,5.3,1.542,200,0.0,6.9,2.056,280,84.94,2.79509


In [7]:
data_1819['applicable_date'].min(), data_1819['applicable_date'].max(), (data_1819['applicable_date'].max() - data_1819['applicable_date'].min())

(Timestamp('2018-09-30 22:00:00'),
 Timestamp('2019-12-12 23:00:00'),
 Timedelta('438 days 01:00:00'))

### Function to normalize data

In [8]:
def normalize(x_train,x_test):
    from sklearn.preprocessing import MinMaxScaler
    scaler = MinMaxScaler().fit(x_train)
    x_train_norm = scaler.transform(x_train)
    x_test_norm = scaler.transform(x_test)
    return x_train_norm,x_test_norm

### Evaluation metrics

In [9]:
from sklearn.metrics import mean_absolute_error,mean_squared_error,r2_score
def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / np.mean(y_true))) * 100
def eval_metrics(y_true,y_pred):
    MAE = mean_absolute_error(y_true,y_pred)
    MSE = mean_squared_error(y_true,y_pred)
    RMSE = mean_squared_error(y_true,y_pred,squared=False)
    MAPE = mean_absolute_percentage_error(y_true,y_pred)
    R2 = r2_score(y_true, y_pred)
    print("MAE: ",round((MAE),4))
    print("MSE: ",round((MSE),4))
    print("RMSE: ",round((RMSE),4))
    print("MAPE: ",round((MAPE),4))
    print("Coefficient of determination (R-squared): ",round((R2),4))
    print('\n')
    return MAE,MSE,RMSE,MAPE,R2

### Function to convert single time step target to multiple(24) time steps

In [10]:
def multi_step(data,target):

    for time_step in range(1,24):
        name = 't'+'+'+str(time_step)
        data[name] = data[target].shift(-time_step)

    return data

### Function to create lags for features

In [11]:
def calc_24lags(data,feature):
    if feature == 'elec_prices':
        for time_step in range(1,25):
            name = 'lag_'+str(time_step)+'_'+feature
            data[name] = data[feature].shift(time_step).bfill(axis ='rows')
    else:
        for time_step in range(1,24):
            name = 'lag_'+str(time_step)+'_'+feature
            data[name] = data[feature].shift(time_step).bfill(axis ='rows')
            
    return data

### Function to remove 23 rows per day

In [12]:
def remove_23rows_hour_col(data):
    data = data[data['Hour']==23]
    data= data.drop(['Hour'], axis=1)
    data = data.reset_index(drop=True)
    print(data.shape)
    return data

### Function to perform time based splitting

In [13]:
#80-20split
def train_test_split_tbs(final_data):
    train,test = final_data.iloc[:int(0.8*final_data.shape[0]), :] , final_data.iloc[int(0.8*final_data.shape[0]):, :]
    print(train.shape)
    print(test.shape)
    
    return train,test

### Functions of Boosting models

#### GBM

In [14]:
def GBM_training(X,y):

    start_time = datetime.now()

    wrapper1 = MultiOutputRegressor(
        GradientBoostingRegressor(max_features='sqrt', random_state=100))

    hyperparameters = {'estimator__learning_rate':[0.1,0.05,0.025,0.0125],
                       'estimator__n_estimators':range(20,101,10),
                       'estimator__max_depth':range(5,14,2),
                       'estimator__min_samples_split':range(100,501,100),
                       'estimator__max_features':range(1,7,1),
                       'estimator__min_samples_leaf':range(10,61,10),
                       'estimator__subsample': [0.5,0.6,0.7,0.75,0.8,0.85,0.9]}

    rsearch = RandomizedSearchCV(
        estimator = wrapper1, param_distributions = hyperparameters,
        scoring='neg_mean_absolute_error',n_jobs = -1, cv=10,
        return_train_score=True)

    rsearch.fit(X,y)

    end_time = datetime.now()
    print('Duration: {}'.format(end_time - start_time))
    print("\n\n")
    print(rsearch.cv_results_)
    print(rsearch.best_params_)
    print(rsearch.best_score_)


In [15]:
def GBM_model(X,y,learning_rate,n_estimators,max_depth,min_samples_split,max_features,min_samples_leaf,subsample):
    
    learning_rate=learning_rate
    n_estimators=n_estimators
    max_depth= max_depth
    min_samples_split= min_samples_split
    max_features= max_features
    min_samples_leaf= min_samples_leaf
    subsample= subsample

    start_time = datetime.now()

    GBM = MultiOutputRegressor(GradientBoostingRegressor(learning_rate=learning_rate, n_estimators=n_estimators, 
                                                         max_depth= max_depth,min_samples_split= min_samples_split,
                                                         max_features= max_features, min_samples_leaf= min_samples_leaf,
                                                         subsample= subsample, random_state=100))

    GBM.fit(X,y)

    end_time = datetime.now()
    print('Duration: {}'.format(end_time - start_time))

    return GBM

#### XGBM

In [16]:
def XGBM_training(X,y):

    start_time = datetime.now()

    wrapper = MultiOutputRegressor(XGBRegressor(n_jobs = -1))

    hyperparameters = {'estimator__learning_rate':[0.2,0.1,0.05,0.025,0.0125],
                       'estimator__n_estimators':range(50, 500, 50),
                       'estimator__max_depth':range(3,14,2),
                       'estimator__subsample':np.arange(0.1, 1.1, 0.1),
                           'estimator__colsample_bytree':np.arange(0.1, 1.1, 0.1)}

    rsearch = RandomizedSearchCV(
        estimator = wrapper, param_distributions = hyperparameters,
        scoring='neg_mean_absolute_error',
        n_jobs = -1, cv=10, return_train_score=True)

    rsearch.fit(X,y)

    end_time = datetime.now()
    print('Duration: {}'.format(end_time - start_time))
    print("\n\n")
    print(rsearch.cv_results_)
    print(rsearch.best_params_)
    print(rsearch.best_score_)

In [17]:
def XGBM_model(X,y,learning_rate,n_estimators,max_depth,subsample,colsample_bytree):

    learning_rate = learning_rate
    n_estimators = n_estimators
    max_depth = max_depth
    subsample = subsample
    colsample_bytree = colsample_bytree

    start_time = datetime.now() 

    XGBM = MultiOutputRegressor(XGBRegressor(learning_rate=learning_rate, n_estimators=n_estimators, max_depth= max_depth,
                                            subsample= subsample, colsample_bytree = colsample_bytree, random_state=100))

    XGBM.fit(X,y)

    end_time = datetime.now()
    print('Duration: {}'.format(end_time - start_time))

    return XGBM

#### LGBM

In [18]:
def LGBM_training(X,y):

    start_time = datetime.now()

    wrapper = MultiOutputRegressor(LGBMRegressor(random_state=100, n_jobs = -1))

    hyperparameters = {'estimator__learning_rate':[0.2,0.1,0.05,0.025,0.0125],
                       'estimator__n_estimators':range(50, 501, 50),
                       'estimator__max_depth':range(3,14,2),
                       'estimator__subsample':np.arange(0.1, 1.1, 0.1),
                       'estimator__colsample_bytree':np.arange(0.1, 1.1, 0.1),
                       'estimator__num_leaves':[round(0.6*2**x) for x in range(3,14,2)],
                      'estimator__min_child_samples ':range(10,71,10)}

    rsearch = RandomizedSearchCV(
        estimator = wrapper, param_distributions = hyperparameters,
        scoring='neg_mean_absolute_error',
        n_jobs = -1, cv=10, return_train_score=True)

    rsearch.fit(X,y)

    end_time = datetime.now()
    print('Duration: {}'.format(end_time - start_time))
    print("\n\n")
    print(rsearch.cv_results_)
    print(rsearch.best_params_)
    print(rsearch.best_score_)

In [19]:
def LGBM_model(X,y,num_leaves,min_child_samples,learning_rate,n_estimators,max_depth,subsample,colsample_bytree):

    num_leaves = num_leaves
    min_child_samples = min_child_samples
    learning_rate = learning_rate
    n_estimators = n_estimators
    max_depth= max_depth
    subsample= subsample
    colsample_bytree = colsample_bytree

    start_time = datetime.now()

    LGB = MultiOutputRegressor(LGBMRegressor(num_leaves=num_leaves,min_child_samples = min_child_samples,learning_rate=learning_rate,n_estimators=n_estimators,
                                               max_depth= max_depth,subsample= subsample,colsample_bytree = colsample_bytree,random_state=100,
                                               n_jobs = -1))

    LGB.fit(X,y)

    end_time = datetime.now()
    print('Duration: {}'.format(end_time - start_time))
    
    return LGB

### Rndomized splitting for testing

In [20]:
def random_splitting(data):
    from sklearn.model_selection import train_test_split
    X= data[x_names]; y= data[y_names]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20)
    X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size=0.5)
    print(X_train.shape,X_val.shape,X_test.shape)
    print(y_train.shape,y_val.shape,y_test.shape)
    
    return X_train,X_val,X_test,y_train,y_val, y_test

### Testing iterations/runs

In [21]:
def testing_runs(n,data,model):
    mae = []; mse = []; rmse = []; mape = []; r2 = []
    for i in range(n):
        print("Round {}: ".format(i+1))
        X_train,X_val,X_test,y_train,y_val, y_test = random_splitting(data)
        x_train_norm,x_test_norm = normalize(X_train,X_test)
        y_pred= model.predict(x_test_norm)
        MAE,MSE,RMSE,MAPE,R2 = eval_metrics(y_test,y_pred)
        mae.append(round((MAE),4))
        mse.append(round(MSE,4))
        rmse.append(round(RMSE,4))
        mape.append(round(MAPE,4))
        r2.append(round(R2,4))
    print("Average MAE: ",round(np.mean(mae),4))
    print("Average MSE: ",round(np.mean(mse),4))
    print("Average RMSE: ", round(np.mean(rmse),4))
    print("Average MAPE: ", round(np.mean(mape),4))
    print("Average R2: ", round(np.mean(r2),4))
    
    return mae,mse,rmse,mape,r2

<h1 style="text-align:center;font-size:40px;" > Basic time based features </h1>

In [22]:
def feature_pipeline_1(basic_time):

    basic_time= basic_time[['applicable_date','elec_prices']]
    basic_time['month']=basic_time['applicable_date'].dt.month
    basic_time['dayofyear_num']=basic_time['applicable_date'].dt.dayofyear
    basic_time['weekofyear_num']=basic_time['applicable_date'].dt.weekofyear
    basic_time['Hour'] = basic_time['applicable_date'].dt.hour
    #basic_time['expanding_window_price'] = basic_time['elec_prices'].expanding().mean()
    return basic_time

In [24]:
basic_time= data.copy()
basic_time = feature_pipeline_1(basic_time)
basic_time.head()

,applicable_date,elec_prices,month,dayofyear_num,weekofyear_num,Hour
0,2019-01-01 00:00:00,70.28,1,1,1,0
1,2019-01-01 01:00:00,65.18,1,1,1,1
2,2019-01-01 02:00:00,60.91,1,1,1,2
3,2019-01-01 03:00:00,65.16,1,1,1,3
4,2019-01-01 04:00:00,65.16,1,1,1,4


In [25]:
basic_time = multi_step(basic_time,'elec_prices')
basic_time.head()

,applicable_date,elec_prices,month,dayofyear_num,weekofyear_num,Hour,t+1,t+2,t+3,t+4,...,t+14,t+15,t+16,t+17,t+18,t+19,t+20,t+21,t+22,t+23
0,2019-01-01 00:00:00,70.28,1,1,1,0,65.18,60.91,65.160,65.160,...,100.278,110.33,185.0,270.0,250.0,148.00,90.000,66.000,89.000,76.090
1,2019-01-01 01:00:00,65.18,1,1,1,1,60.91,65.16,65.160,68.290,...,110.330,185.00,270.0,250.0,148.0,90.00,66.000,89.000,76.090,76.205
2,2019-01-01 02:00:00,60.91,1,1,1,2,65.16,65.16,68.290,63.030,...,185.000,270.00,250.0,148.0,90.0,66.00,89.000,76.090,76.205,73.664
3,2019-01-01 03:00:00,65.16,1,1,1,3,65.16,68.29,63.030,51.621,...,270.000,250.00,148.0,90.0,66.0,89.00,76.090,76.205,73.664,71.380
4,2019-01-01 04:00:00,65.16,1,1,1,4,68.29,63.03,51.621,51.632,...,250.000,148.00,90.0,66.0,89.0,76.09,76.205,73.664,71.380,71.380


In [26]:
basic_time = calc_24lags(basic_time,'elec_prices')
print(basic_time.shape)
print(basic_time.columns)
basic_time.head()

(8304, 53)
Index(['applicable_date', 'elec_prices', 'month', 'dayofyear_num',
       'weekofyear_num', 'Hour', 't+1', 't+2', 't+3', 't+4', 't+5', 't+6',
       't+7', 't+8', 't+9', 't+10', 't+11', 't+12', 't+13', 't+14', 't+15',
       't+16', 't+17', 't+18', 't+19', 't+20', 't+21', 't+22', 't+23',
       'lag_1_elec_prices', 'lag_2_elec_prices', 'lag_3_elec_prices',
       'lag_4_elec_prices', 'lag_5_elec_prices', 'lag_6_elec_prices',
       'lag_7_elec_prices', 'lag_8_elec_prices', 'lag_9_elec_prices',
       'lag_10_elec_prices', 'lag_11_elec_prices', 'lag_12_elec_prices',
       'lag_13_elec_prices', 'lag_14_elec_prices', 'lag_15_elec_prices',
       'lag_16_elec_prices', 'lag_17_elec_prices', 'lag_18_elec_prices',
       'lag_19_elec_prices', 'lag_20_elec_prices', 'lag_21_elec_prices',
       'lag_22_elec_prices', 'lag_23_elec_prices', 'lag_24_elec_prices'],
      dtype='object')


,applicable_date,elec_prices,month,dayofyear_num,weekofyear_num,Hour,t+1,t+2,t+3,t+4,...,lag_15_elec_prices,lag_16_elec_prices,lag_17_elec_prices,lag_18_elec_prices,lag_19_elec_prices,lag_20_elec_prices,lag_21_elec_prices,lag_22_elec_prices,lag_23_elec_prices,lag_24_elec_prices
0,2019-01-01 00:00:00,70.28,1,1,1,0,65.18,60.91,65.160,65.160,...,70.28,70.28,70.28,70.28,70.28,70.28,70.28,70.28,70.28,70.28
1,2019-01-01 01:00:00,65.18,1,1,1,1,60.91,65.16,65.160,68.290,...,70.28,70.28,70.28,70.28,70.28,70.28,70.28,70.28,70.28,70.28
2,2019-01-01 02:00:00,60.91,1,1,1,2,65.16,65.16,68.290,63.030,...,70.28,70.28,70.28,70.28,70.28,70.28,70.28,70.28,70.28,70.28
3,2019-01-01 03:00:00,65.16,1,1,1,3,65.16,68.29,63.030,51.621,...,70.28,70.28,70.28,70.28,70.28,70.28,70.28,70.28,70.28,70.28
4,2019-01-01 04:00:00,65.16,1,1,1,4,68.29,63.03,51.621,51.632,...,70.28,70.28,70.28,70.28,70.28,70.28,70.28,70.28,70.28,70.28


In [27]:
#Dropping rows with Nan's
basic_time = basic_time.dropna(axis=0)
print(basic_time.shape)

(8281, 53)


In [28]:
basic_time.isnull().values.any()

False

In [29]:
basic_time = remove_23rows_hour_col(basic_time)

(345, 52)


In [30]:
basic_time.head()

,applicable_date,elec_prices,month,dayofyear_num,weekofyear_num,t+1,t+2,t+3,t+4,t+5,...,lag_15_elec_prices,lag_16_elec_prices,lag_17_elec_prices,lag_18_elec_prices,lag_19_elec_prices,lag_20_elec_prices,lag_21_elec_prices,lag_22_elec_prices,lag_23_elec_prices,lag_24_elec_prices
0,2019-01-01 23:00:00,76.09,1,1,1,76.205,73.664,71.380,71.380,66.98,...,51.632,51.621,63.030,68.290,65.16,65.160,60.910,65.180,70.280,70.28
1,2019-01-02 23:00:00,71.01,1,2,1,70.720,66.720,66.720,62.720,62.72,...,77.520,75.000,71.380,71.380,66.98,71.380,71.380,73.664,76.205,76.09
2,2019-01-03 23:00:00,72.69,1,3,1,68.690,69.000,59.286,59.195,58.40,...,81.746,75.203,69.801,62.720,62.72,62.720,66.720,66.720,70.720,71.01
3,2019-01-04 23:00:00,82.99,1,4,1,72.570,72.570,72.570,72.570,67.56,...,85.859,75.501,72.940,65.432,58.40,59.195,59.286,69.000,68.690,72.69
4,2019-01-05 23:00:00,72.95,1,5,1,72.950,65.582,62.074,68.380,72.95,...,67.628,63.731,63.894,69.000,67.56,72.570,72.570,72.570,72.570,82.99


In [31]:
train,test = train_test_split_tbs(basic_time)

(276, 52)
(69, 52)


In [32]:
#selecting columns for X & y
cols = train.columns.to_list()
x_names = cols[2:5]
lags = cols[-24:]
x_names.extend(lags)
y_names = [cols[1]]
time_steps = cols[5:28]
y_names.extend(time_steps)
print(x_names)
print(y_names)

['month', 'dayofyear_num', 'weekofyear_num', 'lag_1_elec_prices', 'lag_2_elec_prices', 'lag_3_elec_prices', 'lag_4_elec_prices', 'lag_5_elec_prices', 'lag_6_elec_prices', 'lag_7_elec_prices', 'lag_8_elec_prices', 'lag_9_elec_prices', 'lag_10_elec_prices', 'lag_11_elec_prices', 'lag_12_elec_prices', 'lag_13_elec_prices', 'lag_14_elec_prices', 'lag_15_elec_prices', 'lag_16_elec_prices', 'lag_17_elec_prices', 'lag_18_elec_prices', 'lag_19_elec_prices', 'lag_20_elec_prices', 'lag_21_elec_prices', 'lag_22_elec_prices', 'lag_23_elec_prices', 'lag_24_elec_prices']
['elec_prices', 't+1', 't+2', 't+3', 't+4', 't+5', 't+6', 't+7', 't+8', 't+9', 't+10', 't+11', 't+12', 't+13', 't+14', 't+15', 't+16', 't+17', 't+18', 't+19', 't+20', 't+21', 't+22', 't+23']


In [33]:
x_train, y_train = train[x_names], train[y_names]
x_test, y_test = test[x_names], test[y_names]
print(x_train.shape,y_train.shape)
print(x_test.shape,y_test.shape)

(276, 27) (276, 24)
(69, 27) (69, 24)


In [34]:
x_train_norm,x_test_norm = normalize(x_train,x_test)

In [35]:
x_train_norm[0]

array([0.        , 0.        , 0.        , 0.61662533, 0.75340416,
       0.64133244, 0.70526554, 0.8790667 , 0.72050347, 0.75552843,
       0.67713422, 0.61125066, 0.67816746, 0.40140845, 0.30311572,
       0.32777225, 0.27032883, 0.34465023, 0.35886044, 0.79629428,
       0.9440173 , 0.90612196, 0.90669856, 0.85911068, 0.89888444,
       0.93126849, 0.85555177])

In [36]:
GBM_training(x_train_norm,y_train)

Duration: 0:00:20.150681



{'mean_fit_time': array([1.57412376, 0.66288426, 0.6930258 , 2.58929369, 1.64751256,
       1.5502471 , 2.18241808, 0.82811649, 2.39596732, 1.27349665]), 'std_fit_time': array([0.21623725, 0.10168755, 0.05301185, 0.50804962, 0.16666509,
       0.21725335, 0.32746825, 0.12948324, 0.40850064, 0.26160961]), 'mean_score_time': array([0.02420907, 0.02772598, 0.0281287 , 0.02840755, 0.02395813,
       0.02700579, 0.02570474, 0.02628424, 0.02291522, 0.0154721 ]), 'std_score_time': array([0.00376039, 0.004925  , 0.00768957, 0.00463359, 0.00403906,
       0.00721252, 0.00777317, 0.01019015, 0.0024736 , 0.00891329]), 'param_estimator__subsample': masked_array(data=[0.7, 0.7, 0.5, 0.5, 0.5, 0.6, 0.6, 0.9, 0.75, 0.7],
             mask=[False, False, False, False, False, False, False, False,
                   False, False],
       fill_value='?',
            dtype=object), 'param_estimator__n_estimators': masked_array(data=[60, 20, 20, 80, 70, 60, 90, 30, 100, 60],
   

In [37]:
#learning_rate,n_estimators,max_depth,min_samples_split,max_features,min_samples_leaf,subsample
gbm1 = GBM_model(x_train_norm,y_train,0.025,80,13,100,4,40,0.5)

Duration: 0:00:01.759754


gbm1 = GBM_model(x_train_norm,y_train,0.05,40,11,100,3,20,0.75)

In [38]:
y_pred= gbm1.predict(x_test_norm)
eval_metrics(y_test,y_pred)

MAE:  10.9702
MSE:  240.0701
RMSE:  15.4942
MAPE:  23.3709
Coefficient of determination (R-squared):  0.0305




(10.970181992737823,
 240.0700999079805,
 15.494195684448435,
 23.37091640778767,
 0.030457104083983527)

In [76]:
#ignore
y_pred= gbm1.predict(x_test_norm)
eval_metrics(y_test,y_pred)

MAE:  10.336479935867416
MSE:  219.23085211778678
RMSE:  14.806446302802938
MAPE:  21.516831192810017
Coefficient of determination (R-squared):  0.05940901220700426


### XGBoost

In [39]:
XGBM_training(x_train_norm,y_train)

Duration: 0:01:51.045160



{'mean_fit_time': array([15.91255999, 10.07287169, 28.41388776,  3.0174392 ,  5.32399516,
        7.52151113, 11.77119944, 20.41493816,  9.53645682,  2.3868947 ]), 'std_fit_time': array([0.7918568 , 2.71886455, 2.99411485, 0.35035348, 0.81735261,
       0.97848383, 1.28477397, 1.91620621, 1.23844062, 0.20455337]), 'mean_score_time': array([0.09829876, 0.08668325, 0.0848738 , 0.07481725, 0.06781745,
       0.07928717, 0.09177375, 0.08537049, 0.07131643, 0.03946774]), 'std_score_time': array([0.01712516, 0.0329322 , 0.01338421, 0.01714961, 0.0207691 ,
       0.01804158, 0.01065337, 0.01800665, 0.01775772, 0.00963258]), 'param_estimator__subsample': masked_array(data=[0.5, 0.7000000000000001, 0.9, 0.6, 0.9, 1.0,
                   0.30000000000000004, 1.0, 0.30000000000000004, 0.6],
             mask=[False, False, False, False, False, False, False, False,
                   False, False],
       fill_value='?',
            dtype=object), 'param_estimator__n_es

In [40]:
#learning_rate,n_estimators,max_depth,subsample,colsample_bytree#
xgb1 = XGBM_model(x_train_norm,y_train,0.1,150,13,1.0,0.2)
#xgb1 = XGBM_model(x_train_norm,y_train,0.025,350,11,0.1,0.4)

Duration: 0:00:04.183839


In [41]:
y_pred= xgb1.predict(x_test_norm)
eval_metrics(y_test,y_pred)

MAE:  10.2978
MSE:  221.0494
RMSE:  14.8677
MAPE:  21.9385
Coefficient of determination (R-squared):  0.1731




(10.29779084445889,
 221.04939457637047,
 14.867729973885403,
 21.938451811469566,
 0.17310823145367807)

In [42]:
#ignore
y_pred= xgb1.predict(x_test_norm)
eval_metrics(y_test,y_pred)

MAE:  10.2978
MSE:  221.0494
RMSE:  14.8677
MAPE:  21.9385
Coefficient of determination (R-squared):  0.1731




(10.29779084445889,
 221.04939457637047,
 14.867729973885403,
 21.938451811469566,
 0.17310823145367807)

### Light GBM

In [43]:
LGBM_training(x_train_norm,y_train)

[LightGBM] [Warning] min_child_samples is set=20, min_child_samples= will be ignored. Current value: min_child_samples=20
[LightGBM] [Warning] Unknown parameter: 60
Duration: 0:00:39.961338



{'mean_fit_time': array([15.18445385,  3.89356411,  2.71869864,  1.03403666,  5.23055911,
        4.54685934,  1.46727183,  2.83700535,  0.92498286,  1.15820942]), 'std_fit_time': array([1.17189644, 0.64028712, 0.32344241, 0.09924017, 0.60652214,
       0.69297017, 0.18222613, 0.29246565, 0.13193398, 0.06769923]), 'mean_score_time': array([0.03406823, 0.04046121, 0.04451714, 0.01940808, 0.05301402,
       0.06707592, 0.0241622 , 0.04542282, 0.02290843, 0.02260563]), 'std_score_time': array([0.00868413, 0.0122303 , 0.00829923, 0.0021161 , 0.00335432,
       0.01400345, 0.00469179, 0.01748948, 0.00559624, 0.00950128]), 'param_estimator__subsample': masked_array(data=[0.7000000000000001, 0.7000000000000001,
                   0.30000000000000004, 0.4, 0.4, 0.9, 0.7000000000000001,
                  

In [44]:
#def LGBM_model(X,y,num_leaves,min_child_samples,learning_rate,n_estimators,max_depth,subsample,colsample_bytree)
lgbm1 = LGBM_model(x_train_norm,y_train,77,60,0.0125,200,9,0.9,0.2)
#lgbm1 = LGBM_model(x_train_norm,y_train,4915,60,0.0125,200,13,0.3,0.2)

Duration: 0:00:01.326786


In [45]:
y_pred= lgbm1.predict(x_test_norm)
eval_metrics(y_test,y_pred)

MAE:  10.7466
MSE:  233.3923
RMSE:  15.2772
MAPE:  22.8945
Coefficient of determination (R-squared):  0.0681




(10.746568844936418,
 233.39226403979407,
 15.277181154905314,
 22.89453012828852,
 0.06805360814658912)

In [201]:
#ignore
y_pred= lgbm1.predict(x_test_norm)
eval_metrics(y_test,y_pred)

MAE:  10.788129573365048
MSE:  229.0761142286291
RMSE:  15.135260626385959
MAPE:  22.45700319223594
Coefficient of determination (R-squared):  0.019239234869801145




(10.788129573365048,
 229.0761142286291,
 15.135260626385959,
 22.45700319223594,
 0.019239234869801145)

### Testing on 2018-19 data

In [50]:
data_1819.head()

,applicable_date,elec_prices,rain_d,temp_d,wdsp_d,wddir_d,rain_c,temp_c,wdsp_c,wddir_c,rain_g,temp_g,wdsp_g,wddir_g,daily_oil_price,daily_natural_gas
0,2018-09-30 22:00:00,71.267,0.0,6.4,5.140,290,0.0,8.2,3.084,340,0.0,5.7,1.028,300,78.89,2.78616
1,2018-09-30 23:00:00,67.212,0.0,5.4,5.140,280,0.0,8.1,3.598,320,0.0,3.9,1.542,290,78.89,2.78616
2,2018-10-01 00:00:00,60.500,0.0,5.2,5.140,280,0.0,7.7,4.112,320,0.0,6.2,1.542,270,84.94,2.79509
3,2018-10-01 01:00:00,63.682,0.0,5.2,5.654,280,0.0,7.4,3.084,320,0.0,7.2,2.056,270,84.94,2.79509
4,2018-10-01 02:00:00,71.617,0.0,4.8,4.626,270,0.0,5.3,1.542,200,0.0,6.9,2.056,280,84.94,2.79509


In [51]:
data_1819['applicable_date'].min(),data_1819['applicable_date'].max()

(Timestamp('2018-09-30 22:00:00'), Timestamp('2019-12-12 23:00:00'))

In [52]:
basic_1819 = data_1819.copy()
basic_1819= feature_pipeline_1(basic_1819)

In [53]:
print(basic_1819.shape)
basic_1819.head()

(10514, 6)


,applicable_date,elec_prices,month,dayofyear_num,weekofyear_num,Hour
0,2018-09-30 22:00:00,71.267,9,273,39,22
1,2018-09-30 23:00:00,67.212,9,273,39,23
2,2018-10-01 00:00:00,60.500,10,274,40,0
3,2018-10-01 01:00:00,63.682,10,274,40,1
4,2018-10-01 02:00:00,71.617,10,274,40,2


In [55]:
basic_1819 = multi_step(basic_1819,'elec_prices')
basic_1819.head()

,applicable_date,elec_prices,month,dayofyear_num,weekofyear_num,Hour,t+1,t+2,t+3,t+4,...,t+14,t+15,t+16,t+17,t+18,t+19,t+20,t+21,t+22,t+23
0,2018-09-30 22:00:00,71.267,9,273,39,22,67.212,60.500,63.682,71.617,...,80.946,75.020,75.213,82.598,89.689,96.100,100.000,79.870,70.506,63.889
1,2018-09-30 23:00:00,67.212,9,273,39,23,60.500,63.682,71.617,72.855,...,75.020,75.213,82.598,89.689,96.100,100.000,79.870,70.506,63.889,62.004
2,2018-10-01 00:00:00,60.500,10,274,40,0,63.682,71.617,72.855,76.937,...,75.213,82.598,89.689,96.100,100.000,79.870,70.506,63.889,62.004,56.940
3,2018-10-01 01:00:00,63.682,10,274,40,1,71.617,72.855,76.937,80.480,...,82.598,89.689,96.100,100.000,79.870,70.506,63.889,62.004,56.940,49.323
4,2018-10-01 02:00:00,71.617,10,274,40,2,72.855,76.937,80.480,89.713,...,89.689,96.100,100.000,79.870,70.506,63.889,62.004,56.940,49.323,46.304


In [56]:
basic_1819 = calc_24lags(basic_1819,'elec_prices')
print(basic_1819.shape)
print(basic_1819.columns)
basic_1819.head()

(10514, 53)
Index(['applicable_date', 'elec_prices', 'month', 'dayofyear_num',
       'weekofyear_num', 'Hour', 't+1', 't+2', 't+3', 't+4', 't+5', 't+6',
       't+7', 't+8', 't+9', 't+10', 't+11', 't+12', 't+13', 't+14', 't+15',
       't+16', 't+17', 't+18', 't+19', 't+20', 't+21', 't+22', 't+23',
       'lag_1_elec_prices', 'lag_2_elec_prices', 'lag_3_elec_prices',
       'lag_4_elec_prices', 'lag_5_elec_prices', 'lag_6_elec_prices',
       'lag_7_elec_prices', 'lag_8_elec_prices', 'lag_9_elec_prices',
       'lag_10_elec_prices', 'lag_11_elec_prices', 'lag_12_elec_prices',
       'lag_13_elec_prices', 'lag_14_elec_prices', 'lag_15_elec_prices',
       'lag_16_elec_prices', 'lag_17_elec_prices', 'lag_18_elec_prices',
       'lag_19_elec_prices', 'lag_20_elec_prices', 'lag_21_elec_prices',
       'lag_22_elec_prices', 'lag_23_elec_prices', 'lag_24_elec_prices'],
      dtype='object')


,applicable_date,elec_prices,month,dayofyear_num,weekofyear_num,Hour,t+1,t+2,t+3,t+4,...,lag_15_elec_prices,lag_16_elec_prices,lag_17_elec_prices,lag_18_elec_prices,lag_19_elec_prices,lag_20_elec_prices,lag_21_elec_prices,lag_22_elec_prices,lag_23_elec_prices,lag_24_elec_prices
0,2018-09-30 22:00:00,71.267,9,273,39,22,67.212,60.500,63.682,71.617,...,71.267,71.267,71.267,71.267,71.267,71.267,71.267,71.267,71.267,71.267
1,2018-09-30 23:00:00,67.212,9,273,39,23,60.500,63.682,71.617,72.855,...,71.267,71.267,71.267,71.267,71.267,71.267,71.267,71.267,71.267,71.267
2,2018-10-01 00:00:00,60.500,10,274,40,0,63.682,71.617,72.855,76.937,...,71.267,71.267,71.267,71.267,71.267,71.267,71.267,71.267,71.267,71.267
3,2018-10-01 01:00:00,63.682,10,274,40,1,71.617,72.855,76.937,80.480,...,71.267,71.267,71.267,71.267,71.267,71.267,71.267,71.267,71.267,71.267
4,2018-10-01 02:00:00,71.617,10,274,40,2,72.855,76.937,80.480,89.713,...,71.267,71.267,71.267,71.267,71.267,71.267,71.267,71.267,71.267,71.267


In [57]:
#Dropping rows with Nan's
basic_1819 = basic_1819.dropna(axis=0)
print(basic_1819.shape)

(10491, 53)


In [58]:
basic_1819.isnull().values.any()

False

In [59]:
basic_1819 = remove_23rows_hour_col(basic_1819)

(438, 52)


In [60]:
basic_1819.head()

,applicable_date,elec_prices,month,dayofyear_num,weekofyear_num,t+1,t+2,t+3,t+4,t+5,...,lag_15_elec_prices,lag_16_elec_prices,lag_17_elec_prices,lag_18_elec_prices,lag_19_elec_prices,lag_20_elec_prices,lag_21_elec_prices,lag_22_elec_prices,lag_23_elec_prices,lag_24_elec_prices
0,2018-09-30 23:00:00,67.212,9,273,39,60.500,63.682,71.617,72.855,76.937,...,71.267,71.267,71.267,71.267,71.267,71.267,71.267,71.267,71.267,71.267
1,2018-10-01 23:00:00,56.940,10,274,40,49.323,46.304,46.060,48.305,59.640,...,101.160,127.400,89.713,80.480,76.937,72.855,71.617,63.682,60.500,67.212
2,2018-10-02 23:00:00,62.290,10,275,40,63.340,60.600,70.700,70.700,75.151,...,79.149,81.300,72.140,68.376,59.640,48.305,46.060,46.304,49.323,56.940
3,2018-10-03 23:00:00,60.350,10,276,40,59.752,58.404,60.350,60.350,72.862,...,91.121,100.436,83.740,82.666,75.151,70.700,70.700,60.600,63.340,62.290
4,2018-10-04 23:00:00,70.000,10,277,40,67.040,65.059,63.556,63.556,69.902,...,79.135,86.294,81.184,68.610,72.862,60.350,60.350,58.404,59.752,60.350


In [61]:
#selecting columns for X & y
cols = basic_1819.columns.to_list()
x_names = cols[2:5]
lags = cols[-24:]
x_names.extend(lags)
y_names = [cols[1]]
time_steps = cols[5:28]
y_names.extend(time_steps)
print(x_names)
print(y_names)

['month', 'dayofyear_num', 'weekofyear_num', 'lag_1_elec_prices', 'lag_2_elec_prices', 'lag_3_elec_prices', 'lag_4_elec_prices', 'lag_5_elec_prices', 'lag_6_elec_prices', 'lag_7_elec_prices', 'lag_8_elec_prices', 'lag_9_elec_prices', 'lag_10_elec_prices', 'lag_11_elec_prices', 'lag_12_elec_prices', 'lag_13_elec_prices', 'lag_14_elec_prices', 'lag_15_elec_prices', 'lag_16_elec_prices', 'lag_17_elec_prices', 'lag_18_elec_prices', 'lag_19_elec_prices', 'lag_20_elec_prices', 'lag_21_elec_prices', 'lag_22_elec_prices', 'lag_23_elec_prices', 'lag_24_elec_prices']
['elec_prices', 't+1', 't+2', 't+3', 't+4', 't+5', 't+6', 't+7', 't+8', 't+9', 't+10', 't+11', 't+12', 't+13', 't+14', 't+15', 't+16', 't+17', 't+18', 't+19', 't+20', 't+21', 't+22', 't+23']


In [62]:
#GBM
mae,mse,rmse,mape,r2 = testing_runs(10,basic_1819,gbm1)

Round 1: 
(350, 27) (44, 27) (44, 27)
(350, 24) (44, 24) (44, 24)
MAE:  11.7933
MSE:  390.3534
RMSE:  19.7574
MAPE:  20.6962
Coefficient of determination (R-squared):  0.4113


Round 2: 
(350, 27) (44, 27) (44, 27)
(350, 24) (44, 24) (44, 24)
MAE:  9.6382
MSE:  207.5779
RMSE:  14.4076
MAPE:  17.7482
Coefficient of determination (R-squared):  0.4098


Round 3: 
(350, 27) (44, 27) (44, 27)
(350, 24) (44, 24) (44, 24)
MAE:  9.9302
MSE:  209.7732
RMSE:  14.4836
MAPE:  19.0942
Coefficient of determination (R-squared):  0.5219


Round 4: 
(350, 27) (44, 27) (44, 27)
(350, 24) (44, 24) (44, 24)
MAE:  9.6918
MSE:  179.6204
RMSE:  13.4023
MAPE:  17.7092
Coefficient of determination (R-squared):  0.4022


Round 5: 
(350, 27) (44, 27) (44, 27)
(350, 24) (44, 24) (44, 24)
MAE:  12.2936
MSE:  383.3449
RMSE:  19.5792
MAPE:  20.8617
Coefficient of determination (R-squared):  0.3609


Round 6: 
(350, 27) (44, 27) (44, 27)
(350, 24) (44, 24) (44, 24)
MAE:  12.0877
MSE:  448.6856
RMSE:  21.1822
MAPE:  2

In [63]:
#XGBM
mae,mse,rmse,mape,r2 = testing_runs(10,basic_1819,xgb1)

Round 1: 
(350, 27) (44, 27) (44, 27)
(350, 24) (44, 24) (44, 24)
MAE:  11.037
MSE:  291.3195
RMSE:  17.0681
MAPE:  20.331
Coefficient of determination (R-squared):  0.4437


Round 2: 
(350, 27) (44, 27) (44, 27)
(350, 24) (44, 24) (44, 24)
MAE:  10.621
MSE:  204.2923
RMSE:  14.2931
MAPE:  20.8453
Coefficient of determination (R-squared):  0.4146


Round 3: 
(350, 27) (44, 27) (44, 27)
(350, 24) (44, 24) (44, 24)
MAE:  9.1497
MSE:  191.0297
RMSE:  13.8214
MAPE:  16.3747
Coefficient of determination (R-squared):  0.5167


Round 4: 
(350, 27) (44, 27) (44, 27)
(350, 24) (44, 24) (44, 24)
MAE:  9.9042
MSE:  260.9991
RMSE:  16.1555
MAPE:  18.0854
Coefficient of determination (R-squared):  0.4814


Round 5: 
(350, 27) (44, 27) (44, 27)
(350, 24) (44, 24) (44, 24)
MAE:  10.8707
MSE:  295.4117
RMSE:  17.1875
MAPE:  20.7485
Coefficient of determination (R-squared):  0.3984


Round 6: 
(350, 27) (44, 27) (44, 27)
(350, 24) (44, 24) (44, 24)
MAE:  11.8454
MSE:  403.3171
RMSE:  20.0828
MAPE:  21.

In [64]:
#light GBM
mae,mse,rmse,mape,r2 = testing_runs(10,basic_1819,lgbm1)

Round 1: 
(350, 27) (44, 27) (44, 27)
(350, 24) (44, 24) (44, 24)
MAE:  10.5499
MSE:  222.8787
RMSE:  14.9291
MAPE:  20.3354
Coefficient of determination (R-squared):  0.3297


Round 2: 
(350, 27) (44, 27) (44, 27)
(350, 24) (44, 24) (44, 24)
MAE:  10.5287
MSE:  283.9699
RMSE:  16.8514
MAPE:  19.6321
Coefficient of determination (R-squared):  0.4723


Round 3: 
(350, 27) (44, 27) (44, 27)
(350, 24) (44, 24) (44, 24)
MAE:  10.331
MSE:  253.1667
RMSE:  15.9112
MAPE:  19.5237
Coefficient of determination (R-squared):  0.3782


Round 4: 
(350, 27) (44, 27) (44, 27)
(350, 24) (44, 24) (44, 24)
MAE:  11.7069
MSE:  349.2773
RMSE:  18.689
MAPE:  20.0044
Coefficient of determination (R-squared):  0.3885


Round 5: 
(350, 27) (44, 27) (44, 27)
(350, 24) (44, 24) (44, 24)
MAE:  10.7447
MSE:  278.0755
RMSE:  16.6756
MAPE:  18.5395
Coefficient of determination (R-squared):  0.4806


Round 6: 
(350, 27) (44, 27) (44, 27)
(350, 24) (44, 24) (44, 24)
MAE:  11.7586
MSE:  306.0003
RMSE:  17.4929
MAPE:  

## Basic time based features + expanding_window_price

In [46]:
basic_time= data.copy()
basic_time = feature_pipeline_1(basic_time)
basic_time.head()

,applicable_date,elec_prices,month,dayofyear_num,weekofyear_num,Hour
0,2019-01-01 00:00:00,70.28,1,1,1,0
1,2019-01-01 01:00:00,65.18,1,1,1,1
2,2019-01-01 02:00:00,60.91,1,1,1,2
3,2019-01-01 03:00:00,65.16,1,1,1,3
4,2019-01-01 04:00:00,65.16,1,1,1,4


In [47]:
basic_time['expanding_window_price'] = basic_time['elec_prices'].expanding().mean()
basic_time.head()

,applicable_date,elec_prices,month,dayofyear_num,weekofyear_num,Hour,expanding_window_price
0,2019-01-01 00:00:00,70.28,1,1,1,0,70.280000
1,2019-01-01 01:00:00,65.18,1,1,1,1,67.730000
2,2019-01-01 02:00:00,60.91,1,1,1,2,65.456667
3,2019-01-01 03:00:00,65.16,1,1,1,3,65.382500
4,2019-01-01 04:00:00,65.16,1,1,1,4,65.338000


In [48]:
basic_time = multi_step(basic_time,'elec_prices')
basic_time.head()

,applicable_date,elec_prices,month,dayofyear_num,weekofyear_num,Hour,expanding_window_price,t+1,t+2,t+3,...,t+14,t+15,t+16,t+17,t+18,t+19,t+20,t+21,t+22,t+23
0,2019-01-01 00:00:00,70.28,1,1,1,0,70.280000,65.18,60.91,65.160,...,100.278,110.33,185.0,270.0,250.0,148.00,90.000,66.000,89.000,76.090
1,2019-01-01 01:00:00,65.18,1,1,1,1,67.730000,60.91,65.16,65.160,...,110.330,185.00,270.0,250.0,148.0,90.00,66.000,89.000,76.090,76.205
2,2019-01-01 02:00:00,60.91,1,1,1,2,65.456667,65.16,65.16,68.290,...,185.000,270.00,250.0,148.0,90.0,66.00,89.000,76.090,76.205,73.664
3,2019-01-01 03:00:00,65.16,1,1,1,3,65.382500,65.16,68.29,63.030,...,270.000,250.00,148.0,90.0,66.0,89.00,76.090,76.205,73.664,71.380
4,2019-01-01 04:00:00,65.16,1,1,1,4,65.338000,68.29,63.03,51.621,...,250.000,148.00,90.0,66.0,89.0,76.09,76.205,73.664,71.380,71.380


In [49]:
basic_time = calc_24lags(basic_time,'elec_prices')
print(basic_time.shape)
print(basic_time.columns)
basic_time.head()

(8304, 54)
Index(['applicable_date', 'elec_prices', 'month', 'dayofyear_num',
       'weekofyear_num', 'Hour', 'expanding_window_price', 't+1', 't+2', 't+3',
       't+4', 't+5', 't+6', 't+7', 't+8', 't+9', 't+10', 't+11', 't+12',
       't+13', 't+14', 't+15', 't+16', 't+17', 't+18', 't+19', 't+20', 't+21',
       't+22', 't+23', 'lag_1_elec_prices', 'lag_2_elec_prices',
       'lag_3_elec_prices', 'lag_4_elec_prices', 'lag_5_elec_prices',
       'lag_6_elec_prices', 'lag_7_elec_prices', 'lag_8_elec_prices',
       'lag_9_elec_prices', 'lag_10_elec_prices', 'lag_11_elec_prices',
       'lag_12_elec_prices', 'lag_13_elec_prices', 'lag_14_elec_prices',
       'lag_15_elec_prices', 'lag_16_elec_prices', 'lag_17_elec_prices',
       'lag_18_elec_prices', 'lag_19_elec_prices', 'lag_20_elec_prices',
       'lag_21_elec_prices', 'lag_22_elec_prices', 'lag_23_elec_prices',
       'lag_24_elec_prices'],
      dtype='object')


,applicable_date,elec_prices,month,dayofyear_num,weekofyear_num,Hour,expanding_window_price,t+1,t+2,t+3,...,lag_15_elec_prices,lag_16_elec_prices,lag_17_elec_prices,lag_18_elec_prices,lag_19_elec_prices,lag_20_elec_prices,lag_21_elec_prices,lag_22_elec_prices,lag_23_elec_prices,lag_24_elec_prices
0,2019-01-01 00:00:00,70.28,1,1,1,0,70.280000,65.18,60.91,65.160,...,70.28,70.28,70.28,70.28,70.28,70.28,70.28,70.28,70.28,70.28
1,2019-01-01 01:00:00,65.18,1,1,1,1,67.730000,60.91,65.16,65.160,...,70.28,70.28,70.28,70.28,70.28,70.28,70.28,70.28,70.28,70.28
2,2019-01-01 02:00:00,60.91,1,1,1,2,65.456667,65.16,65.16,68.290,...,70.28,70.28,70.28,70.28,70.28,70.28,70.28,70.28,70.28,70.28
3,2019-01-01 03:00:00,65.16,1,1,1,3,65.382500,65.16,68.29,63.030,...,70.28,70.28,70.28,70.28,70.28,70.28,70.28,70.28,70.28,70.28
4,2019-01-01 04:00:00,65.16,1,1,1,4,65.338000,68.29,63.03,51.621,...,70.28,70.28,70.28,70.28,70.28,70.28,70.28,70.28,70.28,70.28


In [50]:
#Dropping rows with Nan's
basic_time = basic_time.dropna(axis=0)
print(basic_time.shape)

(8281, 54)


In [51]:
basic_time.isnull().values.any()

False

In [52]:
basic_time = remove_23rows_hour_col(basic_time)

(345, 53)


In [53]:
basic_time.head()

,applicable_date,elec_prices,month,dayofyear_num,weekofyear_num,expanding_window_price,t+1,t+2,t+3,t+4,...,lag_15_elec_prices,lag_16_elec_prices,lag_17_elec_prices,lag_18_elec_prices,lag_19_elec_prices,lag_20_elec_prices,lag_21_elec_prices,lag_22_elec_prices,lag_23_elec_prices,lag_24_elec_prices
0,2019-01-01 23:00:00,76.09,1,1,1,97.003167,76.205,73.664,71.380,71.380,...,51.632,51.621,63.030,68.290,65.16,65.160,60.910,65.180,70.280,70.28
1,2019-01-02 23:00:00,71.01,1,2,1,109.101250,70.720,66.720,66.720,62.720,...,77.520,75.000,71.380,71.380,66.98,71.380,71.380,73.664,76.205,76.09
2,2019-01-03 23:00:00,72.69,1,3,1,106.858903,68.690,69.000,59.286,59.195,...,81.746,75.203,69.801,62.720,62.72,62.720,66.720,66.720,70.720,71.01
3,2019-01-04 23:00:00,82.99,1,4,1,108.463833,72.570,72.570,72.570,72.570,...,85.859,75.501,72.940,65.432,58.40,59.195,59.286,69.000,68.690,72.69
4,2019-01-05 23:00:00,72.95,1,5,1,106.046958,72.950,65.582,62.074,68.380,...,67.628,63.731,63.894,69.000,67.56,72.570,72.570,72.570,72.570,82.99


In [54]:
train,test = train_test_split_tbs(basic_time)

(276, 53)
(69, 53)


In [55]:
train.columns

Index(['applicable_date', 'elec_prices', 'month', 'dayofyear_num',
       'weekofyear_num', 'expanding_window_price', 't+1', 't+2', 't+3', 't+4',
       't+5', 't+6', 't+7', 't+8', 't+9', 't+10', 't+11', 't+12', 't+13',
       't+14', 't+15', 't+16', 't+17', 't+18', 't+19', 't+20', 't+21', 't+22',
       't+23', 'lag_1_elec_prices', 'lag_2_elec_prices', 'lag_3_elec_prices',
       'lag_4_elec_prices', 'lag_5_elec_prices', 'lag_6_elec_prices',
       'lag_7_elec_prices', 'lag_8_elec_prices', 'lag_9_elec_prices',
       'lag_10_elec_prices', 'lag_11_elec_prices', 'lag_12_elec_prices',
       'lag_13_elec_prices', 'lag_14_elec_prices', 'lag_15_elec_prices',
       'lag_16_elec_prices', 'lag_17_elec_prices', 'lag_18_elec_prices',
       'lag_19_elec_prices', 'lag_20_elec_prices', 'lag_21_elec_prices',
       'lag_22_elec_prices', 'lag_23_elec_prices', 'lag_24_elec_prices'],
      dtype='object')

In [56]:
#selecting columns for X & y
cols = train.columns.to_list()
x_names = cols[2:6]
lags = cols[-24:]
x_names.extend(lags)
y_names = [cols[1]]
time_steps = cols[6:29]
y_names.extend(time_steps)
print(x_names)
print(y_names)

['month', 'dayofyear_num', 'weekofyear_num', 'expanding_window_price', 'lag_1_elec_prices', 'lag_2_elec_prices', 'lag_3_elec_prices', 'lag_4_elec_prices', 'lag_5_elec_prices', 'lag_6_elec_prices', 'lag_7_elec_prices', 'lag_8_elec_prices', 'lag_9_elec_prices', 'lag_10_elec_prices', 'lag_11_elec_prices', 'lag_12_elec_prices', 'lag_13_elec_prices', 'lag_14_elec_prices', 'lag_15_elec_prices', 'lag_16_elec_prices', 'lag_17_elec_prices', 'lag_18_elec_prices', 'lag_19_elec_prices', 'lag_20_elec_prices', 'lag_21_elec_prices', 'lag_22_elec_prices', 'lag_23_elec_prices', 'lag_24_elec_prices']
['elec_prices', 't+1', 't+2', 't+3', 't+4', 't+5', 't+6', 't+7', 't+8', 't+9', 't+10', 't+11', 't+12', 't+13', 't+14', 't+15', 't+16', 't+17', 't+18', 't+19', 't+20', 't+21', 't+22', 't+23']


In [57]:
x_train, y_train = train[x_names], train[y_names]
x_test, y_test = test[x_names], test[y_names]
print(x_train.shape,y_train.shape)
print(x_test.shape,y_test.shape)

(276, 28) (276, 24)
(69, 28) (69, 24)


In [58]:
x_train_norm,x_test_norm = normalize(x_train,x_test)

In [59]:
GBM_training(x_train_norm,y_train)

Duration: 0:00:25.463731



{'mean_fit_time': array([2.70354159, 1.66691356, 2.19709337, 1.73477142, 2.88986394,
       2.1110338 , 1.60245035, 2.05201266, 2.40272028, 1.48525157]), 'std_fit_time': array([0.22635616, 0.31447928, 0.30645788, 0.15165288, 0.26424072,
       0.31578876, 0.30633972, 0.35906586, 0.47673326, 0.22893958]), 'mean_score_time': array([0.02606387, 0.03036251, 0.03223083, 0.02857041, 0.02355709,
       0.0274586 , 0.02597933, 0.02476583, 0.02616324, 0.01299677]), 'std_score_time': array([0.00563135, 0.01041011, 0.00769132, 0.01076183, 0.00318022,
       0.00944705, 0.01099383, 0.00780355, 0.01086683, 0.00593315]), 'param_estimator__subsample': masked_array(data=[0.75, 0.85, 0.9, 0.8, 0.9, 0.9, 0.9, 0.8, 0.6, 0.9],
             mask=[False, False, False, False, False, False, False, False,
                   False, False],
       fill_value='?',
            dtype=object), 'param_estimator__n_estimators': masked_array(data=[100, 50, 70, 60, 60, 90, 70, 90, 100, 70],
 

In [60]:
#def GBM_model(X,y,learning_rate,n_estimators,max_depth,min_samples_split,max_features,min_samples_leaf,subsample):
gbm2 = GBM_model(x_train_norm,y_train,0.1,60,5,100,4,10,0.9)

Duration: 0:00:02.052405


In [61]:
y_pred= gbm2.predict(x_test_norm)
eval_metrics(y_test,y_pred)

MAE:  10.9124
MSE:  238.4544
RMSE:  15.442
MAPE:  23.2477
Coefficient of determination (R-squared):  0.0726




(10.912360722086783,
 238.45440244836382,
 15.441968865671367,
 23.247733758322774,
 0.07263742297328739)

### XGBoost

In [62]:
XGBM_training(x_train_norm,y_train)

Duration: 0:01:29.496086



{'mean_fit_time': array([13.12885883,  4.32520573,  2.10976131, 29.07424715,  8.47018967,
        2.0567945 ,  8.85430827, 16.98453519,  1.82061868,  7.0487309 ]), 'std_fit_time': array([1.59863604, 1.45333374, 0.29196908, 2.26991101, 1.30625378,
       0.39604688, 0.95570111, 2.46703463, 0.27166294, 0.47148665]), 'mean_score_time': array([0.08361807, 0.08218007, 0.05787318, 0.0849273 , 0.08066919,
       0.05412147, 0.06861951, 0.08954563, 0.05681555, 0.07114663]), 'std_score_time': array([0.01624066, 0.02179665, 0.01649008, 0.00948712, 0.02188865,
       0.00780677, 0.02452405, 0.01191194, 0.0157818 , 0.03952564]), 'param_estimator__subsample': masked_array(data=[0.5, 0.1, 0.7000000000000001, 0.9, 0.8, 0.1, 0.9,
                   0.30000000000000004, 0.9, 0.6],
             mask=[False, False, False, False, False, False, False, False,
                   False, False],
       fill_value='?',
            dtype=object), 'param_estimator__n_estimators': maske

In [63]:
# def XGBM_model(X,y,learning_rate,n_estimators,max_depth,subsample,colsample_bytree):
xgb2 = XGBM_model(x_train_norm,y_train,0.1,150,11,0.8,0.3)
#xgb1 = XGBM_model(x_train_norm,y_train,0.025,350,11,0.1,0.4)

Duration: 0:00:03.777291


In [64]:
y_pred= xgb2.predict(x_test_norm)
eval_metrics(y_test,y_pred)

MAE:  11.2602
MSE:  250.2964
RMSE:  15.8208
MAPE:  23.9888
Coefficient of determination (R-squared):  0.0636




(11.2601899857475,
 250.29638741936608,
 15.820758117718825,
 23.988750511789426,
 0.06357708451135484)

### Light GBM

In [65]:
LGBM_training(x_train_norm,y_train)

[LightGBM] [Warning] min_child_samples is set=20, min_child_samples= will be ignored. Current value: min_child_samples=20
[LightGBM] [Warning] Unknown parameter: 40
Duration: 0:00:48.768477



{'mean_fit_time': array([ 1.67637215,  1.91490939, 23.69247291,  0.88125381,  3.59738166,
        5.07738419,  3.24622977,  2.65895312,  5.0014683 ,  4.54976645]), 'std_fit_time': array([0.22292785, 0.17167499, 0.52114263, 0.07872797, 0.2310289 ,
       0.37060958, 0.22940952, 0.16328095, 0.34056397, 0.46557326]), 'mean_score_time': array([0.03346357, 0.04682031, 0.05943923, 0.05192628, 0.07319522,
       0.03026288, 0.05002191, 0.04847314, 0.05226867, 0.0386272 ]), 'std_score_time': array([0.00837147, 0.00943977, 0.00984157, 0.04572983, 0.00839155,
       0.00852058, 0.02379953, 0.0073865 , 0.00574805, 0.01419473]), 'param_estimator__subsample': masked_array(data=[0.2, 0.1, 0.9, 0.1, 0.6, 0.7000000000000001, 0.4,
                   0.30000000000000004, 0.5, 0.8],
             mask=[False, False,

In [66]:
#def LGBM_model(X,y,num_leaves,min_child_samples,learning_rate,n_estimators,max_depth,subsample,colsample_bytree)
lgbm2 = LGBM_model(x_train_norm,y_train,19,40,0.0125,100,5,0.2,0.3)
#lgbm1 = LGBM_model(x_train_norm,y_train,4915,60,0.0125,200,13,0.3,0.2)

Duration: 0:00:01.006247


In [67]:
y_pred= lgbm2.predict(x_test_norm)
eval_metrics(y_test,y_pred)

MAE:  10.6977
MSE:  231.8459
RMSE:  15.2265
MAPE:  22.7904
Coefficient of determination (R-squared):  0.0713




(10.697679138851958,
 231.84585654608804,
 15.226485364196428,
 22.79037531710511,
 0.07130972829070613)

### Testing on 2018-2019 data

In [92]:
basic_time = data_1819.copy()
basic_time = feature_pipeline_1(basic_time)
basic_time['expanding_window_price'] = basic_time['elec_prices'].expanding().mean()
basic_time = multi_step(basic_time,'elec_prices')
basic_time = calc_24lags(basic_time,'elec_prices')
print(basic_time.shape)
print(basic_time.columns)
basic_time.head()

(10514, 54)
Index(['applicable_date', 'elec_prices', 'month', 'dayofyear_num',
       'weekofyear_num', 'Hour', 'expanding_window_price', 't+1', 't+2', 't+3',
       't+4', 't+5', 't+6', 't+7', 't+8', 't+9', 't+10', 't+11', 't+12',
       't+13', 't+14', 't+15', 't+16', 't+17', 't+18', 't+19', 't+20', 't+21',
       't+22', 't+23', 'lag_1_elec_prices', 'lag_2_elec_prices',
       'lag_3_elec_prices', 'lag_4_elec_prices', 'lag_5_elec_prices',
       'lag_6_elec_prices', 'lag_7_elec_prices', 'lag_8_elec_prices',
       'lag_9_elec_prices', 'lag_10_elec_prices', 'lag_11_elec_prices',
       'lag_12_elec_prices', 'lag_13_elec_prices', 'lag_14_elec_prices',
       'lag_15_elec_prices', 'lag_16_elec_prices', 'lag_17_elec_prices',
       'lag_18_elec_prices', 'lag_19_elec_prices', 'lag_20_elec_prices',
       'lag_21_elec_prices', 'lag_22_elec_prices', 'lag_23_elec_prices',
       'lag_24_elec_prices'],
      dtype='object')


,applicable_date,elec_prices,month,dayofyear_num,weekofyear_num,Hour,expanding_window_price,t+1,t+2,t+3,...,lag_15_elec_prices,lag_16_elec_prices,lag_17_elec_prices,lag_18_elec_prices,lag_19_elec_prices,lag_20_elec_prices,lag_21_elec_prices,lag_22_elec_prices,lag_23_elec_prices,lag_24_elec_prices
0,2018-09-30 22:00:00,71.267,9,273,39,22,71.267000,67.212,60.500,63.682,...,71.267,71.267,71.267,71.267,71.267,71.267,71.267,71.267,71.267,71.267
1,2018-09-30 23:00:00,67.212,9,273,39,23,69.239500,60.500,63.682,71.617,...,71.267,71.267,71.267,71.267,71.267,71.267,71.267,71.267,71.267,71.267
2,2018-10-01 00:00:00,60.500,10,274,40,0,66.326333,63.682,71.617,72.855,...,71.267,71.267,71.267,71.267,71.267,71.267,71.267,71.267,71.267,71.267
3,2018-10-01 01:00:00,63.682,10,274,40,1,65.665250,71.617,72.855,76.937,...,71.267,71.267,71.267,71.267,71.267,71.267,71.267,71.267,71.267,71.267
4,2018-10-01 02:00:00,71.617,10,274,40,2,66.855600,72.855,76.937,80.480,...,71.267,71.267,71.267,71.267,71.267,71.267,71.267,71.267,71.267,71.267


In [93]:
#Dropping rows with Nan's
basic_time = basic_time.dropna(axis=0)
print(basic_time.shape)

(10491, 54)


In [94]:
basic_time.isnull().values.any()

False

In [95]:
basic_time = remove_23rows_hour_col(basic_time)

(438, 53)


In [96]:
basic_time.head()

,applicable_date,elec_prices,month,dayofyear_num,weekofyear_num,expanding_window_price,t+1,t+2,t+3,t+4,...,lag_15_elec_prices,lag_16_elec_prices,lag_17_elec_prices,lag_18_elec_prices,lag_19_elec_prices,lag_20_elec_prices,lag_21_elec_prices,lag_22_elec_prices,lag_23_elec_prices,lag_24_elec_prices
0,2018-09-30 23:00:00,67.212,9,273,39,69.239500,60.500,63.682,71.617,72.855,...,71.267,71.267,71.267,71.267,71.267,71.267,71.267,71.267,71.267,71.267
1,2018-10-01 23:00:00,56.940,10,274,40,80.490308,49.323,46.304,46.060,48.305,...,101.160,127.400,89.713,80.480,76.937,72.855,71.617,63.682,60.500,67.212
2,2018-10-02 23:00:00,62.290,10,275,40,75.592040,63.340,60.600,70.700,70.700,...,79.149,81.300,72.140,68.376,59.640,48.305,46.060,46.304,49.323,56.940
3,2018-10-03 23:00:00,60.350,10,276,40,76.635622,59.752,58.404,60.350,60.350,...,91.121,100.436,83.740,82.666,75.151,70.700,70.700,60.600,63.340,62.290
4,2018-10-04 23:00:00,70.000,10,277,40,76.759357,67.040,65.059,63.556,63.556,...,79.135,86.294,81.184,68.610,72.862,60.350,60.350,58.404,59.752,60.350


In [97]:
basic_time.columns

Index(['applicable_date', 'elec_prices', 'month', 'dayofyear_num',
       'weekofyear_num', 'expanding_window_price', 't+1', 't+2', 't+3', 't+4',
       't+5', 't+6', 't+7', 't+8', 't+9', 't+10', 't+11', 't+12', 't+13',
       't+14', 't+15', 't+16', 't+17', 't+18', 't+19', 't+20', 't+21', 't+22',
       't+23', 'lag_1_elec_prices', 'lag_2_elec_prices', 'lag_3_elec_prices',
       'lag_4_elec_prices', 'lag_5_elec_prices', 'lag_6_elec_prices',
       'lag_7_elec_prices', 'lag_8_elec_prices', 'lag_9_elec_prices',
       'lag_10_elec_prices', 'lag_11_elec_prices', 'lag_12_elec_prices',
       'lag_13_elec_prices', 'lag_14_elec_prices', 'lag_15_elec_prices',
       'lag_16_elec_prices', 'lag_17_elec_prices', 'lag_18_elec_prices',
       'lag_19_elec_prices', 'lag_20_elec_prices', 'lag_21_elec_prices',
       'lag_22_elec_prices', 'lag_23_elec_prices', 'lag_24_elec_prices'],
      dtype='object')

In [98]:
#selecting columns for X & y
cols = train.columns.to_list()
x_names = cols[2:6]
lags = cols[-24:]
x_names.extend(lags)
y_names = [cols[1]]
time_steps = cols[6:29]
y_names.extend(time_steps)
print(x_names)
print(y_names)

['month', 'dayofyear_num', 'weekofyear_num', 'expanding_window_price', 'lag_1_elec_prices', 'lag_2_elec_prices', 'lag_3_elec_prices', 'lag_4_elec_prices', 'lag_5_elec_prices', 'lag_6_elec_prices', 'lag_7_elec_prices', 'lag_8_elec_prices', 'lag_9_elec_prices', 'lag_10_elec_prices', 'lag_11_elec_prices', 'lag_12_elec_prices', 'lag_13_elec_prices', 'lag_14_elec_prices', 'lag_15_elec_prices', 'lag_16_elec_prices', 'lag_17_elec_prices', 'lag_18_elec_prices', 'lag_19_elec_prices', 'lag_20_elec_prices', 'lag_21_elec_prices', 'lag_22_elec_prices', 'lag_23_elec_prices', 'lag_24_elec_prices']
['elec_prices', 't+1', 't+2', 't+3', 't+4', 't+5', 't+6', 't+7', 't+8', 't+9', 't+10', 't+11', 't+12', 't+13', 't+14', 't+15', 't+16', 't+17', 't+18', 't+19', 't+20', 't+21', 't+22', 't+23']


In [99]:
#GBM
mae,mse,rmse,mape,r2 = testing_runs(10,basic_time,gbm2)

Round 1: 
(350, 28) (44, 28) (44, 28)
(350, 24) (44, 24) (44, 24)
MAE:  10.7827
MSE:  308.0257
RMSE:  17.5507
MAPE:  18.8089
Coefficient of determination (R-squared):  0.5069


Round 2: 
(350, 28) (44, 28) (44, 28)
(350, 24) (44, 24) (44, 24)
MAE:  9.2967
MSE:  195.9483
RMSE:  13.9982
MAPE:  17.2585
Coefficient of determination (R-squared):  0.5952


Round 3: 
(350, 28) (44, 28) (44, 28)
(350, 24) (44, 24) (44, 24)
MAE:  10.5817
MSE:  244.1585
RMSE:  15.6256
MAPE:  19.433
Coefficient of determination (R-squared):  0.4671


Round 4: 
(350, 28) (44, 28) (44, 28)
(350, 24) (44, 24) (44, 24)
MAE:  11.8821
MSE:  366.6835
RMSE:  19.149
MAPE:  20.3288
Coefficient of determination (R-squared):  0.4931


Round 5: 
(350, 28) (44, 28) (44, 28)
(350, 24) (44, 24) (44, 24)
MAE:  10.7171
MSE:  302.6193
RMSE:  17.396
MAPE:  17.7494
Coefficient of determination (R-squared):  0.5003


Round 6: 
(350, 28) (44, 28) (44, 28)
(350, 24) (44, 24) (44, 24)
MAE:  11.0235
MSE:  299.9669
RMSE:  17.3196
MAPE:  21

In [100]:
#XGBM
mae,mse,rmse,mape,r2 = testing_runs(10,basic_time,xgb2)

Round 1: 
(350, 28) (44, 28) (44, 28)
(350, 24) (44, 24) (44, 24)
MAE:  9.1121
MSE:  177.5315
RMSE:  13.3241
MAPE:  18.1391
Coefficient of determination (R-squared):  0.4345


Round 2: 
(350, 28) (44, 28) (44, 28)
(350, 24) (44, 24) (44, 24)
MAE:  10.9126
MSE:  319.4699
RMSE:  17.8737
MAPE:  19.7465
Coefficient of determination (R-squared):  0.3932


Round 3: 
(350, 28) (44, 28) (44, 28)
(350, 24) (44, 24) (44, 24)
MAE:  10.994
MSE:  264.2117
RMSE:  16.2546
MAPE:  20.0545
Coefficient of determination (R-squared):  0.3609


Round 4: 
(350, 28) (44, 28) (44, 28)
(350, 24) (44, 24) (44, 24)
MAE:  11.4178
MSE:  464.2735
RMSE:  21.547
MAPE:  20.0277
Coefficient of determination (R-squared):  0.4038


Round 5: 
(350, 28) (44, 28) (44, 28)
(350, 24) (44, 24) (44, 24)
MAE:  10.2229
MSE:  303.2295
RMSE:  17.4135
MAPE:  18.9108
Coefficient of determination (R-squared):  0.449


Round 6: 
(350, 28) (44, 28) (44, 28)
(350, 24) (44, 24) (44, 24)
MAE:  9.0985
MSE:  231.1921
RMSE:  15.205
MAPE:  17.5

In [101]:
#light GBM
mae,mse,rmse,mape,r2 = testing_runs(10,basic_time,lgbm2)

Round 1: 
(350, 28) (44, 28) (44, 28)
(350, 24) (44, 24) (44, 24)
MAE:  10.9565
MSE:  358.617
RMSE:  18.9372
MAPE:  20.1771
Coefficient of determination (R-squared):  0.3415


Round 2: 
(350, 28) (44, 28) (44, 28)
(350, 24) (44, 24) (44, 24)
MAE:  10.772
MSE:  242.3525
RMSE:  15.5677
MAPE:  18.4259
Coefficient of determination (R-squared):  0.3912


Round 3: 
(350, 28) (44, 28) (44, 28)
(350, 24) (44, 24) (44, 24)
MAE:  11.8372
MSE:  351.1117
RMSE:  18.738
MAPE:  20.8936
Coefficient of determination (R-squared):  0.3625


Round 4: 
(350, 28) (44, 28) (44, 28)
(350, 24) (44, 24) (44, 24)
MAE:  10.6814
MSE:  329.4314
RMSE:  18.1502
MAPE:  18.5816
Coefficient of determination (R-squared):  0.4152


Round 5: 
(350, 28) (44, 28) (44, 28)
(350, 24) (44, 24) (44, 24)
MAE:  13.8633
MSE:  525.7559
RMSE:  22.9294
MAPE:  25.4012
Coefficient of determination (R-squared):  0.3583


Round 6: 
(350, 28) (44, 28) (44, 28)
(350, 24) (44, 24) (44, 24)
MAE:  9.3112
MSE:  229.98
RMSE:  15.1651
MAPE:  18.0

<h1 style="text-align:center;font-size:40px;" > Basic time based features + wdsp_avg (windspeeds in all counties) </h1>

In [107]:
data.shape

(8304, 16)

In [108]:
data_1= data.copy()
data_1 = feature_pipeline_1(data_1)
data_1['wdsp_avg']= (data['wdsp_d']+data['wdsp_c']+data['wdsp_g'])/3
print(data_1.shape)
data_1.head()

(8304, 7)


,applicable_date,elec_prices,month,dayofyear_num,weekofyear_num,Hour,wdsp_avg
0,2019-01-01 00:00:00,70.28,1,1,1,0,2.912667
1,2019-01-01 01:00:00,65.18,1,1,1,1,2.741333
2,2019-01-01 02:00:00,60.91,1,1,1,2,2.741333
3,2019-01-01 03:00:00,65.16,1,1,1,3,3.084000
4,2019-01-01 04:00:00,65.16,1,1,1,4,3.255333


In [109]:
def calc_24lags(data,feature):
    if feature == 'elec_prices':
        for time_step in range(1,25):
            name = 'lag_'+str(time_step)+'_'+feature
            data[name] = data[feature].shift(time_step).bfill(axis ='rows')
    else:
        for time_step in range(1,24):
            name = 'lag_'+str(time_step)+'_'+feature
            data[name] = data[feature].shift(time_step).bfill(axis ='rows')
            
    return data

In [110]:
data_1 = multi_step(data_1,'elec_prices')
data_1 = calc_24lags(data_1,'elec_prices')
data_1 = calc_24lags(data_1,'wdsp_avg')
print(data_1.shape)

(8304, 77)


In [111]:
print(data_1.columns)
data_1.head()

Index(['applicable_date', 'elec_prices', 'month', 'dayofyear_num',
       'weekofyear_num', 'Hour', 'wdsp_avg', 't+1', 't+2', 't+3', 't+4', 't+5',
       't+6', 't+7', 't+8', 't+9', 't+10', 't+11', 't+12', 't+13', 't+14',
       't+15', 't+16', 't+17', 't+18', 't+19', 't+20', 't+21', 't+22', 't+23',
       'lag_1_elec_prices', 'lag_2_elec_prices', 'lag_3_elec_prices',
       'lag_4_elec_prices', 'lag_5_elec_prices', 'lag_6_elec_prices',
       'lag_7_elec_prices', 'lag_8_elec_prices', 'lag_9_elec_prices',
       'lag_10_elec_prices', 'lag_11_elec_prices', 'lag_12_elec_prices',
       'lag_13_elec_prices', 'lag_14_elec_prices', 'lag_15_elec_prices',
       'lag_16_elec_prices', 'lag_17_elec_prices', 'lag_18_elec_prices',
       'lag_19_elec_prices', 'lag_20_elec_prices', 'lag_21_elec_prices',
       'lag_22_elec_prices', 'lag_23_elec_prices', 'lag_24_elec_prices',
       'lag_1_wdsp_avg', 'lag_2_wdsp_avg', 'lag_3_wdsp_avg', 'lag_4_wdsp_avg',
       'lag_5_wdsp_avg', 'lag_6_wdsp_avg', 'l

,applicable_date,elec_prices,month,dayofyear_num,weekofyear_num,Hour,wdsp_avg,t+1,t+2,t+3,...,lag_14_wdsp_avg,lag_15_wdsp_avg,lag_16_wdsp_avg,lag_17_wdsp_avg,lag_18_wdsp_avg,lag_19_wdsp_avg,lag_20_wdsp_avg,lag_21_wdsp_avg,lag_22_wdsp_avg,lag_23_wdsp_avg
0,2019-01-01 00:00:00,70.28,1,1,1,0,2.912667,65.18,60.91,65.160,...,2.912667,2.912667,2.912667,2.912667,2.912667,2.912667,2.912667,2.912667,2.912667,2.912667
1,2019-01-01 01:00:00,65.18,1,1,1,1,2.741333,60.91,65.16,65.160,...,2.912667,2.912667,2.912667,2.912667,2.912667,2.912667,2.912667,2.912667,2.912667,2.912667
2,2019-01-01 02:00:00,60.91,1,1,1,2,2.741333,65.16,65.16,68.290,...,2.912667,2.912667,2.912667,2.912667,2.912667,2.912667,2.912667,2.912667,2.912667,2.912667
3,2019-01-01 03:00:00,65.16,1,1,1,3,3.084000,65.16,68.29,63.030,...,2.912667,2.912667,2.912667,2.912667,2.912667,2.912667,2.912667,2.912667,2.912667,2.912667
4,2019-01-01 04:00:00,65.16,1,1,1,4,3.255333,68.29,63.03,51.621,...,2.912667,2.912667,2.912667,2.912667,2.912667,2.912667,2.912667,2.912667,2.912667,2.912667


In [112]:
data_1.isnull().values.any()

True

In [113]:
#Dropping rows with Nan's
data_1 = data_1.dropna(axis=0)
print(data_1.shape)

(8281, 77)


In [114]:
data_1.isnull().values.any()

False

In [115]:
data_1 = remove_23rows_hour_col(data_1)
# print(data_1.shape)
train,test = train_test_split_tbs(data_1)
train.columns

(345, 76)
(276, 76)
(69, 76)


Index(['applicable_date', 'elec_prices', 'month', 'dayofyear_num',
       'weekofyear_num', 'wdsp_avg', 't+1', 't+2', 't+3', 't+4', 't+5', 't+6',
       't+7', 't+8', 't+9', 't+10', 't+11', 't+12', 't+13', 't+14', 't+15',
       't+16', 't+17', 't+18', 't+19', 't+20', 't+21', 't+22', 't+23',
       'lag_1_elec_prices', 'lag_2_elec_prices', 'lag_3_elec_prices',
       'lag_4_elec_prices', 'lag_5_elec_prices', 'lag_6_elec_prices',
       'lag_7_elec_prices', 'lag_8_elec_prices', 'lag_9_elec_prices',
       'lag_10_elec_prices', 'lag_11_elec_prices', 'lag_12_elec_prices',
       'lag_13_elec_prices', 'lag_14_elec_prices', 'lag_15_elec_prices',
       'lag_16_elec_prices', 'lag_17_elec_prices', 'lag_18_elec_prices',
       'lag_19_elec_prices', 'lag_20_elec_prices', 'lag_21_elec_prices',
       'lag_22_elec_prices', 'lag_23_elec_prices', 'lag_24_elec_prices',
       'lag_1_wdsp_avg', 'lag_2_wdsp_avg', 'lag_3_wdsp_avg', 'lag_4_wdsp_avg',
       'lag_5_wdsp_avg', 'lag_6_wdsp_avg', 'lag_7_wds

In [116]:
#selecting columns for X & y
cols = train.columns.to_list()
x_names = cols[2:6]
lags = cols[-47:]
x_names.extend(lags)
y_names = [cols[1]]
time_steps = cols[6:29]
y_names.extend(time_steps)
print(x_names)
print(y_names)

['month', 'dayofyear_num', 'weekofyear_num', 'wdsp_avg', 'lag_1_elec_prices', 'lag_2_elec_prices', 'lag_3_elec_prices', 'lag_4_elec_prices', 'lag_5_elec_prices', 'lag_6_elec_prices', 'lag_7_elec_prices', 'lag_8_elec_prices', 'lag_9_elec_prices', 'lag_10_elec_prices', 'lag_11_elec_prices', 'lag_12_elec_prices', 'lag_13_elec_prices', 'lag_14_elec_prices', 'lag_15_elec_prices', 'lag_16_elec_prices', 'lag_17_elec_prices', 'lag_18_elec_prices', 'lag_19_elec_prices', 'lag_20_elec_prices', 'lag_21_elec_prices', 'lag_22_elec_prices', 'lag_23_elec_prices', 'lag_24_elec_prices', 'lag_1_wdsp_avg', 'lag_2_wdsp_avg', 'lag_3_wdsp_avg', 'lag_4_wdsp_avg', 'lag_5_wdsp_avg', 'lag_6_wdsp_avg', 'lag_7_wdsp_avg', 'lag_8_wdsp_avg', 'lag_9_wdsp_avg', 'lag_10_wdsp_avg', 'lag_11_wdsp_avg', 'lag_12_wdsp_avg', 'lag_13_wdsp_avg', 'lag_14_wdsp_avg', 'lag_15_wdsp_avg', 'lag_16_wdsp_avg', 'lag_17_wdsp_avg', 'lag_18_wdsp_avg', 'lag_19_wdsp_avg', 'lag_20_wdsp_avg', 'lag_21_wdsp_avg', 'lag_22_wdsp_avg', 'lag_23_wdsp_av

In [117]:
x_train, y_train = train[x_names], train[y_names]
x_test, y_test = test[x_names], test[y_names]
print(x_train.shape,y_train.shape)
print(x_test.shape,y_test.shape)

x_train_norm,x_test_norm = normalize(x_train,x_test)

(276, 51) (276, 24)
(69, 51) (69, 24)


In [118]:
GBM_training(x_train_norm,y_train)

Duration: 0:00:14.792131



{'mean_fit_time': array([0.60650182, 0.57034974, 0.81497879, 1.96191082, 0.97501125,
       1.88959186, 1.64896467, 1.00195687, 1.10255868, 0.93178496]), 'std_fit_time': array([0.01332361, 0.03441236, 0.01889713, 0.06327263, 0.04855754,
       0.05554211, 0.07235526, 0.06486675, 0.07903151, 0.22109263]), 'mean_score_time': array([0.01718926, 0.01875143, 0.02500207, 0.02187645, 0.02187748,
       0.02031457, 0.0230721 , 0.02469065, 0.0272028 , 0.00917308]), 'std_score_time': array([0.00468715, 0.00625042, 0.0103652 , 0.00765493, 0.00765429,
       0.00716043, 0.00763392, 0.00661782, 0.01142649, 0.00949666]), 'param_estimator__subsample': masked_array(data=[0.75, 0.85, 0.85, 0.75, 0.75, 0.6, 0.7, 0.5, 0.85, 0.7],
             mask=[False, False, False, False, False, False, False, False,
                   False, False],
       fill_value='?',
            dtype=object), 'param_estimator__n_estimators': masked_array(data=[30, 20, 40, 100, 50, 100, 90, 50, 50, 50

In [119]:
#def GBM_model(X,y,learning_rate,n_estimators,max_depth,min_samples_split,max_features,min_samples_leaf,subsample):
gbm3 = GBM_model(x_train_norm,y_train,0.05,20,5,100,3,50,0.85)

Duration: 0:00:00.475471


In [120]:
y_pred= gbm3.predict(x_test_norm)
eval_metrics(y_test,y_pred)

MAE:  11.0217
MSE:  242.683
RMSE:  15.5783
MAPE:  23.4806
Coefficient of determination (R-squared):  0.0248




(11.021661412890703,
 242.6829585596589,
 15.578284840111857,
 23.48058835542817,
 0.02475018129748235)

In [121]:
XGBM_training(x_train_norm,y_train)

Duration: 0:01:53.682865



{'mean_fit_time': array([11.61541257,  5.83836038, 19.9743561 , 22.08520217,  8.68327539,
       13.24679494,  4.67459757, 16.27996471, 11.07272706, 10.10763762]), 'std_fit_time': array([0.44530778, 1.29128642, 0.76367741, 0.96291373, 0.23280351,
       0.12169413, 0.13452624, 0.32975326, 0.2036182 , 1.36301693]), 'mean_score_time': array([0.06320636, 0.05641682, 0.05285885, 0.08104284, 0.06801591,
       0.05283203, 0.04697964, 0.07570145, 0.04775982, 0.04380455]), 'std_score_time': array([0.00976668, 0.00935464, 0.00951606, 0.00800508, 0.00985732,
       0.00742435, 0.00637676, 0.00900953, 0.00264309, 0.01360614]), 'param_estimator__subsample': masked_array(data=[0.1, 0.1, 1.0, 0.2, 0.30000000000000004,
                   0.7000000000000001, 0.4, 1.0, 0.30000000000000004, 0.2],
             mask=[False, False, False, False, False, False, False, False,
                   False, False],
       fill_value='?',
            dtype=object), 'param_estimator__n_es

In [123]:
# def XGBM_model(X,y,learning_rate,n_estimators,max_depth,subsample,colsample_bytree):
xgb3 = XGBM_model(x_train_norm,y_train,0.0125,350,3,1.0,1.0)

Duration: 0:00:03.383415


In [124]:
y_pred= xgb3.predict(x_test_norm)
eval_metrics(y_test,y_pred)

MAE:  10.4369
MSE:  237.0256
RMSE:  15.3956
MAPE:  22.2348
Coefficient of determination (R-squared):  0.1451




(10.43687411109808,
 237.02563723415315,
 15.395636954480096,
 22.234755318603572,
 0.1450750489773429)

In [125]:
LGBM_training(x_train_norm,y_train)

[LightGBM] [Warning] min_child_samples is set=20, min_child_samples= will be ignored. Current value: min_child_samples=20
[LightGBM] [Warning] Unknown parameter: 40
Duration: 0:01:33.974408



{'mean_fit_time': array([ 4.0807992 ,  4.6836967 ,  5.89211462,  1.17457352,  2.03735785,
       29.44201651,  9.51334758, 31.66745152,  5.36753953,  3.77409418]), 'std_fit_time': array([0.20374208, 0.31695893, 0.35452679, 0.10362906, 0.09451746,
       1.1169872 , 0.74496609, 0.78233619, 0.23506324, 0.53224393]), 'mean_score_time': array([0.0520057 , 0.0342546 , 0.05271006, 0.02205362, 0.03142285,
       0.07189567, 0.08135543, 0.07003102, 0.08625927, 0.05462294]), 'std_score_time': array([0.00669947, 0.00726672, 0.00953732, 0.00697362, 0.0067035 ,
       0.0070412 , 0.00918282, 0.01497402, 0.01257161, 0.025454  ]), 'param_estimator__subsample': masked_array(data=[1.0, 0.8, 0.7000000000000001, 0.4, 0.4, 1.0,
                   0.30000000000000004, 0.8, 1.0, 1.0],
             mask=[False, False,

In [126]:
#def LGBM_model(X,y,num_leaves,min_child_samples,learning_rate,n_estimators,max_depth,subsample,colsample_bytree)
lgbm3 = LGBM_model(x_train_norm,y_train,5,40,0.0125,400,7,1.0,0.30)

Duration: 0:00:02.349426


In [127]:
y_pred= lgbm3.predict(x_test_norm)
eval_metrics(y_test,y_pred)

MAE:  10.6802
MSE:  222.4878
RMSE:  14.916
MAPE:  22.7532
Coefficient of determination (R-squared):  0.1402




(10.680246855330424,
 222.487771735078,
 14.916023992172915,
 22.753237515631238,
 0.14015252933262415)

### Testing on 2018-2019 data

In [128]:
data_1819.head()

,applicable_date,elec_prices,rain_d,temp_d,wdsp_d,wddir_d,rain_c,temp_c,wdsp_c,wddir_c,rain_g,temp_g,wdsp_g,wddir_g,daily_oil_price,daily_natural_gas
0,2018-09-30 22:00:00,71.267,0.0,6.4,5.140,290,0.0,8.2,3.084,340,0.0,5.7,1.028,300,78.89,2.78616
1,2018-09-30 23:00:00,67.212,0.0,5.4,5.140,280,0.0,8.1,3.598,320,0.0,3.9,1.542,290,78.89,2.78616
2,2018-10-01 00:00:00,60.500,0.0,5.2,5.140,280,0.0,7.7,4.112,320,0.0,6.2,1.542,270,84.94,2.79509
3,2018-10-01 01:00:00,63.682,0.0,5.2,5.654,280,0.0,7.4,3.084,320,0.0,7.2,2.056,270,84.94,2.79509
4,2018-10-01 02:00:00,71.617,0.0,4.8,4.626,270,0.0,5.3,1.542,200,0.0,6.9,2.056,280,84.94,2.79509


In [129]:
data_test = data_1819.copy()
data_test = feature_pipeline_1(data_test)
data_test['wdsp_avg']= (data_1819['wdsp_d']+ data_1819['wdsp_c']+ data_1819['wdsp_g'])/3
print(data_test.shape)
data_test.head()

(10514, 7)


,applicable_date,elec_prices,month,dayofyear_num,weekofyear_num,Hour,wdsp_avg
0,2018-09-30 22:00:00,71.267,9,273,39,22,3.084000
1,2018-09-30 23:00:00,67.212,9,273,39,23,3.426667
2,2018-10-01 00:00:00,60.500,10,274,40,0,3.598000
3,2018-10-01 01:00:00,63.682,10,274,40,1,3.598000
4,2018-10-01 02:00:00,71.617,10,274,40,2,2.741333


In [130]:
data_test = multi_step(data_test,'elec_prices')
data_test = calc_24lags(data_test,'elec_prices')
data_test = calc_24lags(data_test,'wdsp_avg')
print(data_test.shape)
print(data_test.columns)
data_test.head()

(10514, 77)
Index(['applicable_date', 'elec_prices', 'month', 'dayofyear_num',
       'weekofyear_num', 'Hour', 'wdsp_avg', 't+1', 't+2', 't+3', 't+4', 't+5',
       't+6', 't+7', 't+8', 't+9', 't+10', 't+11', 't+12', 't+13', 't+14',
       't+15', 't+16', 't+17', 't+18', 't+19', 't+20', 't+21', 't+22', 't+23',
       'lag_1_elec_prices', 'lag_2_elec_prices', 'lag_3_elec_prices',
       'lag_4_elec_prices', 'lag_5_elec_prices', 'lag_6_elec_prices',
       'lag_7_elec_prices', 'lag_8_elec_prices', 'lag_9_elec_prices',
       'lag_10_elec_prices', 'lag_11_elec_prices', 'lag_12_elec_prices',
       'lag_13_elec_prices', 'lag_14_elec_prices', 'lag_15_elec_prices',
       'lag_16_elec_prices', 'lag_17_elec_prices', 'lag_18_elec_prices',
       'lag_19_elec_prices', 'lag_20_elec_prices', 'lag_21_elec_prices',
       'lag_22_elec_prices', 'lag_23_elec_prices', 'lag_24_elec_prices',
       'lag_1_wdsp_avg', 'lag_2_wdsp_avg', 'lag_3_wdsp_avg', 'lag_4_wdsp_avg',
       'lag_5_wdsp_avg', 'lag_6_w

,applicable_date,elec_prices,month,dayofyear_num,weekofyear_num,Hour,wdsp_avg,t+1,t+2,t+3,...,lag_14_wdsp_avg,lag_15_wdsp_avg,lag_16_wdsp_avg,lag_17_wdsp_avg,lag_18_wdsp_avg,lag_19_wdsp_avg,lag_20_wdsp_avg,lag_21_wdsp_avg,lag_22_wdsp_avg,lag_23_wdsp_avg
0,2018-09-30 22:00:00,71.267,9,273,39,22,3.084000,67.212,60.500,63.682,...,3.084,3.084,3.084,3.084,3.084,3.084,3.084,3.084,3.084,3.084
1,2018-09-30 23:00:00,67.212,9,273,39,23,3.426667,60.500,63.682,71.617,...,3.084,3.084,3.084,3.084,3.084,3.084,3.084,3.084,3.084,3.084
2,2018-10-01 00:00:00,60.500,10,274,40,0,3.598000,63.682,71.617,72.855,...,3.084,3.084,3.084,3.084,3.084,3.084,3.084,3.084,3.084,3.084
3,2018-10-01 01:00:00,63.682,10,274,40,1,3.598000,71.617,72.855,76.937,...,3.084,3.084,3.084,3.084,3.084,3.084,3.084,3.084,3.084,3.084
4,2018-10-01 02:00:00,71.617,10,274,40,2,2.741333,72.855,76.937,80.480,...,3.084,3.084,3.084,3.084,3.084,3.084,3.084,3.084,3.084,3.084


In [131]:
#Dropping rows with Nan's
data_test = data_test.dropna(axis=0)
print(data_test.shape)
data_test.isnull().values.any()

(10491, 77)


False

In [132]:
data_test = remove_23rows_hour_col(data_test)
data_test.columns

(438, 76)


Index(['applicable_date', 'elec_prices', 'month', 'dayofyear_num',
       'weekofyear_num', 'wdsp_avg', 't+1', 't+2', 't+3', 't+4', 't+5', 't+6',
       't+7', 't+8', 't+9', 't+10', 't+11', 't+12', 't+13', 't+14', 't+15',
       't+16', 't+17', 't+18', 't+19', 't+20', 't+21', 't+22', 't+23',
       'lag_1_elec_prices', 'lag_2_elec_prices', 'lag_3_elec_prices',
       'lag_4_elec_prices', 'lag_5_elec_prices', 'lag_6_elec_prices',
       'lag_7_elec_prices', 'lag_8_elec_prices', 'lag_9_elec_prices',
       'lag_10_elec_prices', 'lag_11_elec_prices', 'lag_12_elec_prices',
       'lag_13_elec_prices', 'lag_14_elec_prices', 'lag_15_elec_prices',
       'lag_16_elec_prices', 'lag_17_elec_prices', 'lag_18_elec_prices',
       'lag_19_elec_prices', 'lag_20_elec_prices', 'lag_21_elec_prices',
       'lag_22_elec_prices', 'lag_23_elec_prices', 'lag_24_elec_prices',
       'lag_1_wdsp_avg', 'lag_2_wdsp_avg', 'lag_3_wdsp_avg', 'lag_4_wdsp_avg',
       'lag_5_wdsp_avg', 'lag_6_wdsp_avg', 'lag_7_wds

In [133]:
#selecting columns for X & y
cols = train.columns.to_list()
x_names = cols[2:6]
lags = cols[-47:]
x_names.extend(lags)
y_names = [cols[1]]
time_steps = cols[6:29]
y_names.extend(time_steps)
print(x_names)
print(y_names)

['month', 'dayofyear_num', 'weekofyear_num', 'wdsp_avg', 'lag_1_elec_prices', 'lag_2_elec_prices', 'lag_3_elec_prices', 'lag_4_elec_prices', 'lag_5_elec_prices', 'lag_6_elec_prices', 'lag_7_elec_prices', 'lag_8_elec_prices', 'lag_9_elec_prices', 'lag_10_elec_prices', 'lag_11_elec_prices', 'lag_12_elec_prices', 'lag_13_elec_prices', 'lag_14_elec_prices', 'lag_15_elec_prices', 'lag_16_elec_prices', 'lag_17_elec_prices', 'lag_18_elec_prices', 'lag_19_elec_prices', 'lag_20_elec_prices', 'lag_21_elec_prices', 'lag_22_elec_prices', 'lag_23_elec_prices', 'lag_24_elec_prices', 'lag_1_wdsp_avg', 'lag_2_wdsp_avg', 'lag_3_wdsp_avg', 'lag_4_wdsp_avg', 'lag_5_wdsp_avg', 'lag_6_wdsp_avg', 'lag_7_wdsp_avg', 'lag_8_wdsp_avg', 'lag_9_wdsp_avg', 'lag_10_wdsp_avg', 'lag_11_wdsp_avg', 'lag_12_wdsp_avg', 'lag_13_wdsp_avg', 'lag_14_wdsp_avg', 'lag_15_wdsp_avg', 'lag_16_wdsp_avg', 'lag_17_wdsp_avg', 'lag_18_wdsp_avg', 'lag_19_wdsp_avg', 'lag_20_wdsp_avg', 'lag_21_wdsp_avg', 'lag_22_wdsp_avg', 'lag_23_wdsp_av

In [134]:
#GBM
mae,mse,rmse,mape,r2 = testing_runs(10,data_test,gbm3)

Round 1: 
(350, 51) (44, 51) (44, 51)
(350, 24) (44, 24) (44, 24)
MAE:  9.9072
MSE:  190.7338
RMSE:  13.8106
MAPE:  19.4108
Coefficient of determination (R-squared):  0.1735


Round 2: 
(350, 51) (44, 51) (44, 51)
(350, 24) (44, 24) (44, 24)
MAE:  12.5015
MSE:  410.0055
RMSE:  20.2486
MAPE:  22.9071
Coefficient of determination (R-squared):  0.2423


Round 3: 
(350, 51) (44, 51) (44, 51)
(350, 24) (44, 24) (44, 24)
MAE:  12.2977
MSE:  323.4867
RMSE:  17.9857
MAPE:  22.3719
Coefficient of determination (R-squared):  0.2153


Round 4: 
(350, 51) (44, 51) (44, 51)
(350, 24) (44, 24) (44, 24)
MAE:  13.8023
MSE:  444.1694
RMSE:  21.0753
MAPE:  23.6298
Coefficient of determination (R-squared):  0.209


Round 5: 
(350, 51) (44, 51) (44, 51)
(350, 24) (44, 24) (44, 24)
MAE:  12.9286
MSE:  452.0032
RMSE:  21.2604
MAPE:  23.4192
Coefficient of determination (R-squared):  0.2373


Round 6: 
(350, 51) (44, 51) (44, 51)
(350, 24) (44, 24) (44, 24)
MAE:  11.8409
MSE:  263.9518
RMSE:  16.2466
MAPE:  

In [135]:
#XGBM
mae,mse,rmse,mape,r2 = testing_runs(10,data_test,xgb3)

Round 1: 
(350, 51) (44, 51) (44, 51)
(350, 24) (44, 24) (44, 24)
MAE:  11.1242
MSE:  343.6641
RMSE:  18.5382
MAPE:  20.8783
Coefficient of determination (R-squared):  0.4026


Round 2: 
(350, 51) (44, 51) (44, 51)
(350, 24) (44, 24) (44, 24)
MAE:  9.5691
MSE:  212.3395
RMSE:  14.5719
MAPE:  17.4164
Coefficient of determination (R-squared):  0.3985


Round 3: 
(350, 51) (44, 51) (44, 51)
(350, 24) (44, 24) (44, 24)
MAE:  10.0443
MSE:  306.5532
RMSE:  17.5087
MAPE:  19.1995
Coefficient of determination (R-squared):  0.404


Round 4: 
(350, 51) (44, 51) (44, 51)
(350, 24) (44, 24) (44, 24)
MAE:  11.2828
MSE:  387.3364
RMSE:  19.6809
MAPE:  20.4796
Coefficient of determination (R-squared):  0.4039


Round 5: 
(350, 51) (44, 51) (44, 51)
(350, 24) (44, 24) (44, 24)
MAE:  10.9728
MSE:  259.0864
RMSE:  16.0962
MAPE:  18.9691
Coefficient of determination (R-squared):  0.381


Round 6: 
(350, 51) (44, 51) (44, 51)
(350, 24) (44, 24) (44, 24)
MAE:  9.8442
MSE:  232.2348
RMSE:  15.2393
MAPE:  18

In [136]:
#light GBM
mae,mse,rmse,mape,r2 = testing_runs(10,data_test,lgbm3)

Round 1: 
(350, 51) (44, 51) (44, 51)
(350, 24) (44, 24) (44, 24)
MAE:  10.0683
MSE:  230.1535
RMSE:  15.1708
MAPE:  20.1247
Coefficient of determination (R-squared):  0.4387


Round 2: 
(350, 51) (44, 51) (44, 51)
(350, 24) (44, 24) (44, 24)
MAE:  11.0185
MSE:  256.2825
RMSE:  16.0088
MAPE:  18.9541
Coefficient of determination (R-squared):  0.5132


Round 3: 
(350, 51) (44, 51) (44, 51)
(350, 24) (44, 24) (44, 24)
MAE:  10.5656
MSE:  260.0831
RMSE:  16.1271
MAPE:  18.9046
Coefficient of determination (R-squared):  0.4948


Round 4: 
(350, 51) (44, 51) (44, 51)
(350, 24) (44, 24) (44, 24)
MAE:  9.2213
MSE:  170.0463
RMSE:  13.0402
MAPE:  17.4316
Coefficient of determination (R-squared):  0.5337


Round 5: 
(350, 51) (44, 51) (44, 51)
(350, 24) (44, 24) (44, 24)
MAE:  10.7819
MSE:  234.6698
RMSE:  15.3189
MAPE:  19.5942
Coefficient of determination (R-squared):  0.4924


Round 6: 
(350, 51) (44, 51) (44, 51)
(350, 24) (44, 24) (44, 24)
MAE:  9.9452
MSE:  217.2562
RMSE:  14.7396
MAPE:  

In [137]:
import pickle
# save the model to disk
pickle.dump(gbm1, open('GBM_basic_model.sav', 'wb'))
pickle.dump(gbm2, open('GBM_basic_expanding_window_price_model.sav', 'wb'))
pickle.dump(gbm3, open('GBM_basic_wdsp_sum_model.sav', 'wb'))

pickle.dump(xgb1, open('XGBM_basic_model.sav', 'wb'))
pickle.dump(xgb2, open('XGBM_basic_expanding_window_price_model.sav', 'wb'))
pickle.dump(xgb3, open('XGBM_basic_wdsp_sum_model.sav', 'wb'))

pickle.dump(lgbm1, open('LightGBM_basic_model.sav', 'wb'))
pickle.dump(lgbm2, open('LightGBM_basic_expanding_window_price_model.sav', 'wb'))
pickle.dump(lgbm3, open('LightGBM_basic_wdsp_sum_model.sav', 'wb'))
#gbm = pickle.load(open('GBMtimefeatures1_model.sav', 'rb'))

In [242]:
lgbm = pickle.load(open('LightGBM_basic_wdsp_sum_model.sav', 'rb'))

<h1 style="text-align:center;font-size:40px;" > Basic time based features + wdsp_g (windspeeds in Galway county) </h1>

In [23]:
data.head()

,applicable_date,elec_prices,rain_d,temp_d,wdsp_d,wddir_d,rain_c,temp_c,wdsp_c,wddir_c,rain_g,temp_g,wdsp_g,wddir_g,daily_oil_price,daily_natural_gas
0,2019-01-01 00:00:00,70.28,0.0,8.8,4.626,250,0.0,8.5,2.056,240,0.0,9.9,2.056,260,59.41,2.61649
1,2019-01-01 01:00:00,65.18,0.0,8.3,4.626,250,0.0,8.5,1.542,270,0.0,9.3,2.056,240,59.41,2.61649
2,2019-01-01 02:00:00,60.91,0.0,8.4,4.112,240,0.0,8.4,1.542,280,0.0,9.2,2.570,240,59.41,2.61649
3,2019-01-01 03:00:00,65.16,0.0,8.6,4.112,250,0.0,8.5,2.570,260,0.0,9.1,2.570,250,59.41,2.61649
4,2019-01-01 04:00:00,65.16,0.0,8.4,5.140,250,0.0,8.5,2.570,270,0.0,8.8,2.056,250,59.41,2.61649


In [24]:
data_1= data.copy()
data_1 = feature_pipeline_1(data_1)
data_1['wdsp_g'] = data['wdsp_g']
data_1.head()

,applicable_date,elec_prices,month,dayofyear_num,weekofyear_num,Hour,wdsp_g
0,2019-01-01 00:00:00,70.28,1,1,1,0,2.056
1,2019-01-01 01:00:00,65.18,1,1,1,1,2.056
2,2019-01-01 02:00:00,60.91,1,1,1,2,2.570
3,2019-01-01 03:00:00,65.16,1,1,1,3,2.570
4,2019-01-01 04:00:00,65.16,1,1,1,4,2.056


In [25]:
data_1 = multi_step(data_1,'elec_prices')
data_1 = calc_24lags(data_1,'elec_prices')
data_1 = calc_24lags(data_1,'wdsp_g')
print(data_1.shape)
print(data_1.columns)
data_1.head()

(8304, 77)
Index(['applicable_date', 'elec_prices', 'month', 'dayofyear_num',
       'weekofyear_num', 'Hour', 'wdsp_g', 't+1', 't+2', 't+3', 't+4', 't+5',
       't+6', 't+7', 't+8', 't+9', 't+10', 't+11', 't+12', 't+13', 't+14',
       't+15', 't+16', 't+17', 't+18', 't+19', 't+20', 't+21', 't+22', 't+23',
       'lag_1_elec_prices', 'lag_2_elec_prices', 'lag_3_elec_prices',
       'lag_4_elec_prices', 'lag_5_elec_prices', 'lag_6_elec_prices',
       'lag_7_elec_prices', 'lag_8_elec_prices', 'lag_9_elec_prices',
       'lag_10_elec_prices', 'lag_11_elec_prices', 'lag_12_elec_prices',
       'lag_13_elec_prices', 'lag_14_elec_prices', 'lag_15_elec_prices',
       'lag_16_elec_prices', 'lag_17_elec_prices', 'lag_18_elec_prices',
       'lag_19_elec_prices', 'lag_20_elec_prices', 'lag_21_elec_prices',
       'lag_22_elec_prices', 'lag_23_elec_prices', 'lag_24_elec_prices',
       'lag_1_wdsp_g', 'lag_2_wdsp_g', 'lag_3_wdsp_g', 'lag_4_wdsp_g',
       'lag_5_wdsp_g', 'lag_6_wdsp_g', 'lag_

,applicable_date,elec_prices,month,dayofyear_num,weekofyear_num,Hour,wdsp_g,t+1,t+2,t+3,...,lag_14_wdsp_g,lag_15_wdsp_g,lag_16_wdsp_g,lag_17_wdsp_g,lag_18_wdsp_g,lag_19_wdsp_g,lag_20_wdsp_g,lag_21_wdsp_g,lag_22_wdsp_g,lag_23_wdsp_g
0,2019-01-01 00:00:00,70.28,1,1,1,0,2.056,65.18,60.91,65.160,...,2.056,2.056,2.056,2.056,2.056,2.056,2.056,2.056,2.056,2.056
1,2019-01-01 01:00:00,65.18,1,1,1,1,2.056,60.91,65.16,65.160,...,2.056,2.056,2.056,2.056,2.056,2.056,2.056,2.056,2.056,2.056
2,2019-01-01 02:00:00,60.91,1,1,1,2,2.570,65.16,65.16,68.290,...,2.056,2.056,2.056,2.056,2.056,2.056,2.056,2.056,2.056,2.056
3,2019-01-01 03:00:00,65.16,1,1,1,3,2.570,65.16,68.29,63.030,...,2.056,2.056,2.056,2.056,2.056,2.056,2.056,2.056,2.056,2.056
4,2019-01-01 04:00:00,65.16,1,1,1,4,2.056,68.29,63.03,51.621,...,2.056,2.056,2.056,2.056,2.056,2.056,2.056,2.056,2.056,2.056


In [26]:
data_1.isnull().values.any()

True

In [27]:
#Dropping rows with Nan's
data_1 = data_1.dropna(axis=0)
print(data_1.shape)
data_1.isnull().values.any()

(8281, 77)


False

In [28]:
data_1 = remove_23rows_hour_col(data_1)
train,test = train_test_split_tbs(data_1)
train.columns

(345, 76)
(276, 76)
(69, 76)


Index(['applicable_date', 'elec_prices', 'month', 'dayofyear_num',
       'weekofyear_num', 'wdsp_g', 't+1', 't+2', 't+3', 't+4', 't+5', 't+6',
       't+7', 't+8', 't+9', 't+10', 't+11', 't+12', 't+13', 't+14', 't+15',
       't+16', 't+17', 't+18', 't+19', 't+20', 't+21', 't+22', 't+23',
       'lag_1_elec_prices', 'lag_2_elec_prices', 'lag_3_elec_prices',
       'lag_4_elec_prices', 'lag_5_elec_prices', 'lag_6_elec_prices',
       'lag_7_elec_prices', 'lag_8_elec_prices', 'lag_9_elec_prices',
       'lag_10_elec_prices', 'lag_11_elec_prices', 'lag_12_elec_prices',
       'lag_13_elec_prices', 'lag_14_elec_prices', 'lag_15_elec_prices',
       'lag_16_elec_prices', 'lag_17_elec_prices', 'lag_18_elec_prices',
       'lag_19_elec_prices', 'lag_20_elec_prices', 'lag_21_elec_prices',
       'lag_22_elec_prices', 'lag_23_elec_prices', 'lag_24_elec_prices',
       'lag_1_wdsp_g', 'lag_2_wdsp_g', 'lag_3_wdsp_g', 'lag_4_wdsp_g',
       'lag_5_wdsp_g', 'lag_6_wdsp_g', 'lag_7_wdsp_g', 'lag_8_w

In [29]:
#selecting columns for X & y
cols = train.columns.to_list()
x_names = cols[2:6]
lags = cols[-47:]
x_names.extend(lags)
y_names = [cols[1]]
time_steps = cols[6:29]
y_names.extend(time_steps)
print(x_names)
print(y_names)

['month', 'dayofyear_num', 'weekofyear_num', 'wdsp_g', 'lag_1_elec_prices', 'lag_2_elec_prices', 'lag_3_elec_prices', 'lag_4_elec_prices', 'lag_5_elec_prices', 'lag_6_elec_prices', 'lag_7_elec_prices', 'lag_8_elec_prices', 'lag_9_elec_prices', 'lag_10_elec_prices', 'lag_11_elec_prices', 'lag_12_elec_prices', 'lag_13_elec_prices', 'lag_14_elec_prices', 'lag_15_elec_prices', 'lag_16_elec_prices', 'lag_17_elec_prices', 'lag_18_elec_prices', 'lag_19_elec_prices', 'lag_20_elec_prices', 'lag_21_elec_prices', 'lag_22_elec_prices', 'lag_23_elec_prices', 'lag_24_elec_prices', 'lag_1_wdsp_g', 'lag_2_wdsp_g', 'lag_3_wdsp_g', 'lag_4_wdsp_g', 'lag_5_wdsp_g', 'lag_6_wdsp_g', 'lag_7_wdsp_g', 'lag_8_wdsp_g', 'lag_9_wdsp_g', 'lag_10_wdsp_g', 'lag_11_wdsp_g', 'lag_12_wdsp_g', 'lag_13_wdsp_g', 'lag_14_wdsp_g', 'lag_15_wdsp_g', 'lag_16_wdsp_g', 'lag_17_wdsp_g', 'lag_18_wdsp_g', 'lag_19_wdsp_g', 'lag_20_wdsp_g', 'lag_21_wdsp_g', 'lag_22_wdsp_g', 'lag_23_wdsp_g']
['elec_prices', 't+1', 't+2', 't+3', 't+4', 

In [30]:
x_train, y_train = train[x_names], train[y_names]
x_test, y_test = test[x_names], test[y_names]
print(x_train.shape,y_train.shape)
print(x_test.shape,y_test.shape)

(276, 51) (276, 24)
(69, 51) (69, 24)


In [31]:
x_train_norm,x_test_norm = normalize(x_train,x_test)

In [32]:
GBM_training(x_train_norm,y_train)

Duration: 0:00:14.148737



{'mean_fit_time': array([1.3084975 , 1.15298576, 1.16919658, 0.38524511, 1.15833576,
       1.02391505, 0.47295117, 1.02669818, 1.04195931, 1.09254212]), 'std_fit_time': array([0.01743614, 0.01095284, 0.01665106, 0.00701373, 0.01101761,
       0.02231743, 0.01254667, 0.01531264, 0.01532378, 0.19127414]), 'mean_score_time': array([0.02396662, 0.01718867, 0.01718829, 0.01875241, 0.01718872,
       0.01875281, 0.01562643, 0.0187521 , 0.02031415, 0.00824149]), 'std_score_time': array([7.43621615e-03, 4.68726978e-03, 4.68803257e-03, 6.25302865e-03,
       4.68709472e-03, 6.24990609e-03, 1.79369382e-06, 6.24978710e-03,
       7.16075841e-03, 8.27477124e-03]), 'param_estimator__subsample': masked_array(data=[0.7, 0.9, 0.85, 0.8, 0.9, 0.9, 0.7, 0.9, 0.75, 0.7],
             mask=[False, False, False, False, False, False, False, False,
                   False, False],
       fill_value='?',
            dtype=object), 'param_estimator__n_estimators': masked_array(dat

In [33]:
#def GBM_model(X,y,learning_rate,n_estimators,max_depth,min_samples_split,max_features,min_samples_leaf,subsample):
gbm4 = GBM_model(x_train_norm,y_train,0.05,60,11,100,4,30,0.7)

Duration: 0:00:01.369721


In [34]:
y_pred= gbm4.predict(x_test_norm)
eval_metrics(y_test,y_pred)

MAE:  10.6368
MSE:  225.1096
RMSE:  15.0037
MAPE:  22.6607
Coefficient of determination (R-squared):  0.1239




(10.636810229496865,
 225.10955471909043,
 15.003651379550593,
 22.66069996684084,
 0.12386368385382183)

In [35]:
XGBM_training(x_train_norm,y_train)

Duration: 0:01:50.537449



{'mean_fit_time': array([12.39821999,  6.32969604, 10.15205202, 16.10051751, 11.35452895,
       15.59014776,  7.07336607, 21.47354755, 10.74738541,  6.89431748]), 'std_fit_time': array([0.70357408, 2.15707834, 0.51139416, 0.76923499, 0.72989497,
       0.47671236, 0.11898248, 0.63771429, 0.40625581, 1.15017046]), 'mean_score_time': array([0.05469174, 0.04675493, 0.0620537 , 0.06434014, 0.06113451,
       0.05030043, 0.04710364, 0.06283238, 0.04271157, 0.04807134]), 'std_score_time': array([0.0078126 , 0.00425999, 0.00962855, 0.01189812, 0.01095392,
       0.00647366, 0.00045091, 0.01186294, 0.01291287, 0.01153587]), 'param_estimator__subsample': masked_array(data=[0.1, 1.0, 0.30000000000000004, 0.8, 0.8,
                   0.7000000000000001, 0.2, 0.6, 1.0, 0.4],
             mask=[False, False, False, False, False, False, False, False,
                   False, False],
       fill_value='?',
            dtype=object), 'param_estimator__n_estimators': maske

In [36]:
# def XGBM_model(X,y,learning_rate,n_estimators,max_depth,subsample,colsample_bytree):
xgb4 = XGBM_model(x_train_norm,y_train,0.0125,300,5,0.8,0.4)

Duration: 0:00:03.465679


In [37]:
y_pred= xgb4.predict(x_test_norm)
eval_metrics(y_test,y_pred)

MAE:  10.4933
MSE:  240.407
RMSE:  15.5051
MAPE:  22.355
Coefficient of determination (R-squared):  0.1294




(10.493326811401166,
 240.40704132499187,
 15.505065021630573,
 22.355022360722902,
 0.12935215281028553)

In [38]:
LGBM_training(x_train_norm,y_train)

[LightGBM] [Warning] min_child_samples is set=20, min_child_samples= will be ignored. Current value: min_child_samples=20
[LightGBM] [Warning] Unknown parameter: 10
Duration: 0:00:55.427519



{'mean_fit_time': array([ 4.4549386 ,  4.38233917,  3.04551175,  4.74508829,  7.43108628,
        9.42428732,  2.70066876,  1.09706779, 16.62820189,  5.27606895]), 'std_fit_time': array([0.44017476, 0.27222622, 0.1312005 , 0.30873213, 0.21849186,
       0.3262926 , 0.11471754, 0.06089217, 0.5636754 , 1.48666583]), 'mean_score_time': array([0.05629692, 0.05931966, 0.03275704, 0.04998229, 0.07312951,
       0.08327792, 0.04799821, 0.03585379, 0.05440478, 0.0400054 ]), 'std_score_time': array([0.00536749, 0.01794304, 0.00646869, 0.01490914, 0.01831923,
       0.02608562, 0.03767316, 0.01904067, 0.0296793 , 0.01715756]), 'param_estimator__subsample': masked_array(data=[0.5, 0.5, 0.5, 0.7000000000000001, 0.5, 0.1,
                   0.30000000000000004, 1.0, 0.2, 0.7000000000000001],
             mask

In [39]:
#def LGBM_model(X,y,num_leaves,min_child_samples,learning_rate,n_estimators,max_depth,subsample,colsample_bytree)
lgbm4 = LGBM_model(x_train_norm,y_train,1229,10,0.025,200,3,0.5,1.0)

Duration: 0:00:01.807942


In [40]:
y_pred= lgbm4.predict(x_test_norm)
eval_metrics(y_test,y_pred)

MAE:  10.7501
MSE:  241.2853
RMSE:  15.5334
MAPE:  22.9021
Coefficient of determination (R-squared):  0.0946




(10.750101474575862,
 241.28530462678637,
 15.533361021581465,
 22.902056055575574,
 0.09461462030423962)

### Testing on 2018-2019 data

In [41]:
data_test = data_1819.copy()
data_test = feature_pipeline_1(data_test)
data_test['wdsp_g']= data_1819['wdsp_g']
data_test = multi_step(data_test,'elec_prices')
data_test = calc_24lags(data_test,'elec_prices')
data_test = calc_24lags(data_test,'wdsp_g')
print(data_test.shape)
print(data_test.columns)
data_test.head()

(10514, 77)
Index(['applicable_date', 'elec_prices', 'month', 'dayofyear_num',
       'weekofyear_num', 'Hour', 'wdsp_g', 't+1', 't+2', 't+3', 't+4', 't+5',
       't+6', 't+7', 't+8', 't+9', 't+10', 't+11', 't+12', 't+13', 't+14',
       't+15', 't+16', 't+17', 't+18', 't+19', 't+20', 't+21', 't+22', 't+23',
       'lag_1_elec_prices', 'lag_2_elec_prices', 'lag_3_elec_prices',
       'lag_4_elec_prices', 'lag_5_elec_prices', 'lag_6_elec_prices',
       'lag_7_elec_prices', 'lag_8_elec_prices', 'lag_9_elec_prices',
       'lag_10_elec_prices', 'lag_11_elec_prices', 'lag_12_elec_prices',
       'lag_13_elec_prices', 'lag_14_elec_prices', 'lag_15_elec_prices',
       'lag_16_elec_prices', 'lag_17_elec_prices', 'lag_18_elec_prices',
       'lag_19_elec_prices', 'lag_20_elec_prices', 'lag_21_elec_prices',
       'lag_22_elec_prices', 'lag_23_elec_prices', 'lag_24_elec_prices',
       'lag_1_wdsp_g', 'lag_2_wdsp_g', 'lag_3_wdsp_g', 'lag_4_wdsp_g',
       'lag_5_wdsp_g', 'lag_6_wdsp_g', 'lag

,applicable_date,elec_prices,month,dayofyear_num,weekofyear_num,Hour,wdsp_g,t+1,t+2,t+3,...,lag_14_wdsp_g,lag_15_wdsp_g,lag_16_wdsp_g,lag_17_wdsp_g,lag_18_wdsp_g,lag_19_wdsp_g,lag_20_wdsp_g,lag_21_wdsp_g,lag_22_wdsp_g,lag_23_wdsp_g
0,2018-09-30 22:00:00,71.267,9,273,39,22,1.028,67.212,60.500,63.682,...,1.028,1.028,1.028,1.028,1.028,1.028,1.028,1.028,1.028,1.028
1,2018-09-30 23:00:00,67.212,9,273,39,23,1.542,60.500,63.682,71.617,...,1.028,1.028,1.028,1.028,1.028,1.028,1.028,1.028,1.028,1.028
2,2018-10-01 00:00:00,60.500,10,274,40,0,1.542,63.682,71.617,72.855,...,1.028,1.028,1.028,1.028,1.028,1.028,1.028,1.028,1.028,1.028
3,2018-10-01 01:00:00,63.682,10,274,40,1,2.056,71.617,72.855,76.937,...,1.028,1.028,1.028,1.028,1.028,1.028,1.028,1.028,1.028,1.028
4,2018-10-01 02:00:00,71.617,10,274,40,2,2.056,72.855,76.937,80.480,...,1.028,1.028,1.028,1.028,1.028,1.028,1.028,1.028,1.028,1.028


In [42]:
#Dropping rows with Nan's
data_test = data_test.dropna(axis=0)
print(data_test.shape)
data_test.isnull().values.any()

(10491, 77)


False

In [43]:
data_test = remove_23rows_hour_col(data_test)
data_test.columns

(438, 76)


Index(['applicable_date', 'elec_prices', 'month', 'dayofyear_num',
       'weekofyear_num', 'wdsp_g', 't+1', 't+2', 't+3', 't+4', 't+5', 't+6',
       't+7', 't+8', 't+9', 't+10', 't+11', 't+12', 't+13', 't+14', 't+15',
       't+16', 't+17', 't+18', 't+19', 't+20', 't+21', 't+22', 't+23',
       'lag_1_elec_prices', 'lag_2_elec_prices', 'lag_3_elec_prices',
       'lag_4_elec_prices', 'lag_5_elec_prices', 'lag_6_elec_prices',
       'lag_7_elec_prices', 'lag_8_elec_prices', 'lag_9_elec_prices',
       'lag_10_elec_prices', 'lag_11_elec_prices', 'lag_12_elec_prices',
       'lag_13_elec_prices', 'lag_14_elec_prices', 'lag_15_elec_prices',
       'lag_16_elec_prices', 'lag_17_elec_prices', 'lag_18_elec_prices',
       'lag_19_elec_prices', 'lag_20_elec_prices', 'lag_21_elec_prices',
       'lag_22_elec_prices', 'lag_23_elec_prices', 'lag_24_elec_prices',
       'lag_1_wdsp_g', 'lag_2_wdsp_g', 'lag_3_wdsp_g', 'lag_4_wdsp_g',
       'lag_5_wdsp_g', 'lag_6_wdsp_g', 'lag_7_wdsp_g', 'lag_8_w

In [44]:
#selecting columns for X & y
cols = train.columns.to_list()
x_names = cols[2:6]
lags = cols[-47:]
x_names.extend(lags)
y_names = [cols[1]]
time_steps = cols[6:29]
y_names.extend(time_steps)
print(x_names)
print(y_names)

['month', 'dayofyear_num', 'weekofyear_num', 'wdsp_g', 'lag_1_elec_prices', 'lag_2_elec_prices', 'lag_3_elec_prices', 'lag_4_elec_prices', 'lag_5_elec_prices', 'lag_6_elec_prices', 'lag_7_elec_prices', 'lag_8_elec_prices', 'lag_9_elec_prices', 'lag_10_elec_prices', 'lag_11_elec_prices', 'lag_12_elec_prices', 'lag_13_elec_prices', 'lag_14_elec_prices', 'lag_15_elec_prices', 'lag_16_elec_prices', 'lag_17_elec_prices', 'lag_18_elec_prices', 'lag_19_elec_prices', 'lag_20_elec_prices', 'lag_21_elec_prices', 'lag_22_elec_prices', 'lag_23_elec_prices', 'lag_24_elec_prices', 'lag_1_wdsp_g', 'lag_2_wdsp_g', 'lag_3_wdsp_g', 'lag_4_wdsp_g', 'lag_5_wdsp_g', 'lag_6_wdsp_g', 'lag_7_wdsp_g', 'lag_8_wdsp_g', 'lag_9_wdsp_g', 'lag_10_wdsp_g', 'lag_11_wdsp_g', 'lag_12_wdsp_g', 'lag_13_wdsp_g', 'lag_14_wdsp_g', 'lag_15_wdsp_g', 'lag_16_wdsp_g', 'lag_17_wdsp_g', 'lag_18_wdsp_g', 'lag_19_wdsp_g', 'lag_20_wdsp_g', 'lag_21_wdsp_g', 'lag_22_wdsp_g', 'lag_23_wdsp_g']
['elec_prices', 't+1', 't+2', 't+3', 't+4', 

In [45]:
#GBM
mae,mse,rmse,mape,r2 = testing_runs(10,data_test,gbm4)

Round 1: 
(350, 51) (44, 51) (44, 51)
(350, 24) (44, 24) (44, 24)
MAE:  10.661
MSE:  319.6512
RMSE:  17.8788
MAPE:  19.201
Coefficient of determination (R-squared):  0.4435


Round 2: 
(350, 51) (44, 51) (44, 51)
(350, 24) (44, 24) (44, 24)
MAE:  11.7433
MSE:  345.9073
RMSE:  18.5986
MAPE:  20.9532
Coefficient of determination (R-squared):  0.4191


Round 3: 
(350, 51) (44, 51) (44, 51)
(350, 24) (44, 24) (44, 24)
MAE:  10.8078
MSE:  309.1581
RMSE:  17.5829
MAPE:  18.8359
Coefficient of determination (R-squared):  0.3828


Round 4: 
(350, 51) (44, 51) (44, 51)
(350, 24) (44, 24) (44, 24)
MAE:  10.1492
MSE:  242.0146
RMSE:  15.5568
MAPE:  17.8545
Coefficient of determination (R-squared):  0.4885


Round 5: 
(350, 51) (44, 51) (44, 51)
(350, 24) (44, 24) (44, 24)
MAE:  12.1508
MSE:  411.5447
RMSE:  20.2866
MAPE:  21.8778
Coefficient of determination (R-squared):  0.466


Round 6: 
(350, 51) (44, 51) (44, 51)
(350, 24) (44, 24) (44, 24)
MAE:  10.8876
MSE:  283.6984
RMSE:  16.8433
MAPE:  1

In [46]:
#XGBM
mae,mse,rmse,mape,r2 = testing_runs(10,data_test,xgb4)

Round 1: 
(350, 51) (44, 51) (44, 51)
(350, 24) (44, 24) (44, 24)
MAE:  9.6061
MSE:  197.6972
RMSE:  14.0605
MAPE:  18.2537
Coefficient of determination (R-squared):  0.4647


Round 2: 
(350, 51) (44, 51) (44, 51)
(350, 24) (44, 24) (44, 24)
MAE:  10.709
MSE:  358.4323
RMSE:  18.9323
MAPE:  18.3791
Coefficient of determination (R-squared):  0.4804


Round 3: 
(350, 51) (44, 51) (44, 51)
(350, 24) (44, 24) (44, 24)
MAE:  9.0868
MSE:  246.6901
RMSE:  15.7064
MAPE:  16.3424
Coefficient of determination (R-squared):  0.5093


Round 4: 
(350, 51) (44, 51) (44, 51)
(350, 24) (44, 24) (44, 24)
MAE:  10.3298
MSE:  256.8193
RMSE:  16.0256
MAPE:  18.5249
Coefficient of determination (R-squared):  0.411


Round 5: 
(350, 51) (44, 51) (44, 51)
(350, 24) (44, 24) (44, 24)
MAE:  10.8964
MSE:  401.9626
RMSE:  20.049
MAPE:  19.4328
Coefficient of determination (R-squared):  0.4211


Round 6: 
(350, 51) (44, 51) (44, 51)
(350, 24) (44, 24) (44, 24)
MAE:  8.4705
MSE:  189.1514
RMSE:  13.7532
MAPE:  15.8

In [47]:
#light GBM
mae,mse,rmse,mape,r2 = testing_runs(10,data_test,lgbm4)

Round 1: 
(350, 51) (44, 51) (44, 51)
(350, 24) (44, 24) (44, 24)
MAE:  9.9649
MSE:  223.1123
RMSE:  14.9369
MAPE:  18.6834
Coefficient of determination (R-squared):  0.4503


Round 2: 
(350, 51) (44, 51) (44, 51)
(350, 24) (44, 24) (44, 24)
MAE:  9.2335
MSE:  191.3308
RMSE:  13.8322
MAPE:  17.4398
Coefficient of determination (R-squared):  0.558


Round 3: 
(350, 51) (44, 51) (44, 51)
(350, 24) (44, 24) (44, 24)
MAE:  11.0055
MSE:  353.2451
RMSE:  18.7948
MAPE:  18.6901
Coefficient of determination (R-squared):  0.541


Round 4: 
(350, 51) (44, 51) (44, 51)
(350, 24) (44, 24) (44, 24)
MAE:  9.8452
MSE:  245.4801
RMSE:  15.6678
MAPE:  17.9711
Coefficient of determination (R-squared):  0.547


Round 5: 
(350, 51) (44, 51) (44, 51)
(350, 24) (44, 24) (44, 24)
MAE:  10.1483
MSE:  253.6209
RMSE:  15.9255
MAPE:  18.4514
Coefficient of determination (R-squared):  0.5129


Round 6: 
(350, 51) (44, 51) (44, 51)
(350, 24) (44, 24) (44, 24)
MAE:  9.491
MSE:  260.7123
RMSE:  16.1466
MAPE:  17.433

<h1 style="text-align:center;font-size:40px;" > Basic time based features + wdsp_sqr (square of avg of windspeeds in all counties) </h1>

In [73]:
data_1= data.copy()
data_1 = feature_pipeline_1(data_1)
wdsp_sum= data['wdsp_d']+data['wdsp_c']+data['wdsp_g']
wdsp_avg = wdsp_sum/3
wdsp_avg_sq = wdsp_avg*wdsp_avg
data_1['wdsp_avg_sq'] = round(wdsp_avg_sq,4)
data_1.head()

,applicable_date,elec_prices,month,dayofyear_num,weekofyear_num,Hour,wdsp_avg_sq
0,2019-01-01 00:00:00,70.28,1,1,1,0,8.4836
1,2019-01-01 01:00:00,65.18,1,1,1,1,7.5149
2,2019-01-01 02:00:00,60.91,1,1,1,2,7.5149
3,2019-01-01 03:00:00,65.16,1,1,1,3,9.5111
4,2019-01-01 04:00:00,65.16,1,1,1,4,10.5972


In [74]:
data_1 = multi_step(data_1,'elec_prices')
data_1 = calc_24lags(data_1,'elec_prices')
data_1 = calc_24lags(data_1,'wdsp_avg_sq')
print(data_1.shape)
print(data_1.columns)
data_1.head()

(8304, 77)
Index(['applicable_date', 'elec_prices', 'month', 'dayofyear_num',
       'weekofyear_num', 'Hour', 'wdsp_avg_sq', 't+1', 't+2', 't+3', 't+4',
       't+5', 't+6', 't+7', 't+8', 't+9', 't+10', 't+11', 't+12', 't+13',
       't+14', 't+15', 't+16', 't+17', 't+18', 't+19', 't+20', 't+21', 't+22',
       't+23', 'lag_1_elec_prices', 'lag_2_elec_prices', 'lag_3_elec_prices',
       'lag_4_elec_prices', 'lag_5_elec_prices', 'lag_6_elec_prices',
       'lag_7_elec_prices', 'lag_8_elec_prices', 'lag_9_elec_prices',
       'lag_10_elec_prices', 'lag_11_elec_prices', 'lag_12_elec_prices',
       'lag_13_elec_prices', 'lag_14_elec_prices', 'lag_15_elec_prices',
       'lag_16_elec_prices', 'lag_17_elec_prices', 'lag_18_elec_prices',
       'lag_19_elec_prices', 'lag_20_elec_prices', 'lag_21_elec_prices',
       'lag_22_elec_prices', 'lag_23_elec_prices', 'lag_24_elec_prices',
       'lag_1_wdsp_avg_sq', 'lag_2_wdsp_avg_sq', 'lag_3_wdsp_avg_sq',
       'lag_4_wdsp_avg_sq', 'lag_5_wdsp_

,applicable_date,elec_prices,month,dayofyear_num,weekofyear_num,Hour,wdsp_avg_sq,t+1,t+2,t+3,...,lag_14_wdsp_avg_sq,lag_15_wdsp_avg_sq,lag_16_wdsp_avg_sq,lag_17_wdsp_avg_sq,lag_18_wdsp_avg_sq,lag_19_wdsp_avg_sq,lag_20_wdsp_avg_sq,lag_21_wdsp_avg_sq,lag_22_wdsp_avg_sq,lag_23_wdsp_avg_sq
0,2019-01-01 00:00:00,70.28,1,1,1,0,8.4836,65.18,60.91,65.160,...,8.4836,8.4836,8.4836,8.4836,8.4836,8.4836,8.4836,8.4836,8.4836,8.4836
1,2019-01-01 01:00:00,65.18,1,1,1,1,7.5149,60.91,65.16,65.160,...,8.4836,8.4836,8.4836,8.4836,8.4836,8.4836,8.4836,8.4836,8.4836,8.4836
2,2019-01-01 02:00:00,60.91,1,1,1,2,7.5149,65.16,65.16,68.290,...,8.4836,8.4836,8.4836,8.4836,8.4836,8.4836,8.4836,8.4836,8.4836,8.4836
3,2019-01-01 03:00:00,65.16,1,1,1,3,9.5111,65.16,68.29,63.030,...,8.4836,8.4836,8.4836,8.4836,8.4836,8.4836,8.4836,8.4836,8.4836,8.4836
4,2019-01-01 04:00:00,65.16,1,1,1,4,10.5972,68.29,63.03,51.621,...,8.4836,8.4836,8.4836,8.4836,8.4836,8.4836,8.4836,8.4836,8.4836,8.4836


In [75]:
data_1.isnull().values.any()

True

In [76]:
#Dropping rows with Nan's
data_1 = data_1.dropna(axis=0)
print(data_1.shape)
data_1.isnull().values.any()

(8281, 77)


False

In [77]:
data_1 = remove_23rows_hour_col(data_1)
train,test = train_test_split_tbs(data_1)
train.columns

(345, 76)
(276, 76)
(69, 76)


Index(['applicable_date', 'elec_prices', 'month', 'dayofyear_num',
       'weekofyear_num', 'wdsp_avg_sq', 't+1', 't+2', 't+3', 't+4', 't+5',
       't+6', 't+7', 't+8', 't+9', 't+10', 't+11', 't+12', 't+13', 't+14',
       't+15', 't+16', 't+17', 't+18', 't+19', 't+20', 't+21', 't+22', 't+23',
       'lag_1_elec_prices', 'lag_2_elec_prices', 'lag_3_elec_prices',
       'lag_4_elec_prices', 'lag_5_elec_prices', 'lag_6_elec_prices',
       'lag_7_elec_prices', 'lag_8_elec_prices', 'lag_9_elec_prices',
       'lag_10_elec_prices', 'lag_11_elec_prices', 'lag_12_elec_prices',
       'lag_13_elec_prices', 'lag_14_elec_prices', 'lag_15_elec_prices',
       'lag_16_elec_prices', 'lag_17_elec_prices', 'lag_18_elec_prices',
       'lag_19_elec_prices', 'lag_20_elec_prices', 'lag_21_elec_prices',
       'lag_22_elec_prices', 'lag_23_elec_prices', 'lag_24_elec_prices',
       'lag_1_wdsp_avg_sq', 'lag_2_wdsp_avg_sq', 'lag_3_wdsp_avg_sq',
       'lag_4_wdsp_avg_sq', 'lag_5_wdsp_avg_sq', 'lag_6_wds

In [78]:
#selecting columns for X & y
cols = train.columns.to_list()
x_names = cols[2:6]
lags = cols[-47:]
x_names.extend(lags)
y_names = [cols[1]]
time_steps = cols[6:29]
y_names.extend(time_steps)
print(x_names)
print(y_names)

['month', 'dayofyear_num', 'weekofyear_num', 'wdsp_avg_sq', 'lag_1_elec_prices', 'lag_2_elec_prices', 'lag_3_elec_prices', 'lag_4_elec_prices', 'lag_5_elec_prices', 'lag_6_elec_prices', 'lag_7_elec_prices', 'lag_8_elec_prices', 'lag_9_elec_prices', 'lag_10_elec_prices', 'lag_11_elec_prices', 'lag_12_elec_prices', 'lag_13_elec_prices', 'lag_14_elec_prices', 'lag_15_elec_prices', 'lag_16_elec_prices', 'lag_17_elec_prices', 'lag_18_elec_prices', 'lag_19_elec_prices', 'lag_20_elec_prices', 'lag_21_elec_prices', 'lag_22_elec_prices', 'lag_23_elec_prices', 'lag_24_elec_prices', 'lag_1_wdsp_avg_sq', 'lag_2_wdsp_avg_sq', 'lag_3_wdsp_avg_sq', 'lag_4_wdsp_avg_sq', 'lag_5_wdsp_avg_sq', 'lag_6_wdsp_avg_sq', 'lag_7_wdsp_avg_sq', 'lag_8_wdsp_avg_sq', 'lag_9_wdsp_avg_sq', 'lag_10_wdsp_avg_sq', 'lag_11_wdsp_avg_sq', 'lag_12_wdsp_avg_sq', 'lag_13_wdsp_avg_sq', 'lag_14_wdsp_avg_sq', 'lag_15_wdsp_avg_sq', 'lag_16_wdsp_avg_sq', 'lag_17_wdsp_avg_sq', 'lag_18_wdsp_avg_sq', 'lag_19_wdsp_avg_sq', 'lag_20_wdsp

In [79]:
x_train, y_train = train[x_names], train[y_names]
x_test, y_test = test[x_names], test[y_names]
print(x_train.shape,y_train.shape)
print(x_test.shape,y_test.shape)

(276, 51) (276, 24)
(69, 51) (69, 24)


In [80]:
x_train_norm,x_test_norm = normalize(x_train,x_test)

In [81]:
GBM_training(x_train_norm,y_train)

Duration: 0:00:17.178796



{'mean_fit_time': array([1.16715224, 0.79060259, 1.30543711, 1.76856117, 0.50796747,
       1.47047176, 1.25915604, 1.10477786, 1.66615272, 1.67272787]), 'std_fit_time': array([0.03837145, 0.07256069, 0.09924324, 0.08527566, 0.06105159,
       0.0390178 , 0.02948039, 0.04581741, 0.08016151, 0.29628552]), 'mean_score_time': array([0.01952689, 0.01760192, 0.01733341, 0.02320359, 0.01952338,
       0.01920183, 0.02108006, 0.02000279, 0.02000203, 0.00960028]), 'std_score_time': array([0.00403776, 0.00320008, 0.00273005, 0.00560055, 0.00520003,
       0.00391919, 0.00762141, 0.0040009 , 0.00400004, 0.00480068]), 'param_estimator__subsample': masked_array(data=[0.75, 0.85, 0.75, 0.8, 0.6, 0.7, 0.75, 0.8, 0.7, 0.5],
             mask=[False, False, False, False, False, False, False, False,
                   False, False],
       fill_value='?',
            dtype=object), 'param_estimator__n_estimators': masked_array(data=[60, 40, 70, 90, 20, 80, 70, 60, 90, 100],


In [82]:
#learning_rate,n_estimators,max_depth,min_samples_split,max_features,min_samples_leaf,subsample
gbm5 = GBM_model(x_train_norm,y_train,0.0125,100,5,100,4,40,0.5)

Duration: 0:00:01.781604


In [83]:
y_pred= gbm5.predict(x_test_norm)
eval_metrics(y_test,y_pred)

MAE:  11.1146
MSE:  243.0179
RMSE:  15.589
MAPE:  23.6785
Coefficient of determination (R-squared):  0.0102




(11.114573813002279,
 243.01794669158122,
 15.58903289789271,
 23.678529277255357,
 0.010156974245822148)

In [84]:
XGBM_training(x_train_norm,y_train)
# def XGBM_model(X,y,learning_rate,n_estimators,max_depth,subsample,colsample_bytree):

Duration: 0:02:02.785836



{'mean_fit_time': array([17.08888087,  9.50101132,  5.1661201 , 16.95782909, 14.92631729,
       27.94508731,  2.7283848 , 30.48591654,  1.94191594,  5.63181181]), 'std_fit_time': array([0.68722742, 1.9836147 , 0.3316538 , 0.91334967, 0.58920936,
       1.37972703, 0.17394082, 0.98216225, 0.02943965, 0.46955505]), 'mean_score_time': array([0.07012973, 0.07550805, 0.05001915, 0.04864876, 0.05680661,
       0.09730017, 0.05069547, 0.06560726, 0.03552012, 0.030405  ]), 'std_score_time': array([0.01296647, 0.02305086, 0.00713283, 0.0043638 , 0.00754798,
       0.0096052 , 0.01634535, 0.01631829, 0.00377258, 0.00480015]), 'param_estimator__subsample': masked_array(data=[0.8, 0.2, 0.4, 1.0, 0.8, 0.8, 0.2, 1.0,
                   0.7000000000000001, 1.0],
             mask=[False, False, False, False, False, False, False, False,
                   False, False],
       fill_value='?',
            dtype=object), 'param_estimator__n_estimators': masked_array(data=[35

In [85]:
#learning_rate,n_estimators,max_depth,subsample,colsample_bytree)
xgb5 = XGBM_model(x_train_norm,y_train,0.025,350,13,0.8,0.4)

Duration: 0:00:07.752241


In [86]:
y_pred= xgb5.predict(x_test_norm)
eval_metrics(y_test,y_pred)

MAE:  10.4503
MSE:  234.0328
RMSE:  15.2981
MAPE:  22.2633
Coefficient of determination (R-squared):  0.1552




(10.450292074498348,
 234.03283367600977,
 15.298131705408009,
 22.26334099760124,
 0.1551772609980425)

In [87]:
LGBM_training(x_train_norm,y_train)

[LightGBM] [Warning] min_child_samples is set=20, min_child_samples= will be ignored. Current value: min_child_samples=20
[LightGBM] [Warning] Unknown parameter: 40
Duration: 0:00:46.294536



{'mean_fit_time': array([ 5.50115898,  8.74902201,  1.18010442,  3.35124333,  3.85258625,
        3.07864966,  5.57991691,  3.70962961, 10.64737973,  3.17002885]), 'std_fit_time': array([0.16369127, 0.26751051, 0.06567371, 0.03602533, 0.06219553,
       0.05758515, 0.07546711, 0.07875671, 0.28982779, 0.14219067]), 'mean_score_time': array([0.06757143, 0.04093096, 0.02697158, 0.04880691, 0.04899335,
       0.05220282, 0.06045012, 0.0513206 , 0.05648909, 0.04332583]), 'std_score_time': array([0.01953172, 0.00430056, 0.02069964, 0.01406536, 0.01657202,
       0.01758585, 0.01828609, 0.01191221, 0.01512399, 0.01102107]), 'param_estimator__subsample': masked_array(data=[0.4, 0.30000000000000004, 0.9, 0.4, 0.1, 0.8, 0.9,
                   0.30000000000000004, 0.8, 0.2],
             mask=[False, False

In [88]:
#num_leaves,min_child_samples,learning_rate,n_estimators,max_depth,subsample,colsample_bytree
lgbm5 = LGBM_model(x_train_norm,y_train,5,40,0.0125,400,9,0.2,0.2)

Duration: 0:00:02.493023


In [89]:
y_pred= lgbm5.predict(x_test_norm)
eval_metrics(y_test,y_pred)

MAE:  10.5456
MSE:  220.7817
RMSE:  14.8587
MAPE:  22.4665
Coefficient of determination (R-squared):  0.1485




(10.545644680559782,
 220.78167161854807,
 14.858723754702087,
 22.466480543235193,
 0.14847417936918786)

### Testing on 2018-2019 data

In [90]:
data_test = data_1819.copy()
data_test = feature_pipeline_1(data_test)
wdsp_sum= data_1819['wdsp_d']+data_1819['wdsp_c']+data_1819['wdsp_g']
wdsp_avg = wdsp_sum/3
wdsp_avg_sq = wdsp_avg*wdsp_avg
data_test['wdsp_avg_sq'] = round(wdsp_avg_sq,4)
#data_test['wdsp_sqr'] = round(data_1819['wdsp_d']*data_1819['wdsp_d']+data_1819['wdsp_c']*data_1819['wdsp_c']+data_1819['wdsp_g']*data_1819['wdsp_g'],4)
data_test = multi_step(data_test,'elec_prices')
data_test = calc_24lags(data_test,'elec_prices')
data_test = calc_24lags(data_test,'wdsp_avg_sq')
print(data_test.shape)
print(data_test.columns)
data_test.head()

(10514, 77)
Index(['applicable_date', 'elec_prices', 'month', 'dayofyear_num',
       'weekofyear_num', 'Hour', 'wdsp_avg_sq', 't+1', 't+2', 't+3', 't+4',
       't+5', 't+6', 't+7', 't+8', 't+9', 't+10', 't+11', 't+12', 't+13',
       't+14', 't+15', 't+16', 't+17', 't+18', 't+19', 't+20', 't+21', 't+22',
       't+23', 'lag_1_elec_prices', 'lag_2_elec_prices', 'lag_3_elec_prices',
       'lag_4_elec_prices', 'lag_5_elec_prices', 'lag_6_elec_prices',
       'lag_7_elec_prices', 'lag_8_elec_prices', 'lag_9_elec_prices',
       'lag_10_elec_prices', 'lag_11_elec_prices', 'lag_12_elec_prices',
       'lag_13_elec_prices', 'lag_14_elec_prices', 'lag_15_elec_prices',
       'lag_16_elec_prices', 'lag_17_elec_prices', 'lag_18_elec_prices',
       'lag_19_elec_prices', 'lag_20_elec_prices', 'lag_21_elec_prices',
       'lag_22_elec_prices', 'lag_23_elec_prices', 'lag_24_elec_prices',
       'lag_1_wdsp_avg_sq', 'lag_2_wdsp_avg_sq', 'lag_3_wdsp_avg_sq',
       'lag_4_wdsp_avg_sq', 'lag_5_wdsp

,applicable_date,elec_prices,month,dayofyear_num,weekofyear_num,Hour,wdsp_avg_sq,t+1,t+2,t+3,...,lag_14_wdsp_avg_sq,lag_15_wdsp_avg_sq,lag_16_wdsp_avg_sq,lag_17_wdsp_avg_sq,lag_18_wdsp_avg_sq,lag_19_wdsp_avg_sq,lag_20_wdsp_avg_sq,lag_21_wdsp_avg_sq,lag_22_wdsp_avg_sq,lag_23_wdsp_avg_sq
0,2018-09-30 22:00:00,71.267,9,273,39,22,9.5111,67.212,60.500,63.682,...,9.5111,9.5111,9.5111,9.5111,9.5111,9.5111,9.5111,9.5111,9.5111,9.5111
1,2018-09-30 23:00:00,67.212,9,273,39,23,11.7420,60.500,63.682,71.617,...,9.5111,9.5111,9.5111,9.5111,9.5111,9.5111,9.5111,9.5111,9.5111,9.5111
2,2018-10-01 00:00:00,60.500,10,274,40,0,12.9456,63.682,71.617,72.855,...,9.5111,9.5111,9.5111,9.5111,9.5111,9.5111,9.5111,9.5111,9.5111,9.5111
3,2018-10-01 01:00:00,63.682,10,274,40,1,12.9456,71.617,72.855,76.937,...,9.5111,9.5111,9.5111,9.5111,9.5111,9.5111,9.5111,9.5111,9.5111,9.5111
4,2018-10-01 02:00:00,71.617,10,274,40,2,7.5149,72.855,76.937,80.480,...,9.5111,9.5111,9.5111,9.5111,9.5111,9.5111,9.5111,9.5111,9.5111,9.5111


In [91]:
#Dropping rows with Nan's
data_test = data_test.dropna(axis=0)
print(data_test.shape)
data_test.isnull().values.any()

(10491, 77)


False

In [92]:
data_test = remove_23rows_hour_col(data_test)
data_test.columns

(438, 76)


Index(['applicable_date', 'elec_prices', 'month', 'dayofyear_num',
       'weekofyear_num', 'wdsp_avg_sq', 't+1', 't+2', 't+3', 't+4', 't+5',
       't+6', 't+7', 't+8', 't+9', 't+10', 't+11', 't+12', 't+13', 't+14',
       't+15', 't+16', 't+17', 't+18', 't+19', 't+20', 't+21', 't+22', 't+23',
       'lag_1_elec_prices', 'lag_2_elec_prices', 'lag_3_elec_prices',
       'lag_4_elec_prices', 'lag_5_elec_prices', 'lag_6_elec_prices',
       'lag_7_elec_prices', 'lag_8_elec_prices', 'lag_9_elec_prices',
       'lag_10_elec_prices', 'lag_11_elec_prices', 'lag_12_elec_prices',
       'lag_13_elec_prices', 'lag_14_elec_prices', 'lag_15_elec_prices',
       'lag_16_elec_prices', 'lag_17_elec_prices', 'lag_18_elec_prices',
       'lag_19_elec_prices', 'lag_20_elec_prices', 'lag_21_elec_prices',
       'lag_22_elec_prices', 'lag_23_elec_prices', 'lag_24_elec_prices',
       'lag_1_wdsp_avg_sq', 'lag_2_wdsp_avg_sq', 'lag_3_wdsp_avg_sq',
       'lag_4_wdsp_avg_sq', 'lag_5_wdsp_avg_sq', 'lag_6_wds

In [93]:
#selecting columns for X & y
cols = train.columns.to_list()
x_names = cols[2:6]
lags = cols[-47:]
x_names.extend(lags)
y_names = [cols[1]]
time_steps = cols[6:29]
y_names.extend(time_steps)
print(x_names)
print(y_names)

['month', 'dayofyear_num', 'weekofyear_num', 'wdsp_avg_sq', 'lag_1_elec_prices', 'lag_2_elec_prices', 'lag_3_elec_prices', 'lag_4_elec_prices', 'lag_5_elec_prices', 'lag_6_elec_prices', 'lag_7_elec_prices', 'lag_8_elec_prices', 'lag_9_elec_prices', 'lag_10_elec_prices', 'lag_11_elec_prices', 'lag_12_elec_prices', 'lag_13_elec_prices', 'lag_14_elec_prices', 'lag_15_elec_prices', 'lag_16_elec_prices', 'lag_17_elec_prices', 'lag_18_elec_prices', 'lag_19_elec_prices', 'lag_20_elec_prices', 'lag_21_elec_prices', 'lag_22_elec_prices', 'lag_23_elec_prices', 'lag_24_elec_prices', 'lag_1_wdsp_avg_sq', 'lag_2_wdsp_avg_sq', 'lag_3_wdsp_avg_sq', 'lag_4_wdsp_avg_sq', 'lag_5_wdsp_avg_sq', 'lag_6_wdsp_avg_sq', 'lag_7_wdsp_avg_sq', 'lag_8_wdsp_avg_sq', 'lag_9_wdsp_avg_sq', 'lag_10_wdsp_avg_sq', 'lag_11_wdsp_avg_sq', 'lag_12_wdsp_avg_sq', 'lag_13_wdsp_avg_sq', 'lag_14_wdsp_avg_sq', 'lag_15_wdsp_avg_sq', 'lag_16_wdsp_avg_sq', 'lag_17_wdsp_avg_sq', 'lag_18_wdsp_avg_sq', 'lag_19_wdsp_avg_sq', 'lag_20_wdsp

In [94]:
#GBM
mae,mse,rmse,mape,r2 = testing_runs(10,data_test,gbm5)

Round 1: 
(350, 51) (44, 51) (44, 51)
(350, 24) (44, 24) (44, 24)
MAE:  12.4548
MSE:  367.9409
RMSE:  19.1818
MAPE:  22.0615
Coefficient of determination (R-squared):  0.2003


Round 2: 
(350, 51) (44, 51) (44, 51)
(350, 24) (44, 24) (44, 24)
MAE:  12.5829
MSE:  361.3574
RMSE:  19.0094
MAPE:  23.2437
Coefficient of determination (R-squared):  0.224


Round 3: 
(350, 51) (44, 51) (44, 51)
(350, 24) (44, 24) (44, 24)
MAE:  10.3508
MSE:  215.0739
RMSE:  14.6654
MAPE:  20.5064
Coefficient of determination (R-squared):  0.2653


Round 4: 
(350, 51) (44, 51) (44, 51)
(350, 24) (44, 24) (44, 24)
MAE:  14.1661
MSE:  445.8843
RMSE:  21.116
MAPE:  23.7372
Coefficient of determination (R-squared):  0.2208


Round 5: 
(350, 51) (44, 51) (44, 51)
(350, 24) (44, 24) (44, 24)
MAE:  13.4756
MSE:  392.1029
RMSE:  19.8016
MAPE:  24.0775
Coefficient of determination (R-squared):  0.2638


Round 6: 
(350, 51) (44, 51) (44, 51)
(350, 24) (44, 24) (44, 24)
MAE:  11.8331
MSE:  247.566
RMSE:  15.7342
MAPE:  2

In [95]:
#XGBM
mae,mse,rmse,mape,r2 = testing_runs(10,data_test,xgb5)

Round 1: 
(350, 51) (44, 51) (44, 51)
(350, 24) (44, 24) (44, 24)
MAE:  6.7752
MSE:  103.4473
RMSE:  10.1709
MAPE:  12.6796
Coefficient of determination (R-squared):  0.768


Round 2: 
(350, 51) (44, 51) (44, 51)
(350, 24) (44, 24) (44, 24)
MAE:  9.5053
MSE:  333.3745
RMSE:  18.2585
MAPE:  16.7053
Coefficient of determination (R-squared):  0.5795


Round 3: 
(350, 51) (44, 51) (44, 51)
(350, 24) (44, 24) (44, 24)
MAE:  8.4095
MSE:  199.2479
RMSE:  14.1155
MAPE:  15.7055
Coefficient of determination (R-squared):  0.6052


Round 4: 
(350, 51) (44, 51) (44, 51)
(350, 24) (44, 24) (44, 24)
MAE:  9.4942
MSE:  199.7108
RMSE:  14.1319
MAPE:  17.5616
Coefficient of determination (R-squared):  0.5431


Round 5: 
(350, 51) (44, 51) (44, 51)
(350, 24) (44, 24) (44, 24)
MAE:  8.9801
MSE:  223.2121
RMSE:  14.9403
MAPE:  16.2454
Coefficient of determination (R-squared):  0.48


Round 6: 
(350, 51) (44, 51) (44, 51)
(350, 24) (44, 24) (44, 24)
MAE:  10.7009
MSE:  329.5694
RMSE:  18.154
MAPE:  18.2843

In [96]:
#light GBM
mae,mse,rmse,mape,r2 = testing_runs(10,data_test,lgbm5)

Round 1: 
(350, 51) (44, 51) (44, 51)
(350, 24) (44, 24) (44, 24)
MAE:  9.0408
MSE:  195.8713
RMSE:  13.9954
MAPE:  17.1228
Coefficient of determination (R-squared):  0.4914


Round 2: 
(350, 51) (44, 51) (44, 51)
(350, 24) (44, 24) (44, 24)
MAE:  9.9033
MSE:  215.9295
RMSE:  14.6945
MAPE:  18.8826
Coefficient of determination (R-squared):  0.5043


Round 3: 
(350, 51) (44, 51) (44, 51)
(350, 24) (44, 24) (44, 24)
MAE:  12.3819
MSE:  364.1248
RMSE:  19.0821
MAPE:  21.521
Coefficient of determination (R-squared):  0.4746


Round 4: 
(350, 51) (44, 51) (44, 51)
(350, 24) (44, 24) (44, 24)
MAE:  9.5203
MSE:  248.6514
RMSE:  15.7687
MAPE:  18.3418
Coefficient of determination (R-squared):  0.5321


Round 5: 
(350, 51) (44, 51) (44, 51)
(350, 24) (44, 24) (44, 24)
MAE:  10.1078
MSE:  221.6922
RMSE:  14.8893
MAPE:  20.2331
Coefficient of determination (R-squared):  0.4226


Round 6: 
(350, 51) (44, 51) (44, 51)
(350, 24) (44, 24) (44, 24)
MAE:  10.8175
MSE:  272.975
RMSE:  16.522
MAPE:  20.1

<h1 style="text-align:center;font-size:40px;" > Basic time based features + wdsp_avg_sqrt (squareroot of sum of windspeeds in all counties) </h1>

In [48]:
data_1= data.copy()
data_1 = feature_pipeline_1(data_1)
wdsp_avg= (data['wdsp_d']+data['wdsp_c']+data['wdsp_g'])/3
#wdsp_sqr= (data['wdsp_d']*data['wdsp_d']+data['wdsp_c']*data['wdsp_c']+data['wdsp_g']*data['wdsp_g']).values
wdsp_avg_sqrt= np.sqrt(wdsp_avg)
data_1['wdsp_avg_sqrt'] = wdsp_avg_sqrt
data_1.head()

,applicable_date,elec_prices,month,dayofyear_num,weekofyear_num,Hour,wdsp_avg_sqrt
0,2019-01-01 00:00:00,70.28,1,1,1,0,1.706654
1,2019-01-01 01:00:00,65.18,1,1,1,1,1.655697
2,2019-01-01 02:00:00,60.91,1,1,1,2,1.655697
3,2019-01-01 03:00:00,65.16,1,1,1,3,1.756132
4,2019-01-01 04:00:00,65.16,1,1,1,4,1.804254


In [49]:
data_1 = multi_step(data_1,'elec_prices')
data_1 = calc_24lags(data_1,'elec_prices')
data_1 = calc_24lags(data_1,'wdsp_avg_sqrt')
print(data_1.shape)
print(data_1.columns)
data_1.head()

(8304, 77)
Index(['applicable_date', 'elec_prices', 'month', 'dayofyear_num',
       'weekofyear_num', 'Hour', 'wdsp_avg_sqrt', 't+1', 't+2', 't+3', 't+4',
       't+5', 't+6', 't+7', 't+8', 't+9', 't+10', 't+11', 't+12', 't+13',
       't+14', 't+15', 't+16', 't+17', 't+18', 't+19', 't+20', 't+21', 't+22',
       't+23', 'lag_1_elec_prices', 'lag_2_elec_prices', 'lag_3_elec_prices',
       'lag_4_elec_prices', 'lag_5_elec_prices', 'lag_6_elec_prices',
       'lag_7_elec_prices', 'lag_8_elec_prices', 'lag_9_elec_prices',
       'lag_10_elec_prices', 'lag_11_elec_prices', 'lag_12_elec_prices',
       'lag_13_elec_prices', 'lag_14_elec_prices', 'lag_15_elec_prices',
       'lag_16_elec_prices', 'lag_17_elec_prices', 'lag_18_elec_prices',
       'lag_19_elec_prices', 'lag_20_elec_prices', 'lag_21_elec_prices',
       'lag_22_elec_prices', 'lag_23_elec_prices', 'lag_24_elec_prices',
       'lag_1_wdsp_avg_sqrt', 'lag_2_wdsp_avg_sqrt', 'lag_3_wdsp_avg_sqrt',
       'lag_4_wdsp_avg_sqrt', 'l

,applicable_date,elec_prices,month,dayofyear_num,weekofyear_num,Hour,wdsp_avg_sqrt,t+1,t+2,t+3,...,lag_14_wdsp_avg_sqrt,lag_15_wdsp_avg_sqrt,lag_16_wdsp_avg_sqrt,lag_17_wdsp_avg_sqrt,lag_18_wdsp_avg_sqrt,lag_19_wdsp_avg_sqrt,lag_20_wdsp_avg_sqrt,lag_21_wdsp_avg_sqrt,lag_22_wdsp_avg_sqrt,lag_23_wdsp_avg_sqrt
0,2019-01-01 00:00:00,70.28,1,1,1,0,1.706654,65.18,60.91,65.160,...,1.706654,1.706654,1.706654,1.706654,1.706654,1.706654,1.706654,1.706654,1.706654,1.706654
1,2019-01-01 01:00:00,65.18,1,1,1,1,1.655697,60.91,65.16,65.160,...,1.706654,1.706654,1.706654,1.706654,1.706654,1.706654,1.706654,1.706654,1.706654,1.706654
2,2019-01-01 02:00:00,60.91,1,1,1,2,1.655697,65.16,65.16,68.290,...,1.706654,1.706654,1.706654,1.706654,1.706654,1.706654,1.706654,1.706654,1.706654,1.706654
3,2019-01-01 03:00:00,65.16,1,1,1,3,1.756132,65.16,68.29,63.030,...,1.706654,1.706654,1.706654,1.706654,1.706654,1.706654,1.706654,1.706654,1.706654,1.706654
4,2019-01-01 04:00:00,65.16,1,1,1,4,1.804254,68.29,63.03,51.621,...,1.706654,1.706654,1.706654,1.706654,1.706654,1.706654,1.706654,1.706654,1.706654,1.706654


In [50]:
data_1.isnull().values.any()

True

In [51]:
#Dropping rows with Nan's
data_1 = data_1.dropna(axis=0)
print(data_1.shape)
data_1.isnull().values.any()

(8281, 77)


False

In [52]:
data_1 = remove_23rows_hour_col(data_1)
train,test = train_test_split_tbs(data_1)
train.columns

(345, 76)
(276, 76)
(69, 76)


Index(['applicable_date', 'elec_prices', 'month', 'dayofyear_num',
       'weekofyear_num', 'wdsp_avg_sqrt', 't+1', 't+2', 't+3', 't+4', 't+5',
       't+6', 't+7', 't+8', 't+9', 't+10', 't+11', 't+12', 't+13', 't+14',
       't+15', 't+16', 't+17', 't+18', 't+19', 't+20', 't+21', 't+22', 't+23',
       'lag_1_elec_prices', 'lag_2_elec_prices', 'lag_3_elec_prices',
       'lag_4_elec_prices', 'lag_5_elec_prices', 'lag_6_elec_prices',
       'lag_7_elec_prices', 'lag_8_elec_prices', 'lag_9_elec_prices',
       'lag_10_elec_prices', 'lag_11_elec_prices', 'lag_12_elec_prices',
       'lag_13_elec_prices', 'lag_14_elec_prices', 'lag_15_elec_prices',
       'lag_16_elec_prices', 'lag_17_elec_prices', 'lag_18_elec_prices',
       'lag_19_elec_prices', 'lag_20_elec_prices', 'lag_21_elec_prices',
       'lag_22_elec_prices', 'lag_23_elec_prices', 'lag_24_elec_prices',
       'lag_1_wdsp_avg_sqrt', 'lag_2_wdsp_avg_sqrt', 'lag_3_wdsp_avg_sqrt',
       'lag_4_wdsp_avg_sqrt', 'lag_5_wdsp_avg_sqrt'

In [53]:
#selecting columns for X & y
cols = train.columns.to_list()
x_names = cols[2:6]
lags = cols[-47:]
x_names.extend(lags)
y_names = [cols[1]]
time_steps = cols[6:29]
y_names.extend(time_steps)
print(x_names)
print(y_names)

['month', 'dayofyear_num', 'weekofyear_num', 'wdsp_avg_sqrt', 'lag_1_elec_prices', 'lag_2_elec_prices', 'lag_3_elec_prices', 'lag_4_elec_prices', 'lag_5_elec_prices', 'lag_6_elec_prices', 'lag_7_elec_prices', 'lag_8_elec_prices', 'lag_9_elec_prices', 'lag_10_elec_prices', 'lag_11_elec_prices', 'lag_12_elec_prices', 'lag_13_elec_prices', 'lag_14_elec_prices', 'lag_15_elec_prices', 'lag_16_elec_prices', 'lag_17_elec_prices', 'lag_18_elec_prices', 'lag_19_elec_prices', 'lag_20_elec_prices', 'lag_21_elec_prices', 'lag_22_elec_prices', 'lag_23_elec_prices', 'lag_24_elec_prices', 'lag_1_wdsp_avg_sqrt', 'lag_2_wdsp_avg_sqrt', 'lag_3_wdsp_avg_sqrt', 'lag_4_wdsp_avg_sqrt', 'lag_5_wdsp_avg_sqrt', 'lag_6_wdsp_avg_sqrt', 'lag_7_wdsp_avg_sqrt', 'lag_8_wdsp_avg_sqrt', 'lag_9_wdsp_avg_sqrt', 'lag_10_wdsp_avg_sqrt', 'lag_11_wdsp_avg_sqrt', 'lag_12_wdsp_avg_sqrt', 'lag_13_wdsp_avg_sqrt', 'lag_14_wdsp_avg_sqrt', 'lag_15_wdsp_avg_sqrt', 'lag_16_wdsp_avg_sqrt', 'lag_17_wdsp_avg_sqrt', 'lag_18_wdsp_avg_sqr

In [54]:
x_train, y_train = train[x_names], train[y_names]
x_test, y_test = test[x_names], test[y_names]
print(x_train.shape,y_train.shape)
print(x_test.shape,y_test.shape)

(276, 51) (276, 24)
(69, 51) (69, 24)


In [55]:
x_train_norm,x_test_norm = normalize(x_train,x_test)

In [56]:
GBM_training(x_train_norm,y_train)

Duration: 0:00:10.212157



{'mean_fit_time': array([0.24205556, 0.78987186, 1.21097381, 0.79898174, 0.99232504,
       0.61303906, 0.98914223, 0.54295182, 1.1029506 , 0.75174603]), 'std_fit_time': array([0.00572895, 0.00940623, 0.01261228, 0.00676545, 0.01273475,
       0.0082292 , 0.00888633, 0.00862551, 0.01113902, 0.10882871]), 'mean_score_time': array([0.00950272, 0.01050181, 0.01280344, 0.01110229, 0.01120281,
       0.01050205, 0.01110644, 0.01010168, 0.01130295, 0.00730414]), 'std_score_time': array([0.00067104, 0.00067102, 0.00039953, 0.00083089, 0.00107724,
       0.00050025, 0.00053769, 0.00070073, 0.00090019, 0.00214671]), 'param_estimator__subsample': masked_array(data=[0.7, 0.5, 0.7, 0.75, 0.6, 0.7, 0.85, 0.9, 0.9, 0.75],
             mask=[False, False, False, False, False, False, False, False,
                   False, False],
       fill_value='?',
            dtype=object), 'param_estimator__n_estimators': masked_array(data=[20, 80, 90, 80, 100, 60, 100, 50, 70, 80],


In [57]:
#X,y,learning_rate,n_estimators,max_depth,min_samples_split,max_features,min_samples_leaf,subsample
gbm6 = GBM_model(x_train_norm,y_train,0.025,90,9,100,3,10,0.7)

Duration: 0:00:00.684813


In [58]:
y_pred= gbm6.predict(x_test_norm)
eval_metrics(y_test,y_pred)

MAE:  10.5617
MSE:  228.5466
RMSE:  15.1178
MAPE:  22.5006
Coefficient of determination (R-squared):  0.1174




(10.56167971037615,
 228.54658111688332,
 15.117757145717196,
 22.500641630232913,
 0.11741233880964506)

In [59]:
XGBM_training(x_train_norm,y_train)

Duration: 0:00:44.148946



{'mean_fit_time': array([ 5.24877017,  1.408567  ,  7.82825096,  1.60227177,  1.50263562,
        0.76262796,  0.8827055 ,  2.18781011, 18.401846  ,  6.3199934 ]), 'std_fit_time': array([0.12311057, 0.75473216, 0.16743116, 0.04935866, 0.03115538,
       0.01223546, 0.00840768, 0.0407246 , 0.55096997, 0.72616466]), 'mean_score_time': array([0.03420775, 0.02360551, 0.04050882, 0.02720644, 0.02460542,
       0.02220471, 0.02160535, 0.02940588, 0.06302676, 0.02851236]), 'std_score_time': array([0.0059139 , 0.00593763, 0.00128453, 0.00240004, 0.00276517,
       0.00060044, 0.00066385, 0.00237489, 0.01440646, 0.00608407]), 'param_estimator__subsample': masked_array(data=[0.30000000000000004, 0.2, 0.30000000000000004, 0.2,
                   0.7000000000000001, 0.1, 0.30000000000000004, 0.9, 0.8,
                   0.2],
             mask=[False, False, False, False, False, False, False, False,
                   False, False],
       fill_value='?',
            dt

In [60]:
# def XGBM_model(X,y,learning_rate,n_estimators,max_depth,subsample,colsample_bytree):
xgb6 = XGBM_model(x_train_norm,y_train,0.1,100,3,0.7,0.4)

Duration: 0:00:00.603142


In [61]:
y_pred= xgb6.predict(x_test_norm)
eval_metrics(y_test,y_pred)

MAE:  10.3016
MSE:  215.6364
RMSE:  14.6846
MAPE:  21.9465
Coefficient of determination (R-squared):  0.1719




(10.301557154752208,
 215.63643542740127,
 14.68456452971627,
 21.946475572888396,
 0.17186133463591322)

In [62]:
LGBM_training(x_train_norm,y_train)

[LightGBM] [Warning] min_child_samples is set=20, min_child_samples= will be ignored. Current value: min_child_samples=20
[LightGBM] [Warning] Unknown parameter: 70
Duration: 0:00:59.500979



{'mean_fit_time': array([ 2.96685903, 10.70252831,  4.48552938, 29.07256978,  2.81496546,
        3.57013006,  0.94965179,  2.51220706,  1.56596215,  6.35710018]), 'std_fit_time': array([0.09600472, 0.29840069, 0.4397459 , 0.7672146 , 0.082874  ,
       0.40526946, 0.02311585, 0.13400355, 0.08278572, 0.18958813]), 'mean_score_time': array([0.03150694, 0.04877081, 0.07891688, 0.05381238, 0.05351045,
       0.04181068, 0.01880445, 0.02390621, 0.02610641, 0.03536444]), 'std_score_time': array([0.00634585, 0.00276387, 0.06681024, 0.0113231 , 0.01148495,
       0.00697116, 0.00166163, 0.00443784, 0.00230006, 0.00894898]), 'param_estimator__subsample': masked_array(data=[0.30000000000000004, 0.7000000000000001, 0.9, 1.0, 0.6,
                   0.4, 1.0, 0.6, 0.5, 0.5],
             mask=[False, False,

In [63]:
#X,y,num_leaves,min_child_samples,learning_rate,n_estimators,max_depth,subsample,colsample_bytree
lgbm6 = LGBM_model(x_train_norm,y_train,4915,70,0.025,250,7,0.3,0.6)

Duration: 0:00:00.797684


In [64]:
y_pred= lgbm6.predict(x_test_norm)
eval_metrics(y_test,y_pred)

MAE:  10.7472
MSE:  225.6607
RMSE:  15.022
MAPE:  22.8958
Coefficient of determination (R-squared):  0.1062




(10.747168694356091,
 225.66069979981694,
 15.022007182790752,
 22.895808049717147,
 0.10618937693225183)

### Testing on 2018-2019 data

In [65]:
data_test = data_1819.copy()
data_test = feature_pipeline_1(data_test)
wdsp_avg= (data_1819['wdsp_d']+data_1819['wdsp_c']+data_1819['wdsp_g'])/3
wdsp_avg_sqrt= np.sqrt(wdsp_avg)
data_test['wdsp_avg_sqrt'] = wdsp_avg_sqrt

data_test = multi_step(data_test,'elec_prices')
data_test = calc_24lags(data_test,'elec_prices')
data_test = calc_24lags(data_test,'wdsp_avg_sqrt')
print(data_test.shape)
print(data_test.columns)
data_test.head()

(10514, 77)
Index(['applicable_date', 'elec_prices', 'month', 'dayofyear_num',
       'weekofyear_num', 'Hour', 'wdsp_avg_sqrt', 't+1', 't+2', 't+3', 't+4',
       't+5', 't+6', 't+7', 't+8', 't+9', 't+10', 't+11', 't+12', 't+13',
       't+14', 't+15', 't+16', 't+17', 't+18', 't+19', 't+20', 't+21', 't+22',
       't+23', 'lag_1_elec_prices', 'lag_2_elec_prices', 'lag_3_elec_prices',
       'lag_4_elec_prices', 'lag_5_elec_prices', 'lag_6_elec_prices',
       'lag_7_elec_prices', 'lag_8_elec_prices', 'lag_9_elec_prices',
       'lag_10_elec_prices', 'lag_11_elec_prices', 'lag_12_elec_prices',
       'lag_13_elec_prices', 'lag_14_elec_prices', 'lag_15_elec_prices',
       'lag_16_elec_prices', 'lag_17_elec_prices', 'lag_18_elec_prices',
       'lag_19_elec_prices', 'lag_20_elec_prices', 'lag_21_elec_prices',
       'lag_22_elec_prices', 'lag_23_elec_prices', 'lag_24_elec_prices',
       'lag_1_wdsp_avg_sqrt', 'lag_2_wdsp_avg_sqrt', 'lag_3_wdsp_avg_sqrt',
       'lag_4_wdsp_avg_sqrt', '

,applicable_date,elec_prices,month,dayofyear_num,weekofyear_num,Hour,wdsp_avg_sqrt,t+1,t+2,t+3,...,lag_14_wdsp_avg_sqrt,lag_15_wdsp_avg_sqrt,lag_16_wdsp_avg_sqrt,lag_17_wdsp_avg_sqrt,lag_18_wdsp_avg_sqrt,lag_19_wdsp_avg_sqrt,lag_20_wdsp_avg_sqrt,lag_21_wdsp_avg_sqrt,lag_22_wdsp_avg_sqrt,lag_23_wdsp_avg_sqrt
0,2018-09-30 22:00:00,71.267,9,273,39,22,1.756132,67.212,60.500,63.682,...,1.756132,1.756132,1.756132,1.756132,1.756132,1.756132,1.756132,1.756132,1.756132,1.756132
1,2018-09-30 23:00:00,67.212,9,273,39,23,1.851126,60.500,63.682,71.617,...,1.756132,1.756132,1.756132,1.756132,1.756132,1.756132,1.756132,1.756132,1.756132,1.756132
2,2018-10-01 00:00:00,60.500,10,274,40,0,1.896839,63.682,71.617,72.855,...,1.756132,1.756132,1.756132,1.756132,1.756132,1.756132,1.756132,1.756132,1.756132,1.756132
3,2018-10-01 01:00:00,63.682,10,274,40,1,1.896839,71.617,72.855,76.937,...,1.756132,1.756132,1.756132,1.756132,1.756132,1.756132,1.756132,1.756132,1.756132,1.756132
4,2018-10-01 02:00:00,71.617,10,274,40,2,1.655697,72.855,76.937,80.480,...,1.756132,1.756132,1.756132,1.756132,1.756132,1.756132,1.756132,1.756132,1.756132,1.756132


In [66]:
#Dropping rows with Nan's
data_test = data_test.dropna(axis=0)
print(data_test.shape)
data_test.isnull().values.any()

(10491, 77)


False

In [67]:
data_test = remove_23rows_hour_col(data_test)
data_test.columns

(438, 76)


Index(['applicable_date', 'elec_prices', 'month', 'dayofyear_num',
       'weekofyear_num', 'wdsp_avg_sqrt', 't+1', 't+2', 't+3', 't+4', 't+5',
       't+6', 't+7', 't+8', 't+9', 't+10', 't+11', 't+12', 't+13', 't+14',
       't+15', 't+16', 't+17', 't+18', 't+19', 't+20', 't+21', 't+22', 't+23',
       'lag_1_elec_prices', 'lag_2_elec_prices', 'lag_3_elec_prices',
       'lag_4_elec_prices', 'lag_5_elec_prices', 'lag_6_elec_prices',
       'lag_7_elec_prices', 'lag_8_elec_prices', 'lag_9_elec_prices',
       'lag_10_elec_prices', 'lag_11_elec_prices', 'lag_12_elec_prices',
       'lag_13_elec_prices', 'lag_14_elec_prices', 'lag_15_elec_prices',
       'lag_16_elec_prices', 'lag_17_elec_prices', 'lag_18_elec_prices',
       'lag_19_elec_prices', 'lag_20_elec_prices', 'lag_21_elec_prices',
       'lag_22_elec_prices', 'lag_23_elec_prices', 'lag_24_elec_prices',
       'lag_1_wdsp_avg_sqrt', 'lag_2_wdsp_avg_sqrt', 'lag_3_wdsp_avg_sqrt',
       'lag_4_wdsp_avg_sqrt', 'lag_5_wdsp_avg_sqrt'

In [68]:
#selecting columns for X & y
cols = train.columns.to_list()
x_names = cols[2:6]
lags = cols[-47:]
x_names.extend(lags)
y_names = [cols[1]]
time_steps = cols[6:29]
y_names.extend(time_steps)
print(x_names)
print(y_names)

['month', 'dayofyear_num', 'weekofyear_num', 'wdsp_avg_sqrt', 'lag_1_elec_prices', 'lag_2_elec_prices', 'lag_3_elec_prices', 'lag_4_elec_prices', 'lag_5_elec_prices', 'lag_6_elec_prices', 'lag_7_elec_prices', 'lag_8_elec_prices', 'lag_9_elec_prices', 'lag_10_elec_prices', 'lag_11_elec_prices', 'lag_12_elec_prices', 'lag_13_elec_prices', 'lag_14_elec_prices', 'lag_15_elec_prices', 'lag_16_elec_prices', 'lag_17_elec_prices', 'lag_18_elec_prices', 'lag_19_elec_prices', 'lag_20_elec_prices', 'lag_21_elec_prices', 'lag_22_elec_prices', 'lag_23_elec_prices', 'lag_24_elec_prices', 'lag_1_wdsp_avg_sqrt', 'lag_2_wdsp_avg_sqrt', 'lag_3_wdsp_avg_sqrt', 'lag_4_wdsp_avg_sqrt', 'lag_5_wdsp_avg_sqrt', 'lag_6_wdsp_avg_sqrt', 'lag_7_wdsp_avg_sqrt', 'lag_8_wdsp_avg_sqrt', 'lag_9_wdsp_avg_sqrt', 'lag_10_wdsp_avg_sqrt', 'lag_11_wdsp_avg_sqrt', 'lag_12_wdsp_avg_sqrt', 'lag_13_wdsp_avg_sqrt', 'lag_14_wdsp_avg_sqrt', 'lag_15_wdsp_avg_sqrt', 'lag_16_wdsp_avg_sqrt', 'lag_17_wdsp_avg_sqrt', 'lag_18_wdsp_avg_sqr

In [69]:
#GBM
mae,mse,rmse,mape,r2 = testing_runs(10,data_test,gbm6)

Round 1: 
(350, 51) (44, 51) (44, 51)
(350, 24) (44, 24) (44, 24)
MAE:  11.4015
MSE:  272.0212
RMSE:  16.4931
MAPE:  19.6889
Coefficient of determination (R-squared):  0.4244


Round 2: 
(350, 51) (44, 51) (44, 51)
(350, 24) (44, 24) (44, 24)
MAE:  10.2277
MSE:  228.1032
RMSE:  15.1031
MAPE:  19.3926
Coefficient of determination (R-squared):  0.4871


Round 3: 
(350, 51) (44, 51) (44, 51)
(350, 24) (44, 24) (44, 24)
MAE:  10.3285
MSE:  261.4466
RMSE:  16.1693
MAPE:  19.2426
Coefficient of determination (R-squared):  0.4295


Round 4: 
(350, 51) (44, 51) (44, 51)
(350, 24) (44, 24) (44, 24)
MAE:  11.0041
MSE:  290.4813
RMSE:  17.0435
MAPE:  19.3772
Coefficient of determination (R-squared):  0.5108


Round 5: 
(350, 51) (44, 51) (44, 51)
(350, 24) (44, 24) (44, 24)
MAE:  11.6307
MSE:  302.7691
RMSE:  17.4003
MAPE:  21.3034
Coefficient of determination (R-squared):  0.415


Round 6: 
(350, 51) (44, 51) (44, 51)
(350, 24) (44, 24) (44, 24)
MAE:  10.938
MSE:  304.2611
RMSE:  17.4431
MAPE:  

In [70]:
#XGBM
mae,mse,rmse,mape,r2 = testing_runs(10,data_test,xgb6)

Round 1: 
(350, 51) (44, 51) (44, 51)
(350, 24) (44, 24) (44, 24)
MAE:  9.9594
MSE:  238.3592
RMSE:  15.4389
MAPE:  19.3215
Coefficient of determination (R-squared):  0.5278


Round 2: 
(350, 51) (44, 51) (44, 51)
(350, 24) (44, 24) (44, 24)
MAE:  9.884
MSE:  254.3929
RMSE:  15.9497
MAPE:  18.2565
Coefficient of determination (R-squared):  0.5178


Round 3: 
(350, 51) (44, 51) (44, 51)
(350, 24) (44, 24) (44, 24)
MAE:  10.9517
MSE:  281.3749
RMSE:  16.7742
MAPE:  18.7302
Coefficient of determination (R-squared):  0.5523


Round 4: 
(350, 51) (44, 51) (44, 51)
(350, 24) (44, 24) (44, 24)
MAE:  8.9519
MSE:  179.0159
RMSE:  13.3797
MAPE:  15.7251
Coefficient of determination (R-squared):  0.7051


Round 5: 
(350, 51) (44, 51) (44, 51)
(350, 24) (44, 24) (44, 24)
MAE:  9.6478
MSE:  204.6213
RMSE:  14.3046
MAPE:  17.6825
Coefficient of determination (R-squared):  0.4437


Round 6: 
(350, 51) (44, 51) (44, 51)
(350, 24) (44, 24) (44, 24)
MAE:  8.2253
MSE:  146.3129
RMSE:  12.096
MAPE:  15.50

In [71]:
#light GBM
mae,mse,rmse,mape,r2 = testing_runs(10,data_test,lgbm6)

Round 1: 
(350, 51) (44, 51) (44, 51)
(350, 24) (44, 24) (44, 24)
MAE:  11.046
MSE:  399.0907
RMSE:  19.9773
MAPE:  19.2476
Coefficient of determination (R-squared):  0.4418


Round 2: 
(350, 51) (44, 51) (44, 51)
(350, 24) (44, 24) (44, 24)
MAE:  11.408
MSE:  318.2932
RMSE:  17.8408
MAPE:  20.5872
Coefficient of determination (R-squared):  0.4278


Round 3: 
(350, 51) (44, 51) (44, 51)
(350, 24) (44, 24) (44, 24)
MAE:  10.2455
MSE:  289.4847
RMSE:  17.0142
MAPE:  18.2744
Coefficient of determination (R-squared):  0.4604


Round 4: 
(350, 51) (44, 51) (44, 51)
(350, 24) (44, 24) (44, 24)
MAE:  10.8495
MSE:  266.717
RMSE:  16.3315
MAPE:  19.3138
Coefficient of determination (R-squared):  0.485


Round 5: 
(350, 51) (44, 51) (44, 51)
(350, 24) (44, 24) (44, 24)
MAE:  11.6522
MSE:  368.3558
RMSE:  19.1926
MAPE:  20.5458
Coefficient of determination (R-squared):  0.4605


Round 6: 
(350, 51) (44, 51) (44, 51)
(350, 24) (44, 24) (44, 24)
MAE:  10.1562
MSE:  234.1412
RMSE:  15.3017
MAPE:  18

In [97]:
import pickle
# save the model to disk
pickle.dump(gbm4, open('GBM_basic_wdsp_g_model.sav', 'wb'))
pickle.dump(gbm5, open('GBM_basic_wdsp_avg_sqr_model.sav', 'wb'))
pickle.dump(gbm6, open('GBM_basic_wdsp_avg_sqrt_model.sav', 'wb'))

pickle.dump(xgb4, open('XGBM_basic_wdsp_g_model.sav', 'wb'))
pickle.dump(xgb5, open('XGBM_basic_wdsp_avg_sqr_model.sav', 'wb'))
pickle.dump(xgb6, open('XGBM_basic_wdsp_avg_sqrt_model.sav', 'wb'))

pickle.dump(lgbm4, open('LightGBM_basic_wdsp_g_model.sav', 'wb'))
pickle.dump(lgbm5, open('LightGBM_basic_wdsp_avg_sqr_model.sav', 'wb'))
pickle.dump(lgbm6, open('LightGBM_basic_wdsp_avg_sqrt_model.sav', 'wb'))
#gbm = pickle.load(open('GBMtimefeatures1_model.sav', 'rb'))

<h1 style="text-align:center;font-size:40px;" > Basic time based features + wdsp_avg_log (log of avg of windspeeds in all counties)</h1>

In [98]:
data_1= data.copy()
data_1 = feature_pipeline_1(data_1)
wdsp_avg= (data['wdsp_d']+data['wdsp_c']+data['wdsp_g'])/3
wdsp_avg_log = np.log(wdsp_avg)
data_1['wdsp_avg_log'] = wdsp_avg_log
data_1.head()

,applicable_date,elec_prices,month,dayofyear_num,weekofyear_num,Hour,wdsp_avg_log
0,2019-01-01 00:00:00,70.28,1,1,1,0,1.069069
1,2019-01-01 01:00:00,65.18,1,1,1,1,1.008444
2,2019-01-01 02:00:00,60.91,1,1,1,2,1.008444
3,2019-01-01 03:00:00,65.16,1,1,1,3,1.126227
4,2019-01-01 04:00:00,65.16,1,1,1,4,1.180295


In [99]:
data_1 = multi_step(data_1,'elec_prices')
data_1 = calc_24lags(data_1,'elec_prices')
data_1 = calc_24lags(data_1,'wdsp_avg_log')
print(data_1.shape)
print(data_1.columns)
data_1.head()

(8304, 77)
Index(['applicable_date', 'elec_prices', 'month', 'dayofyear_num',
       'weekofyear_num', 'Hour', 'wdsp_avg_log', 't+1', 't+2', 't+3', 't+4',
       't+5', 't+6', 't+7', 't+8', 't+9', 't+10', 't+11', 't+12', 't+13',
       't+14', 't+15', 't+16', 't+17', 't+18', 't+19', 't+20', 't+21', 't+22',
       't+23', 'lag_1_elec_prices', 'lag_2_elec_prices', 'lag_3_elec_prices',
       'lag_4_elec_prices', 'lag_5_elec_prices', 'lag_6_elec_prices',
       'lag_7_elec_prices', 'lag_8_elec_prices', 'lag_9_elec_prices',
       'lag_10_elec_prices', 'lag_11_elec_prices', 'lag_12_elec_prices',
       'lag_13_elec_prices', 'lag_14_elec_prices', 'lag_15_elec_prices',
       'lag_16_elec_prices', 'lag_17_elec_prices', 'lag_18_elec_prices',
       'lag_19_elec_prices', 'lag_20_elec_prices', 'lag_21_elec_prices',
       'lag_22_elec_prices', 'lag_23_elec_prices', 'lag_24_elec_prices',
       'lag_1_wdsp_avg_log', 'lag_2_wdsp_avg_log', 'lag_3_wdsp_avg_log',
       'lag_4_wdsp_avg_log', 'lag_5_

,applicable_date,elec_prices,month,dayofyear_num,weekofyear_num,Hour,wdsp_avg_log,t+1,t+2,t+3,...,lag_14_wdsp_avg_log,lag_15_wdsp_avg_log,lag_16_wdsp_avg_log,lag_17_wdsp_avg_log,lag_18_wdsp_avg_log,lag_19_wdsp_avg_log,lag_20_wdsp_avg_log,lag_21_wdsp_avg_log,lag_22_wdsp_avg_log,lag_23_wdsp_avg_log
0,2019-01-01 00:00:00,70.28,1,1,1,0,1.069069,65.18,60.91,65.160,...,1.069069,1.069069,1.069069,1.069069,1.069069,1.069069,1.069069,1.069069,1.069069,1.069069
1,2019-01-01 01:00:00,65.18,1,1,1,1,1.008444,60.91,65.16,65.160,...,1.069069,1.069069,1.069069,1.069069,1.069069,1.069069,1.069069,1.069069,1.069069,1.069069
2,2019-01-01 02:00:00,60.91,1,1,1,2,1.008444,65.16,65.16,68.290,...,1.069069,1.069069,1.069069,1.069069,1.069069,1.069069,1.069069,1.069069,1.069069,1.069069
3,2019-01-01 03:00:00,65.16,1,1,1,3,1.126227,65.16,68.29,63.030,...,1.069069,1.069069,1.069069,1.069069,1.069069,1.069069,1.069069,1.069069,1.069069,1.069069
4,2019-01-01 04:00:00,65.16,1,1,1,4,1.180295,68.29,63.03,51.621,...,1.069069,1.069069,1.069069,1.069069,1.069069,1.069069,1.069069,1.069069,1.069069,1.069069


In [100]:
data_1.isnull().values.any()

True

In [101]:
#Dropping rows with Nan's
data_1 = data_1.dropna(axis=0)
print(data_1.shape)
data_1.isnull().values.any()

(8281, 77)


False

In [102]:
data_1 = remove_23rows_hour_col(data_1)
train,test = train_test_split_tbs(data_1)
train.columns

(345, 76)
(276, 76)
(69, 76)


Index(['applicable_date', 'elec_prices', 'month', 'dayofyear_num',
       'weekofyear_num', 'wdsp_avg_log', 't+1', 't+2', 't+3', 't+4', 't+5',
       't+6', 't+7', 't+8', 't+9', 't+10', 't+11', 't+12', 't+13', 't+14',
       't+15', 't+16', 't+17', 't+18', 't+19', 't+20', 't+21', 't+22', 't+23',
       'lag_1_elec_prices', 'lag_2_elec_prices', 'lag_3_elec_prices',
       'lag_4_elec_prices', 'lag_5_elec_prices', 'lag_6_elec_prices',
       'lag_7_elec_prices', 'lag_8_elec_prices', 'lag_9_elec_prices',
       'lag_10_elec_prices', 'lag_11_elec_prices', 'lag_12_elec_prices',
       'lag_13_elec_prices', 'lag_14_elec_prices', 'lag_15_elec_prices',
       'lag_16_elec_prices', 'lag_17_elec_prices', 'lag_18_elec_prices',
       'lag_19_elec_prices', 'lag_20_elec_prices', 'lag_21_elec_prices',
       'lag_22_elec_prices', 'lag_23_elec_prices', 'lag_24_elec_prices',
       'lag_1_wdsp_avg_log', 'lag_2_wdsp_avg_log', 'lag_3_wdsp_avg_log',
       'lag_4_wdsp_avg_log', 'lag_5_wdsp_avg_log', 'lag

In [103]:
#selecting columns for X & y
cols = train.columns.to_list()
x_names = cols[2:6]
lags = cols[-47:]
x_names.extend(lags)
y_names = [cols[1]]
time_steps = cols[6:29]
y_names.extend(time_steps)
print(x_names)
print(y_names)

['month', 'dayofyear_num', 'weekofyear_num', 'wdsp_avg_log', 'lag_1_elec_prices', 'lag_2_elec_prices', 'lag_3_elec_prices', 'lag_4_elec_prices', 'lag_5_elec_prices', 'lag_6_elec_prices', 'lag_7_elec_prices', 'lag_8_elec_prices', 'lag_9_elec_prices', 'lag_10_elec_prices', 'lag_11_elec_prices', 'lag_12_elec_prices', 'lag_13_elec_prices', 'lag_14_elec_prices', 'lag_15_elec_prices', 'lag_16_elec_prices', 'lag_17_elec_prices', 'lag_18_elec_prices', 'lag_19_elec_prices', 'lag_20_elec_prices', 'lag_21_elec_prices', 'lag_22_elec_prices', 'lag_23_elec_prices', 'lag_24_elec_prices', 'lag_1_wdsp_avg_log', 'lag_2_wdsp_avg_log', 'lag_3_wdsp_avg_log', 'lag_4_wdsp_avg_log', 'lag_5_wdsp_avg_log', 'lag_6_wdsp_avg_log', 'lag_7_wdsp_avg_log', 'lag_8_wdsp_avg_log', 'lag_9_wdsp_avg_log', 'lag_10_wdsp_avg_log', 'lag_11_wdsp_avg_log', 'lag_12_wdsp_avg_log', 'lag_13_wdsp_avg_log', 'lag_14_wdsp_avg_log', 'lag_15_wdsp_avg_log', 'lag_16_wdsp_avg_log', 'lag_17_wdsp_avg_log', 'lag_18_wdsp_avg_log', 'lag_19_wdsp_av

In [104]:
x_train, y_train = train[x_names], train[y_names]
x_test, y_test = test[x_names], test[y_names]
print(x_train.shape,y_train.shape)
print(x_test.shape,y_test.shape)

(276, 51) (276, 24)
(69, 51) (69, 24)


In [105]:
x_train_norm,x_test_norm = normalize(x_train,x_test)

In [106]:
GBM_training(x_train_norm,y_train)

Duration: 0:00:11.152630



{'mean_fit_time': array([0.60476024, 0.87341268, 0.99016154, 0.58993344, 0.59130366,
       1.51980748, 0.25246727, 0.8633637 , 0.50701351, 1.61598761]), 'std_fit_time': array([0.03874662, 0.05664378, 0.0710207 , 0.0491993 , 0.05464844,
       0.08713728, 0.02683321, 0.09178738, 0.05689981, 0.08961913]), 'mean_score_time': array([0.01090314, 0.01220841, 0.01360245, 0.01370363, 0.0106029 ,
       0.01420305, 0.00995481, 0.01310811, 0.01085482, 0.00830135]), 'std_score_time': array([0.00130078, 0.00124804, 0.00210708, 0.00536841, 0.00079978,
       0.00172093, 0.00078782, 0.00305391, 0.0019739 , 0.00223852]), 'param_estimator__subsample': masked_array(data=[0.75, 0.9, 0.85, 0.8, 0.75, 0.75, 0.7, 0.7, 0.6, 0.8],
             mask=[False, False, False, False, False, False, False, False,
                   False, False],
       fill_value='?',
            dtype=object), 'param_estimator__n_estimators': masked_array(data=[50, 80, 90, 50, 50, 100, 20, 70, 40, 100],

In [107]:
#learning_rate,n_estimators,max_depth,min_samples_split,max_features,min_samples_leaf,subsample
gbm7 = GBM_model(x_train_norm,y_train,0.0125,100,5,100,6,20,0.8)

Duration: 0:00:01.007852


In [108]:
y_pred= gbm7.predict(x_test_norm)
eval_metrics(y_test,y_pred)

MAE:  10.3964
MSE:  223.8562
RMSE:  14.9618
MAPE:  22.1486
Coefficient of determination (R-squared):  0.1224




(10.396432563424177,
 223.85620688051063,
 14.961824984957905,
 22.14859849543382,
 0.12243998936230004)

In [109]:
XGBM_training(x_train_norm,y_train)

Duration: 0:01:37.465233



{'mean_fit_time': array([ 9.49345071, 18.97604203,  5.07564266,  3.09415035, 27.58806062,
        4.12073832,  8.21083953, 17.42505805, 11.71148062,  3.31917408]), 'std_fit_time': array([0.66511245, 0.65334238, 0.21441308, 0.22661295, 0.93677443,
       0.23060117, 0.32539544, 0.68808693, 0.66402804, 0.29184882]), 'mean_score_time': array([0.02840641, 0.05098789, 0.05130422, 0.03003206, 0.060513  ,
       0.03384576, 0.03430965, 0.06018419, 0.04051723, 0.02950301]), 'std_score_time': array([0.00142895, 0.00671883, 0.02948163, 0.00471334, 0.02282515,
       0.00299022, 0.00191633, 0.0060867 , 0.00749085, 0.00631942]), 'param_estimator__subsample': masked_array(data=[0.9, 0.8, 0.4, 0.1, 0.6, 0.8, 0.9, 0.8,
                   0.7000000000000001, 0.5],
             mask=[False, False, False, False, False, False, False, False,
                   False, False],
       fill_value='?',
            dtype=object), 'param_estimator__n_estimators': masked_array(data=[30

In [110]:
#learning_rate,n_estimators,max_depth,subsample,colsample_bytree
xgb7 = XGBM_model(x_train_norm,y_train,0.0125,400,5,0.4,0.2)

Duration: 0:00:02.361008


In [111]:
y_pred= xgb7.predict(x_test_norm)
eval_metrics(y_test,y_pred)

MAE:  10.0453
MSE:  212.0578
RMSE:  14.5622
MAPE:  21.4006
Coefficient of determination (R-squared):  0.2083




(10.0453471711062,
 212.05781015068854,
 14.56220485196828,
 21.400644873398143,
 0.20834733504988656)

In [112]:
LGBM_training(x_train_norm,y_train)

[LightGBM] [Warning] min_child_samples is set=20, min_child_samples= will be ignored. Current value: min_child_samples=20
[LightGBM] [Warning] Unknown parameter: 60
Duration: 0:00:29.441424



{'mean_fit_time': array([3.41503475, 1.50185614, 5.4380743 , 5.95853066, 4.36434412,
       2.46607664, 1.89814892, 2.08817492, 1.5977602 , 2.52875566]), 'std_fit_time': array([0.19335829, 0.08388482, 0.25397216, 0.54717434, 0.45043789,
       0.10927884, 0.09248763, 0.09026012, 0.06850198, 0.06994777]), 'mean_score_time': array([0.03192384, 0.02607749, 0.03172162, 0.07938826, 0.02486138,
       0.05182543, 0.03036053, 0.0407248 , 0.02255623, 0.03947351]), 'std_score_time': array([0.00391074, 0.00569742, 0.01081069, 0.01744706, 0.02476718,
       0.04930841, 0.00734278, 0.01319207, 0.00330445, 0.01677739]), 'param_estimator__subsample': masked_array(data=[0.1, 0.8, 0.8, 0.1, 0.7000000000000001,
                   0.30000000000000004, 0.30000000000000004, 0.8, 0.6,
                   0.6],
       

In [113]:
#num_leaves,min_child_samples,learning_rate,n_estimators,max_depth,subsample,colsample_bytree
lgbm7 = LGBM_model(x_train_norm,y_train,5,60,0.0125,250,13,0.8,0.4)

Duration: 0:00:00.803220


In [114]:
y_pred= lgbm7.predict(x_test_norm)
eval_metrics(y_test,y_pred)

MAE:  10.721
MSE:  226.995
RMSE:  15.0664
MAPE:  22.8401
Coefficient of determination (R-squared):  0.1108




(10.721040957021605,
 226.99504057025283,
 15.066354587963634,
 22.84014542118709,
 0.11082000116383406)

### Testing on 2018-2019 data

In [115]:
data_test = data_1819.copy()
data_test = feature_pipeline_1(data_test)
wdsp_avg= (data_1819['wdsp_d']+data_1819['wdsp_c']+data_1819['wdsp_g'])/3
wdsp_avg_log = np.log(wdsp_avg)
data_test['wdsp_avg_log'] = wdsp_avg_log

data_test = multi_step(data_test,'elec_prices')
data_test = calc_24lags(data_test,'elec_prices')
data_test = calc_24lags(data_test,'wdsp_avg_log')
print(data_test.shape)
print(data_test.columns)
data_test.head()


(10514, 77)
Index(['applicable_date', 'elec_prices', 'month', 'dayofyear_num',
       'weekofyear_num', 'Hour', 'wdsp_avg_log', 't+1', 't+2', 't+3', 't+4',
       't+5', 't+6', 't+7', 't+8', 't+9', 't+10', 't+11', 't+12', 't+13',
       't+14', 't+15', 't+16', 't+17', 't+18', 't+19', 't+20', 't+21', 't+22',
       't+23', 'lag_1_elec_prices', 'lag_2_elec_prices', 'lag_3_elec_prices',
       'lag_4_elec_prices', 'lag_5_elec_prices', 'lag_6_elec_prices',
       'lag_7_elec_prices', 'lag_8_elec_prices', 'lag_9_elec_prices',
       'lag_10_elec_prices', 'lag_11_elec_prices', 'lag_12_elec_prices',
       'lag_13_elec_prices', 'lag_14_elec_prices', 'lag_15_elec_prices',
       'lag_16_elec_prices', 'lag_17_elec_prices', 'lag_18_elec_prices',
       'lag_19_elec_prices', 'lag_20_elec_prices', 'lag_21_elec_prices',
       'lag_22_elec_prices', 'lag_23_elec_prices', 'lag_24_elec_prices',
       'lag_1_wdsp_avg_log', 'lag_2_wdsp_avg_log', 'lag_3_wdsp_avg_log',
       'lag_4_wdsp_avg_log', 'lag_5

,applicable_date,elec_prices,month,dayofyear_num,weekofyear_num,Hour,wdsp_avg_log,t+1,t+2,t+3,...,lag_14_wdsp_avg_log,lag_15_wdsp_avg_log,lag_16_wdsp_avg_log,lag_17_wdsp_avg_log,lag_18_wdsp_avg_log,lag_19_wdsp_avg_log,lag_20_wdsp_avg_log,lag_21_wdsp_avg_log,lag_22_wdsp_avg_log,lag_23_wdsp_avg_log
0,2018-09-30 22:00:00,71.267,9,273,39,22,1.126227,67.212,60.500,63.682,...,1.126227,1.126227,1.126227,1.126227,1.126227,1.126227,1.126227,1.126227,1.126227,1.126227
1,2018-09-30 23:00:00,67.212,9,273,39,23,1.231588,60.500,63.682,71.617,...,1.126227,1.126227,1.126227,1.126227,1.126227,1.126227,1.126227,1.126227,1.126227,1.126227
2,2018-10-01 00:00:00,60.500,10,274,40,0,1.280378,63.682,71.617,72.855,...,1.126227,1.126227,1.126227,1.126227,1.126227,1.126227,1.126227,1.126227,1.126227,1.126227
3,2018-10-01 01:00:00,63.682,10,274,40,1,1.280378,71.617,72.855,76.937,...,1.126227,1.126227,1.126227,1.126227,1.126227,1.126227,1.126227,1.126227,1.126227,1.126227
4,2018-10-01 02:00:00,71.617,10,274,40,2,1.008444,72.855,76.937,80.480,...,1.126227,1.126227,1.126227,1.126227,1.126227,1.126227,1.126227,1.126227,1.126227,1.126227


In [116]:
#Dropping rows with Nan's
data_test = data_test.dropna(axis=0)
print(data_test.shape)
data_test.isnull().values.any()

(10491, 77)


False

In [117]:
data_test = remove_23rows_hour_col(data_test)
data_test.columns

(438, 76)


Index(['applicable_date', 'elec_prices', 'month', 'dayofyear_num',
       'weekofyear_num', 'wdsp_avg_log', 't+1', 't+2', 't+3', 't+4', 't+5',
       't+6', 't+7', 't+8', 't+9', 't+10', 't+11', 't+12', 't+13', 't+14',
       't+15', 't+16', 't+17', 't+18', 't+19', 't+20', 't+21', 't+22', 't+23',
       'lag_1_elec_prices', 'lag_2_elec_prices', 'lag_3_elec_prices',
       'lag_4_elec_prices', 'lag_5_elec_prices', 'lag_6_elec_prices',
       'lag_7_elec_prices', 'lag_8_elec_prices', 'lag_9_elec_prices',
       'lag_10_elec_prices', 'lag_11_elec_prices', 'lag_12_elec_prices',
       'lag_13_elec_prices', 'lag_14_elec_prices', 'lag_15_elec_prices',
       'lag_16_elec_prices', 'lag_17_elec_prices', 'lag_18_elec_prices',
       'lag_19_elec_prices', 'lag_20_elec_prices', 'lag_21_elec_prices',
       'lag_22_elec_prices', 'lag_23_elec_prices', 'lag_24_elec_prices',
       'lag_1_wdsp_avg_log', 'lag_2_wdsp_avg_log', 'lag_3_wdsp_avg_log',
       'lag_4_wdsp_avg_log', 'lag_5_wdsp_avg_log', 'lag

In [118]:
#selecting columns for X & y
cols = train.columns.to_list()
x_names = cols[2:6]
lags = cols[-47:]
x_names.extend(lags)
y_names = [cols[1]]
time_steps = cols[6:29]
y_names.extend(time_steps)
print(x_names)
print(y_names)

['month', 'dayofyear_num', 'weekofyear_num', 'wdsp_avg_log', 'lag_1_elec_prices', 'lag_2_elec_prices', 'lag_3_elec_prices', 'lag_4_elec_prices', 'lag_5_elec_prices', 'lag_6_elec_prices', 'lag_7_elec_prices', 'lag_8_elec_prices', 'lag_9_elec_prices', 'lag_10_elec_prices', 'lag_11_elec_prices', 'lag_12_elec_prices', 'lag_13_elec_prices', 'lag_14_elec_prices', 'lag_15_elec_prices', 'lag_16_elec_prices', 'lag_17_elec_prices', 'lag_18_elec_prices', 'lag_19_elec_prices', 'lag_20_elec_prices', 'lag_21_elec_prices', 'lag_22_elec_prices', 'lag_23_elec_prices', 'lag_24_elec_prices', 'lag_1_wdsp_avg_log', 'lag_2_wdsp_avg_log', 'lag_3_wdsp_avg_log', 'lag_4_wdsp_avg_log', 'lag_5_wdsp_avg_log', 'lag_6_wdsp_avg_log', 'lag_7_wdsp_avg_log', 'lag_8_wdsp_avg_log', 'lag_9_wdsp_avg_log', 'lag_10_wdsp_avg_log', 'lag_11_wdsp_avg_log', 'lag_12_wdsp_avg_log', 'lag_13_wdsp_avg_log', 'lag_14_wdsp_avg_log', 'lag_15_wdsp_avg_log', 'lag_16_wdsp_avg_log', 'lag_17_wdsp_avg_log', 'lag_18_wdsp_avg_log', 'lag_19_wdsp_av

In [119]:
#GBM
mae,mse,rmse,mape,r2 = testing_runs(10,data_test,gbm7)

Round 1: 
(350, 51) (44, 51) (44, 51)
(350, 24) (44, 24) (44, 24)
MAE:  10.5722
MSE:  258.8523
RMSE:  16.0889
MAPE:  19.4595
Coefficient of determination (R-squared):  0.3212


Round 2: 
(350, 51) (44, 51) (44, 51)
(350, 24) (44, 24) (44, 24)
MAE:  10.1208
MSE:  182.9553
RMSE:  13.5261
MAPE:  19.3479
Coefficient of determination (R-squared):  0.3172


Round 3: 
(350, 51) (44, 51) (44, 51)
(350, 24) (44, 24) (44, 24)
MAE:  10.6586
MSE:  233.8682
RMSE:  15.2928
MAPE:  19.7755
Coefficient of determination (R-squared):  0.3499


Round 4: 
(350, 51) (44, 51) (44, 51)
(350, 24) (44, 24) (44, 24)
MAE:  11.6888
MSE:  373.5392
RMSE:  19.3272
MAPE:  22.3045
Coefficient of determination (R-squared):  0.4338


Round 5: 
(350, 51) (44, 51) (44, 51)
(350, 24) (44, 24) (44, 24)
MAE:  12.0319
MSE:  325.3706
RMSE:  18.038
MAPE:  22.9725
Coefficient of determination (R-squared):  0.3346


Round 6: 
(350, 51) (44, 51) (44, 51)
(350, 24) (44, 24) (44, 24)
MAE:  11.8593
MSE:  306.753
RMSE:  17.5144
MAPE:  

In [120]:
#XGBM
mae,mse,rmse,mape,r2 = testing_runs(10,data_test,xgb7)

Round 1: 
(350, 51) (44, 51) (44, 51)
(350, 24) (44, 24) (44, 24)
MAE:  10.4416
MSE:  220.4334
RMSE:  14.847
MAPE:  18.6824
Coefficient of determination (R-squared):  0.4119


Round 2: 
(350, 51) (44, 51) (44, 51)
(350, 24) (44, 24) (44, 24)
MAE:  9.5714
MSE:  208.9955
RMSE:  14.4567
MAPE:  17.9312
Coefficient of determination (R-squared):  0.6145


Round 3: 
(350, 51) (44, 51) (44, 51)
(350, 24) (44, 24) (44, 24)
MAE:  8.7822
MSE:  198.2927
RMSE:  14.0816
MAPE:  17.3203
Coefficient of determination (R-squared):  0.5701


Round 4: 
(350, 51) (44, 51) (44, 51)
(350, 24) (44, 24) (44, 24)
MAE:  9.4898
MSE:  190.2418
RMSE:  13.7928
MAPE:  17.2697
Coefficient of determination (R-squared):  0.4846


Round 5: 
(350, 51) (44, 51) (44, 51)
(350, 24) (44, 24) (44, 24)
MAE:  8.2991
MSE:  201.4682
RMSE:  14.194
MAPE:  16.8147
Coefficient of determination (R-squared):  0.4171


Round 6: 
(350, 51) (44, 51) (44, 51)
(350, 24) (44, 24) (44, 24)
MAE:  8.1308
MSE:  151.3131
RMSE:  12.3009
MAPE:  14.44

In [121]:
#light GBM
mae,mse,rmse,mape,r2 = testing_runs(10,data_test,lgbm7)

Round 1: 
(350, 51) (44, 51) (44, 51)
(350, 24) (44, 24) (44, 24)
MAE:  12.3618
MSE:  425.9258
RMSE:  20.638
MAPE:  20.5951
Coefficient of determination (R-squared):  0.416


Round 2: 
(350, 51) (44, 51) (44, 51)
(350, 24) (44, 24) (44, 24)
MAE:  13.2546
MSE:  517.5523
RMSE:  22.7498
MAPE:  23.9662
Coefficient of determination (R-squared):  0.4472


Round 3: 
(350, 51) (44, 51) (44, 51)
(350, 24) (44, 24) (44, 24)
MAE:  10.2544
MSE:  260.8176
RMSE:  16.1498
MAPE:  18.797
Coefficient of determination (R-squared):  0.4321


Round 4: 
(350, 51) (44, 51) (44, 51)
(350, 24) (44, 24) (44, 24)
MAE:  12.3896
MSE:  503.6676
RMSE:  22.4425
MAPE:  20.2058
Coefficient of determination (R-squared):  0.4631


Round 5: 
(350, 51) (44, 51) (44, 51)
(350, 24) (44, 24) (44, 24)
MAE:  10.382
MSE:  235.4944
RMSE:  15.3458
MAPE:  19.1095
Coefficient of determination (R-squared):  0.5092


Round 6: 
(350, 51) (44, 51) (44, 51)
(350, 24) (44, 24) (44, 24)
MAE:  9.6177
MSE:  243.7273
RMSE:  15.6118
MAPE:  17.

<h1 style="text-align:center;font-size:40px;" > Basic time based features + wdsp_c (windspeeds in Cork county) </h1>

In [122]:
data_1= data.copy()
data_1 = feature_pipeline_1(data_1)
data_1['wdsp_c'] = data['wdsp_c']
data_1.head()

,applicable_date,elec_prices,month,dayofyear_num,weekofyear_num,Hour,wdsp_c
0,2019-01-01 00:00:00,70.28,1,1,1,0,2.056
1,2019-01-01 01:00:00,65.18,1,1,1,1,1.542
2,2019-01-01 02:00:00,60.91,1,1,1,2,1.542
3,2019-01-01 03:00:00,65.16,1,1,1,3,2.570
4,2019-01-01 04:00:00,65.16,1,1,1,4,2.570


In [123]:
data_1 = multi_step(data_1,'elec_prices')
data_1 = calc_24lags(data_1,'elec_prices')
data_1 = calc_24lags(data_1,'wdsp_c')
print(data_1.shape)
print(data_1.columns)
data_1.head()

(8304, 77)
Index(['applicable_date', 'elec_prices', 'month', 'dayofyear_num',
       'weekofyear_num', 'Hour', 'wdsp_c', 't+1', 't+2', 't+3', 't+4', 't+5',
       't+6', 't+7', 't+8', 't+9', 't+10', 't+11', 't+12', 't+13', 't+14',
       't+15', 't+16', 't+17', 't+18', 't+19', 't+20', 't+21', 't+22', 't+23',
       'lag_1_elec_prices', 'lag_2_elec_prices', 'lag_3_elec_prices',
       'lag_4_elec_prices', 'lag_5_elec_prices', 'lag_6_elec_prices',
       'lag_7_elec_prices', 'lag_8_elec_prices', 'lag_9_elec_prices',
       'lag_10_elec_prices', 'lag_11_elec_prices', 'lag_12_elec_prices',
       'lag_13_elec_prices', 'lag_14_elec_prices', 'lag_15_elec_prices',
       'lag_16_elec_prices', 'lag_17_elec_prices', 'lag_18_elec_prices',
       'lag_19_elec_prices', 'lag_20_elec_prices', 'lag_21_elec_prices',
       'lag_22_elec_prices', 'lag_23_elec_prices', 'lag_24_elec_prices',
       'lag_1_wdsp_c', 'lag_2_wdsp_c', 'lag_3_wdsp_c', 'lag_4_wdsp_c',
       'lag_5_wdsp_c', 'lag_6_wdsp_c', 'lag_

,applicable_date,elec_prices,month,dayofyear_num,weekofyear_num,Hour,wdsp_c,t+1,t+2,t+3,...,lag_14_wdsp_c,lag_15_wdsp_c,lag_16_wdsp_c,lag_17_wdsp_c,lag_18_wdsp_c,lag_19_wdsp_c,lag_20_wdsp_c,lag_21_wdsp_c,lag_22_wdsp_c,lag_23_wdsp_c
0,2019-01-01 00:00:00,70.28,1,1,1,0,2.056,65.18,60.91,65.160,...,2.056,2.056,2.056,2.056,2.056,2.056,2.056,2.056,2.056,2.056
1,2019-01-01 01:00:00,65.18,1,1,1,1,1.542,60.91,65.16,65.160,...,2.056,2.056,2.056,2.056,2.056,2.056,2.056,2.056,2.056,2.056
2,2019-01-01 02:00:00,60.91,1,1,1,2,1.542,65.16,65.16,68.290,...,2.056,2.056,2.056,2.056,2.056,2.056,2.056,2.056,2.056,2.056
3,2019-01-01 03:00:00,65.16,1,1,1,3,2.570,65.16,68.29,63.030,...,2.056,2.056,2.056,2.056,2.056,2.056,2.056,2.056,2.056,2.056
4,2019-01-01 04:00:00,65.16,1,1,1,4,2.570,68.29,63.03,51.621,...,2.056,2.056,2.056,2.056,2.056,2.056,2.056,2.056,2.056,2.056


In [124]:
data_1.isnull().values.any()

True

In [125]:
#Dropping rows with Nan's
data_1 = data_1.dropna(axis=0)
print(data_1.shape)
data_1.isnull().values.any()

(8281, 77)


False

In [126]:
data_1 = remove_23rows_hour_col(data_1)
train,test = train_test_split_tbs(data_1)
train.columns

(345, 76)
(276, 76)
(69, 76)


Index(['applicable_date', 'elec_prices', 'month', 'dayofyear_num',
       'weekofyear_num', 'wdsp_c', 't+1', 't+2', 't+3', 't+4', 't+5', 't+6',
       't+7', 't+8', 't+9', 't+10', 't+11', 't+12', 't+13', 't+14', 't+15',
       't+16', 't+17', 't+18', 't+19', 't+20', 't+21', 't+22', 't+23',
       'lag_1_elec_prices', 'lag_2_elec_prices', 'lag_3_elec_prices',
       'lag_4_elec_prices', 'lag_5_elec_prices', 'lag_6_elec_prices',
       'lag_7_elec_prices', 'lag_8_elec_prices', 'lag_9_elec_prices',
       'lag_10_elec_prices', 'lag_11_elec_prices', 'lag_12_elec_prices',
       'lag_13_elec_prices', 'lag_14_elec_prices', 'lag_15_elec_prices',
       'lag_16_elec_prices', 'lag_17_elec_prices', 'lag_18_elec_prices',
       'lag_19_elec_prices', 'lag_20_elec_prices', 'lag_21_elec_prices',
       'lag_22_elec_prices', 'lag_23_elec_prices', 'lag_24_elec_prices',
       'lag_1_wdsp_c', 'lag_2_wdsp_c', 'lag_3_wdsp_c', 'lag_4_wdsp_c',
       'lag_5_wdsp_c', 'lag_6_wdsp_c', 'lag_7_wdsp_c', 'lag_8_w

In [127]:
#selecting columns for X & y
cols = train.columns.to_list()
x_names = cols[2:6]
lags = cols[-47:]
x_names.extend(lags)
y_names = [cols[1]]
time_steps = cols[6:29]
y_names.extend(time_steps)
print(x_names)
print(y_names)

['month', 'dayofyear_num', 'weekofyear_num', 'wdsp_c', 'lag_1_elec_prices', 'lag_2_elec_prices', 'lag_3_elec_prices', 'lag_4_elec_prices', 'lag_5_elec_prices', 'lag_6_elec_prices', 'lag_7_elec_prices', 'lag_8_elec_prices', 'lag_9_elec_prices', 'lag_10_elec_prices', 'lag_11_elec_prices', 'lag_12_elec_prices', 'lag_13_elec_prices', 'lag_14_elec_prices', 'lag_15_elec_prices', 'lag_16_elec_prices', 'lag_17_elec_prices', 'lag_18_elec_prices', 'lag_19_elec_prices', 'lag_20_elec_prices', 'lag_21_elec_prices', 'lag_22_elec_prices', 'lag_23_elec_prices', 'lag_24_elec_prices', 'lag_1_wdsp_c', 'lag_2_wdsp_c', 'lag_3_wdsp_c', 'lag_4_wdsp_c', 'lag_5_wdsp_c', 'lag_6_wdsp_c', 'lag_7_wdsp_c', 'lag_8_wdsp_c', 'lag_9_wdsp_c', 'lag_10_wdsp_c', 'lag_11_wdsp_c', 'lag_12_wdsp_c', 'lag_13_wdsp_c', 'lag_14_wdsp_c', 'lag_15_wdsp_c', 'lag_16_wdsp_c', 'lag_17_wdsp_c', 'lag_18_wdsp_c', 'lag_19_wdsp_c', 'lag_20_wdsp_c', 'lag_21_wdsp_c', 'lag_22_wdsp_c', 'lag_23_wdsp_c']
['elec_prices', 't+1', 't+2', 't+3', 't+4', 

In [128]:
x_train, y_train = train[x_names], train[y_names]
x_test, y_test = test[x_names], test[y_names]
print(x_train.shape,y_train.shape)
print(x_test.shape,y_test.shape)

(276, 51) (276, 24)
(69, 51) (69, 24)


In [129]:
x_train_norm,x_test_norm = normalize(x_train,x_test)

In [130]:
GBM_training(x_train_norm,y_train)

Duration: 0:00:09.327124



{'mean_fit_time': array([0.87872462, 0.56528425, 0.79023778, 0.57661672, 0.8371594 ,
       0.97701159, 0.8361836 , 0.34948833, 0.98624327, 0.41980064]), 'std_fit_time': array([0.04663371, 0.03373862, 0.07365707, 0.04159278, 0.02310864,
       0.03330272, 0.0400019 , 0.03244526, 0.07540865, 0.07497144]), 'mean_score_time': array([0.01190848, 0.0115068 , 0.0109993 , 0.01050258, 0.01310308,
       0.01130261, 0.01270297, 0.0104023 , 0.01180217, 0.00750413]), 'std_score_time': array([0.00394944, 0.00161847, 0.00077253, 0.00067056, 0.00238626,
       0.00064041, 0.00100496, 0.00135702, 0.00124884, 0.00237805]), 'param_estimator__subsample': masked_array(data=[0.75, 0.5, 0.9, 0.5, 0.6, 0.9, 0.8, 0.9, 0.8, 0.85],
             mask=[False, False, False, False, False, False, False, False,
                   False, False],
       fill_value='?',
            dtype=object), 'param_estimator__n_estimators': masked_array(data=[80, 50, 70, 50, 70, 80, 50, 30, 90, 40],
   

In [131]:
#def GBM_model(X,y,learning_rate,n_estimators,max_depth,min_samples_split,max_features,min_samples_leaf,subsample):
gbm8 = GBM_model(x_train_norm,y_train,0.1,50,9,100,5,20,0.8)

Duration: 0:00:00.470714


In [132]:
y_pred= gbm8.predict(x_test_norm)
eval_metrics(y_test,y_pred)

MAE:  10.6552
MSE:  226.3701
RMSE:  15.0456
MAPE:  22.6999
Coefficient of determination (R-squared):  0.115




(10.65520401233672,
 226.37005625294606,
 15.045599232099267,
 22.699886150028853,
 0.11499889740801443)

In [133]:
XGBM_training(x_train_norm,y_train)

Duration: 0:01:08.997303



{'mean_fit_time': array([22.51955912,  3.47498205,  1.6335078 ,  1.65720346,  2.67100604,
        3.20998151,  7.11023226,  2.58625536, 20.84771988,  5.46077235]), 'std_fit_time': array([1.05544432, 0.17215612, 0.04423465, 0.10314715, 0.09722802,
       0.21139507, 0.25806063, 0.15887205, 0.61946528, 0.86165047]), 'mean_score_time': array([0.04666612, 0.03720832, 0.02705908, 0.02787127, 0.02678125,
       0.04419765, 0.0313448 , 0.03110318, 0.05183144, 0.02220366]), 'std_score_time': array([0.00703822, 0.00223765, 0.00269119, 0.00301313, 0.00336414,
       0.01059828, 0.00502498, 0.00572945, 0.00981044, 0.00407812]), 'param_estimator__subsample': masked_array(data=[1.0, 1.0, 0.1, 0.8, 0.9, 0.1, 0.30000000000000004, 0.1,
                   0.8, 0.4],
             mask=[False, False, False, False, False, False, False, False,
                   False, False],
       fill_value='?',
            dtype=object), 'param_estimator__n_estimators': masked_array(data=[3

In [134]:
# def XGBM_model(X,y,learning_rate,n_estimators,max_depth,subsample,colsample_bytree):
xgb8 = XGBM_model(x_train_norm,y_train,0.025,400,9,0.8,0.7)

Duration: 0:00:05.001748


In [135]:
y_pred= xgb8.predict(x_test_norm)
eval_metrics(y_test,y_pred)

MAE:  10.9503
MSE:  246.8154
RMSE:  15.7104
MAPE:  23.3286
Coefficient of determination (R-squared):  0.0944




(10.950318608631834,
 246.8154323938861,
 15.710360670394747,
 23.32859938061126,
 0.09443789475240338)

In [136]:
LGBM_training(x_train_norm,y_train)

[LightGBM] [Warning] min_child_samples is set=20, min_child_samples= will be ignored. Current value: min_child_samples=20
[LightGBM] [Warning] Unknown parameter: 70
Duration: 0:00:29.280401



{'mean_fit_time': array([4.27121105, 1.37691891, 3.36724367, 1.01977837, 0.76909816,
       1.12355661, 2.74151366, 5.33750019, 6.98144839, 2.89204078]), 'std_fit_time': array([0.34562371, 0.08761532, 0.10505302, 0.07615312, 0.05592277,
       0.06410832, 0.15239922, 0.32596441, 0.44780453, 0.7089851 ]), 'mean_score_time': array([0.04495311, 0.02160127, 0.04114542, 0.02040515, 0.02161417,
       0.03481121, 0.03116007, 0.06859863, 0.03313699, 0.03803895]), 'std_score_time': array([0.00730777, 0.01255518, 0.00610589, 0.01407054, 0.0152906 ,
       0.02207793, 0.00317981, 0.05461833, 0.01123903, 0.01343834]), 'param_estimator__subsample': masked_array(data=[0.30000000000000004, 0.30000000000000004, 0.4, 0.8,
                   0.2, 0.6, 0.9, 1.0, 0.5, 0.9],
             mask=[False, False, False, F

In [137]:
#def LGBM_model(X,y,num_leaves,min_child_samples,learning_rate,n_estimators,max_depth,subsample,colsample_bytree)
lgbm8 = LGBM_model(x_train_norm,y_train,19,70,0.0125,300,11,0.9,0.4)

Duration: 0:00:00.792210


In [138]:
y_pred= lgbm8.predict(x_test_norm)
eval_metrics(y_test,y_pred)

MAE:  10.7502
MSE:  224.6783
RMSE:  14.9893
MAPE:  22.9022
Coefficient of determination (R-squared):  0.1046




(10.750170995779243,
 224.67827100696908,
 14.989271863802093,
 22.90220416380513,
 0.1046418203122722)

### Testing on 2018-2019 data

In [139]:
data_test = data_1819.copy()
data_test = feature_pipeline_1(data_test)
data_test['wdsp_c']= data_1819['wdsp_c']
data_test = multi_step(data_test,'elec_prices')
data_test = calc_24lags(data_test,'elec_prices')
data_test = calc_24lags(data_test,'wdsp_c')
print(data_test.shape)
print(data_test.columns)
data_test.head()

(10514, 77)
Index(['applicable_date', 'elec_prices', 'month', 'dayofyear_num',
       'weekofyear_num', 'Hour', 'wdsp_c', 't+1', 't+2', 't+3', 't+4', 't+5',
       't+6', 't+7', 't+8', 't+9', 't+10', 't+11', 't+12', 't+13', 't+14',
       't+15', 't+16', 't+17', 't+18', 't+19', 't+20', 't+21', 't+22', 't+23',
       'lag_1_elec_prices', 'lag_2_elec_prices', 'lag_3_elec_prices',
       'lag_4_elec_prices', 'lag_5_elec_prices', 'lag_6_elec_prices',
       'lag_7_elec_prices', 'lag_8_elec_prices', 'lag_9_elec_prices',
       'lag_10_elec_prices', 'lag_11_elec_prices', 'lag_12_elec_prices',
       'lag_13_elec_prices', 'lag_14_elec_prices', 'lag_15_elec_prices',
       'lag_16_elec_prices', 'lag_17_elec_prices', 'lag_18_elec_prices',
       'lag_19_elec_prices', 'lag_20_elec_prices', 'lag_21_elec_prices',
       'lag_22_elec_prices', 'lag_23_elec_prices', 'lag_24_elec_prices',
       'lag_1_wdsp_c', 'lag_2_wdsp_c', 'lag_3_wdsp_c', 'lag_4_wdsp_c',
       'lag_5_wdsp_c', 'lag_6_wdsp_c', 'lag

,applicable_date,elec_prices,month,dayofyear_num,weekofyear_num,Hour,wdsp_c,t+1,t+2,t+3,...,lag_14_wdsp_c,lag_15_wdsp_c,lag_16_wdsp_c,lag_17_wdsp_c,lag_18_wdsp_c,lag_19_wdsp_c,lag_20_wdsp_c,lag_21_wdsp_c,lag_22_wdsp_c,lag_23_wdsp_c
0,2018-09-30 22:00:00,71.267,9,273,39,22,3.084,67.212,60.500,63.682,...,3.084,3.084,3.084,3.084,3.084,3.084,3.084,3.084,3.084,3.084
1,2018-09-30 23:00:00,67.212,9,273,39,23,3.598,60.500,63.682,71.617,...,3.084,3.084,3.084,3.084,3.084,3.084,3.084,3.084,3.084,3.084
2,2018-10-01 00:00:00,60.500,10,274,40,0,4.112,63.682,71.617,72.855,...,3.084,3.084,3.084,3.084,3.084,3.084,3.084,3.084,3.084,3.084
3,2018-10-01 01:00:00,63.682,10,274,40,1,3.084,71.617,72.855,76.937,...,3.084,3.084,3.084,3.084,3.084,3.084,3.084,3.084,3.084,3.084
4,2018-10-01 02:00:00,71.617,10,274,40,2,1.542,72.855,76.937,80.480,...,3.084,3.084,3.084,3.084,3.084,3.084,3.084,3.084,3.084,3.084


In [140]:
#Dropping rows with Nan's
data_test = data_test.dropna(axis=0)
print(data_test.shape)
data_test.isnull().values.any()

(10491, 77)


False

In [141]:
data_test = remove_23rows_hour_col(data_test)
data_test.columns

(438, 76)


Index(['applicable_date', 'elec_prices', 'month', 'dayofyear_num',
       'weekofyear_num', 'wdsp_c', 't+1', 't+2', 't+3', 't+4', 't+5', 't+6',
       't+7', 't+8', 't+9', 't+10', 't+11', 't+12', 't+13', 't+14', 't+15',
       't+16', 't+17', 't+18', 't+19', 't+20', 't+21', 't+22', 't+23',
       'lag_1_elec_prices', 'lag_2_elec_prices', 'lag_3_elec_prices',
       'lag_4_elec_prices', 'lag_5_elec_prices', 'lag_6_elec_prices',
       'lag_7_elec_prices', 'lag_8_elec_prices', 'lag_9_elec_prices',
       'lag_10_elec_prices', 'lag_11_elec_prices', 'lag_12_elec_prices',
       'lag_13_elec_prices', 'lag_14_elec_prices', 'lag_15_elec_prices',
       'lag_16_elec_prices', 'lag_17_elec_prices', 'lag_18_elec_prices',
       'lag_19_elec_prices', 'lag_20_elec_prices', 'lag_21_elec_prices',
       'lag_22_elec_prices', 'lag_23_elec_prices', 'lag_24_elec_prices',
       'lag_1_wdsp_c', 'lag_2_wdsp_c', 'lag_3_wdsp_c', 'lag_4_wdsp_c',
       'lag_5_wdsp_c', 'lag_6_wdsp_c', 'lag_7_wdsp_c', 'lag_8_w

In [142]:
#selecting columns for X & y
cols = train.columns.to_list()
x_names = cols[2:6]
lags = cols[-47:]
x_names.extend(lags)
y_names = [cols[1]]
time_steps = cols[6:29]
y_names.extend(time_steps)
print(x_names)
print(y_names)

['month', 'dayofyear_num', 'weekofyear_num', 'wdsp_c', 'lag_1_elec_prices', 'lag_2_elec_prices', 'lag_3_elec_prices', 'lag_4_elec_prices', 'lag_5_elec_prices', 'lag_6_elec_prices', 'lag_7_elec_prices', 'lag_8_elec_prices', 'lag_9_elec_prices', 'lag_10_elec_prices', 'lag_11_elec_prices', 'lag_12_elec_prices', 'lag_13_elec_prices', 'lag_14_elec_prices', 'lag_15_elec_prices', 'lag_16_elec_prices', 'lag_17_elec_prices', 'lag_18_elec_prices', 'lag_19_elec_prices', 'lag_20_elec_prices', 'lag_21_elec_prices', 'lag_22_elec_prices', 'lag_23_elec_prices', 'lag_24_elec_prices', 'lag_1_wdsp_c', 'lag_2_wdsp_c', 'lag_3_wdsp_c', 'lag_4_wdsp_c', 'lag_5_wdsp_c', 'lag_6_wdsp_c', 'lag_7_wdsp_c', 'lag_8_wdsp_c', 'lag_9_wdsp_c', 'lag_10_wdsp_c', 'lag_11_wdsp_c', 'lag_12_wdsp_c', 'lag_13_wdsp_c', 'lag_14_wdsp_c', 'lag_15_wdsp_c', 'lag_16_wdsp_c', 'lag_17_wdsp_c', 'lag_18_wdsp_c', 'lag_19_wdsp_c', 'lag_20_wdsp_c', 'lag_21_wdsp_c', 'lag_22_wdsp_c', 'lag_23_wdsp_c']
['elec_prices', 't+1', 't+2', 't+3', 't+4', 

In [143]:
#GBM
mae,mse,rmse,mape,r2 = testing_runs(10,data_test,gbm8)

Round 1: 
(350, 51) (44, 51) (44, 51)
(350, 24) (44, 24) (44, 24)
MAE:  9.5583
MSE:  194.6992
RMSE:  13.9535
MAPE:  16.7855
Coefficient of determination (R-squared):  0.4926


Round 2: 
(350, 51) (44, 51) (44, 51)
(350, 24) (44, 24) (44, 24)
MAE:  11.3225
MSE:  312.5754
RMSE:  17.6798
MAPE:  20.0058
Coefficient of determination (R-squared):  0.4459


Round 3: 
(350, 51) (44, 51) (44, 51)
(350, 24) (44, 24) (44, 24)
MAE:  9.4253
MSE:  240.1937
RMSE:  15.4982
MAPE:  17.3547
Coefficient of determination (R-squared):  0.5338


Round 4: 
(350, 51) (44, 51) (44, 51)
(350, 24) (44, 24) (44, 24)
MAE:  9.2632
MSE:  179.2283
RMSE:  13.3876
MAPE:  18.0424
Coefficient of determination (R-squared):  0.468


Round 5: 
(350, 51) (44, 51) (44, 51)
(350, 24) (44, 24) (44, 24)
MAE:  9.3601
MSE:  212.5656
RMSE:  14.5796
MAPE:  17.8112
Coefficient of determination (R-squared):  0.4267


Round 6: 
(350, 51) (44, 51) (44, 51)
(350, 24) (44, 24) (44, 24)
MAE:  9.2328
MSE:  216.2232
RMSE:  14.7045
MAPE:  16.4

In [144]:
#XGBM
mae,mse,rmse,mape,r2 = testing_runs(10,data_test,xgb8)

Round 1: 
(350, 51) (44, 51) (44, 51)
(350, 24) (44, 24) (44, 24)
MAE:  8.3087
MSE:  175.5249
RMSE:  13.2486
MAPE:  15.2158
Coefficient of determination (R-squared):  0.6382


Round 2: 
(350, 51) (44, 51) (44, 51)
(350, 24) (44, 24) (44, 24)
MAE:  10.657
MSE:  323.6676
RMSE:  17.9908
MAPE:  18.5608
Coefficient of determination (R-squared):  0.4235


Round 3: 
(350, 51) (44, 51) (44, 51)
(350, 24) (44, 24) (44, 24)
MAE:  8.3879
MSE:  128.6159
RMSE:  11.3409
MAPE:  16.0351
Coefficient of determination (R-squared):  0.5964


Round 4: 
(350, 51) (44, 51) (44, 51)
(350, 24) (44, 24) (44, 24)
MAE:  11.3504
MSE:  414.9733
RMSE:  20.3709
MAPE:  19.539
Coefficient of determination (R-squared):  0.5258


Round 5: 
(350, 51) (44, 51) (44, 51)
(350, 24) (44, 24) (44, 24)
MAE:  9.16
MSE:  239.9786
RMSE:  15.4912
MAPE:  16.0328
Coefficient of determination (R-squared):  0.4747


Round 6: 
(350, 51) (44, 51) (44, 51)
(350, 24) (44, 24) (44, 24)
MAE:  10.9576
MSE:  358.1443
RMSE:  18.9247
MAPE:  19.53

In [145]:
#light GBM
mae,mse,rmse,mape,r2 = testing_runs(10,data_test,lgbm8)

Round 1: 
(350, 51) (44, 51) (44, 51)
(350, 24) (44, 24) (44, 24)
MAE:  10.8713
MSE:  301.5317
RMSE:  17.3647
MAPE:  18.4979
Coefficient of determination (R-squared):  0.4394


Round 2: 
(350, 51) (44, 51) (44, 51)
(350, 24) (44, 24) (44, 24)
MAE:  10.3786
MSE:  223.1141
RMSE:  14.937
MAPE:  18.8731
Coefficient of determination (R-squared):  0.3504


Round 3: 
(350, 51) (44, 51) (44, 51)
(350, 24) (44, 24) (44, 24)
MAE:  10.255
MSE:  219.8319
RMSE:  14.8267
MAPE:  19.3491
Coefficient of determination (R-squared):  0.3821


Round 4: 
(350, 51) (44, 51) (44, 51)
(350, 24) (44, 24) (44, 24)
MAE:  10.4667
MSE:  247.3191
RMSE:  15.7264
MAPE:  18.7366
Coefficient of determination (R-squared):  0.4436


Round 5: 
(350, 51) (44, 51) (44, 51)
(350, 24) (44, 24) (44, 24)
MAE:  12.8026
MSE:  420.8767
RMSE:  20.5153
MAPE:  22.655
Coefficient of determination (R-squared):  0.328


Round 6: 
(350, 51) (44, 51) (44, 51)
(350, 24) (44, 24) (44, 24)
MAE:  9.8822
MSE:  266.9536
RMSE:  16.3387
MAPE:  18.

In [146]:
import pickle
# save the model to disk
pickle.dump(gbm7, open('GBM_basic_wdsp_avg_log_model.sav', 'wb'))
pickle.dump(gbm8, open('GBM_basic_wdsp_c_model.sav', 'wb'))
# pickle.dump(gbm6, open('GBM_basic_wdsp_sum_sqrt_model.sav', 'wb'))

pickle.dump(xgb7, open('XGBM_basic_wdsp_avg_log_model.sav', 'wb'))
pickle.dump(xgb8, open('XGBM_basic_wdsp_c_model.sav', 'wb'))
# pickle.dump(xgb6, open('XGBM_basic_wdsp_sum_sqrt_model.sav', 'wb'))

pickle.dump(lgbm7, open('LightGBM_basic_wdsp_avg_log_model.sav', 'wb'))
pickle.dump(lgbm8, open('LightGBM_basic_wdsp_c_model.sav', 'wb'))
# pickle.dump(lgbm6, open('LightGBM_basic_wdsp_sum_sqrt_model.sav', 'wb'))
#gbm = pickle.load(open('GBMtimefeatures1_model.sav', 'rb'))

<h1 style="text-align:center;font-size:40px;" > Basic time based features + ng_sqrt (square root of natural gas prices</h1>

In [171]:
data_1= data.copy()
data_1 = feature_pipeline_1(data_1)
ng = data.daily_natural_gas.values
ng_sqrt = np.sqrt(ng)
data_1['ng_sqrt'] = ng_sqrt
data_1.head()

,applicable_date,elec_prices,month,dayofyear_num,weekofyear_num,Hour,ng_sqrt
0,2019-01-01 00:00:00,70.28,1,1,1,0,1.617557
1,2019-01-01 01:00:00,65.18,1,1,1,1,1.617557
2,2019-01-01 02:00:00,60.91,1,1,1,2,1.617557
3,2019-01-01 03:00:00,65.16,1,1,1,3,1.617557
4,2019-01-01 04:00:00,65.16,1,1,1,4,1.617557


In [172]:
data_1 = multi_step(data_1,'elec_prices')
data_1 = calc_24lags(data_1,'elec_prices')
data_1 = calc_24lags(data_1,'ng_sqrt')
print(data_1.shape)
print(data_1.columns)
data_1.head()

(8304, 77)
Index(['applicable_date', 'elec_prices', 'month', 'dayofyear_num',
       'weekofyear_num', 'Hour', 'ng_sqrt', 't+1', 't+2', 't+3', 't+4', 't+5',
       't+6', 't+7', 't+8', 't+9', 't+10', 't+11', 't+12', 't+13', 't+14',
       't+15', 't+16', 't+17', 't+18', 't+19', 't+20', 't+21', 't+22', 't+23',
       'lag_1_elec_prices', 'lag_2_elec_prices', 'lag_3_elec_prices',
       'lag_4_elec_prices', 'lag_5_elec_prices', 'lag_6_elec_prices',
       'lag_7_elec_prices', 'lag_8_elec_prices', 'lag_9_elec_prices',
       'lag_10_elec_prices', 'lag_11_elec_prices', 'lag_12_elec_prices',
       'lag_13_elec_prices', 'lag_14_elec_prices', 'lag_15_elec_prices',
       'lag_16_elec_prices', 'lag_17_elec_prices', 'lag_18_elec_prices',
       'lag_19_elec_prices', 'lag_20_elec_prices', 'lag_21_elec_prices',
       'lag_22_elec_prices', 'lag_23_elec_prices', 'lag_24_elec_prices',
       'lag_1_ng_sqrt', 'lag_2_ng_sqrt', 'lag_3_ng_sqrt', 'lag_4_ng_sqrt',
       'lag_5_ng_sqrt', 'lag_6_ng_sqrt'

,applicable_date,elec_prices,month,dayofyear_num,weekofyear_num,Hour,ng_sqrt,t+1,t+2,t+3,...,lag_14_ng_sqrt,lag_15_ng_sqrt,lag_16_ng_sqrt,lag_17_ng_sqrt,lag_18_ng_sqrt,lag_19_ng_sqrt,lag_20_ng_sqrt,lag_21_ng_sqrt,lag_22_ng_sqrt,lag_23_ng_sqrt
0,2019-01-01 00:00:00,70.28,1,1,1,0,1.617557,65.18,60.91,65.160,...,1.617557,1.617557,1.617557,1.617557,1.617557,1.617557,1.617557,1.617557,1.617557,1.617557
1,2019-01-01 01:00:00,65.18,1,1,1,1,1.617557,60.91,65.16,65.160,...,1.617557,1.617557,1.617557,1.617557,1.617557,1.617557,1.617557,1.617557,1.617557,1.617557
2,2019-01-01 02:00:00,60.91,1,1,1,2,1.617557,65.16,65.16,68.290,...,1.617557,1.617557,1.617557,1.617557,1.617557,1.617557,1.617557,1.617557,1.617557,1.617557
3,2019-01-01 03:00:00,65.16,1,1,1,3,1.617557,65.16,68.29,63.030,...,1.617557,1.617557,1.617557,1.617557,1.617557,1.617557,1.617557,1.617557,1.617557,1.617557
4,2019-01-01 04:00:00,65.16,1,1,1,4,1.617557,68.29,63.03,51.621,...,1.617557,1.617557,1.617557,1.617557,1.617557,1.617557,1.617557,1.617557,1.617557,1.617557


In [173]:
data_1.isnull().values.any()

True

In [174]:
#Dropping rows with Nan's
data_1 = data_1.dropna(axis=0)
print(data_1.shape)
data_1.isnull().values.any()

(8281, 77)


False

In [175]:
data_1 = remove_23rows_hour_col(data_1)
train,test = train_test_split_tbs(data_1)
train.columns

(345, 76)
(276, 76)
(69, 76)


Index(['applicable_date', 'elec_prices', 'month', 'dayofyear_num',
       'weekofyear_num', 'ng_sqrt', 't+1', 't+2', 't+3', 't+4', 't+5', 't+6',
       't+7', 't+8', 't+9', 't+10', 't+11', 't+12', 't+13', 't+14', 't+15',
       't+16', 't+17', 't+18', 't+19', 't+20', 't+21', 't+22', 't+23',
       'lag_1_elec_prices', 'lag_2_elec_prices', 'lag_3_elec_prices',
       'lag_4_elec_prices', 'lag_5_elec_prices', 'lag_6_elec_prices',
       'lag_7_elec_prices', 'lag_8_elec_prices', 'lag_9_elec_prices',
       'lag_10_elec_prices', 'lag_11_elec_prices', 'lag_12_elec_prices',
       'lag_13_elec_prices', 'lag_14_elec_prices', 'lag_15_elec_prices',
       'lag_16_elec_prices', 'lag_17_elec_prices', 'lag_18_elec_prices',
       'lag_19_elec_prices', 'lag_20_elec_prices', 'lag_21_elec_prices',
       'lag_22_elec_prices', 'lag_23_elec_prices', 'lag_24_elec_prices',
       'lag_1_ng_sqrt', 'lag_2_ng_sqrt', 'lag_3_ng_sqrt', 'lag_4_ng_sqrt',
       'lag_5_ng_sqrt', 'lag_6_ng_sqrt', 'lag_7_ng_sqrt', 

In [176]:
#selecting columns for X & y
cols = train.columns.to_list()
x_names = cols[2:6]
lags = cols[-47:]
x_names.extend(lags)
y_names = [cols[1]]
time_steps = cols[6:29]
y_names.extend(time_steps)
print(x_names)
print(y_names)

['month', 'dayofyear_num', 'weekofyear_num', 'ng_sqrt', 'lag_1_elec_prices', 'lag_2_elec_prices', 'lag_3_elec_prices', 'lag_4_elec_prices', 'lag_5_elec_prices', 'lag_6_elec_prices', 'lag_7_elec_prices', 'lag_8_elec_prices', 'lag_9_elec_prices', 'lag_10_elec_prices', 'lag_11_elec_prices', 'lag_12_elec_prices', 'lag_13_elec_prices', 'lag_14_elec_prices', 'lag_15_elec_prices', 'lag_16_elec_prices', 'lag_17_elec_prices', 'lag_18_elec_prices', 'lag_19_elec_prices', 'lag_20_elec_prices', 'lag_21_elec_prices', 'lag_22_elec_prices', 'lag_23_elec_prices', 'lag_24_elec_prices', 'lag_1_ng_sqrt', 'lag_2_ng_sqrt', 'lag_3_ng_sqrt', 'lag_4_ng_sqrt', 'lag_5_ng_sqrt', 'lag_6_ng_sqrt', 'lag_7_ng_sqrt', 'lag_8_ng_sqrt', 'lag_9_ng_sqrt', 'lag_10_ng_sqrt', 'lag_11_ng_sqrt', 'lag_12_ng_sqrt', 'lag_13_ng_sqrt', 'lag_14_ng_sqrt', 'lag_15_ng_sqrt', 'lag_16_ng_sqrt', 'lag_17_ng_sqrt', 'lag_18_ng_sqrt', 'lag_19_ng_sqrt', 'lag_20_ng_sqrt', 'lag_21_ng_sqrt', 'lag_22_ng_sqrt', 'lag_23_ng_sqrt']
['elec_prices', 't+1

In [177]:
x_train, y_train = train[x_names], train[y_names]
x_test, y_test = test[x_names], test[y_names]
print(x_train.shape,y_train.shape)
print(x_test.shape,y_test.shape)

(276, 51) (276, 24)
(69, 51) (69, 24)


In [178]:
x_train_norm,x_test_norm = normalize(x_train,x_test)

In [179]:
GBM_training(x_train_norm,y_train)

Duration: 0:00:18.238577



{'mean_fit_time': array([0.43803189, 0.60185974, 2.11662617, 2.06666985, 0.85464935,
       2.48237514, 1.88089721, 1.33534949, 2.08972948, 0.59322371]), 'std_fit_time': array([0.01814566, 0.02009222, 0.09896918, 0.09769638, 0.0254477 ,
       0.31046573, 0.1854314 , 0.28657131, 0.263284  , 0.04962388]), 'mean_score_time': array([0.01780453, 0.01839795, 0.02215221, 0.02106054, 0.01885633,
       0.02243395, 0.02709754, 0.02278502, 0.01990807, 0.01496313]), 'std_score_time': array([0.0019657 , 0.00152824, 0.00638768, 0.00294319, 0.00260979,
       0.003414  , 0.01471264, 0.0062778 , 0.00330406, 0.00588741]), 'param_estimator__subsample': masked_array(data=[0.85, 0.7, 0.8, 0.85, 0.85, 0.85, 0.5, 0.9, 0.7, 0.6],
             mask=[False, False, False, False, False, False, False, False,
                   False, False],
       fill_value='?',
            dtype=object), 'param_estimator__n_estimators': masked_array(data=[20, 30, 100, 90, 40, 90, 100, 60, 100, 30]

In [180]:
#learning_rate,n_estimators,max_depth,min_samples_split,max_features,min_samples_leaf,subsample
gbm9 = GBM_model(x_train_norm,y_train,0.025,90,5,100,3,10,0.85)

Duration: 0:00:02.243072


In [181]:
y_pred= gbm9.predict(x_test_norm)
eval_metrics(y_test,y_pred)

MAE:  10.6322
MSE:  231.7294
RMSE:  15.2227
MAPE:  22.6509
Coefficient of determination (R-squared):  0.0888




(10.632209075638476,
 231.7293745384857,
 15.222659903528218,
 22.650897651594388,
 0.08875888196297695)

In [182]:
XGBM_training(x_train_norm,y_train)

Duration: 0:02:01.781259



{'mean_fit_time': array([13.88543363,  6.33938513, 14.17825589,  5.39082806, 33.79062476,
        1.58893013, 20.73271587, 12.15576987, 17.88636498,  6.75592864]), 'std_fit_time': array([0.9832887 , 1.65810263, 1.42809159, 0.46119575, 2.91108621,
       0.09901499, 1.29558988, 1.04012618, 0.52724321, 0.96898736]), 'mean_score_time': array([0.08838589, 0.06266601, 0.0756649 , 0.05327168, 0.11040988,
       0.04114838, 0.06347098, 0.06526361, 0.06564312, 0.03829489]), 'std_score_time': array([0.01528019, 0.00937133, 0.01604571, 0.00459009, 0.04097541,
       0.0067066 , 0.00986086, 0.01652149, 0.01300858, 0.0164976 ]), 'param_estimator__subsample': masked_array(data=[0.30000000000000004, 1.0, 0.4, 1.0, 0.4, 0.8, 0.6,
                   0.30000000000000004, 0.5, 0.5],
             mask=[False, False, False, False, False, False, False, False,
                   False, False],
       fill_value='?',
            dtype=object), 'param_estimator__n_estimators': mask

In [183]:
#learning_rate,n_estimators,max_depth,subsample,colsample_bytree
xgb9 = XGBM_model(x_train_norm,y_train,0.0125,300,9,0.5,0.6)

Duration: 0:00:04.633602


In [184]:
y_pred= xgb9.predict(x_test_norm)
eval_metrics(y_test,y_pred)

MAE:  11.5741
MSE:  270.91
RMSE:  16.4593
MAPE:  24.6574
Coefficient of determination (R-squared):  -0.0102




(11.574054205173455,
 270.90997478862647,
 16.459343084966253,
 24.657408009035642,
 -0.010199074852255621)

In [185]:
LGBM_training(x_train_norm,y_train)

[LightGBM] [Warning] min_child_samples is set=20, min_child_samples= will be ignored. Current value: min_child_samples=20
[LightGBM] [Warning] Unknown parameter: 40
Duration: 0:01:40.191887



{'mean_fit_time': array([ 2.08606818, 35.16866806, 12.32607646,  4.31011786,  6.2294683 ,
        2.31320486,  4.20495195, 36.97972806,  3.93721559,  4.22838519]), 'std_fit_time': array([0.43886382, 2.00054623, 1.52834906, 0.22644108, 0.28634235,
       0.08066886, 0.14009083, 1.33660061, 0.34498555, 0.61771342]), 'mean_score_time': array([0.03214126, 0.06281357, 0.09180608, 0.04498093, 0.06965549,
       0.03930931, 0.06384563, 0.05914624, 0.04718227, 0.03298285]), 'std_score_time': array([0.01597797, 0.0086079 , 0.01890015, 0.01879233, 0.01932937,
       0.00529896, 0.01282975, 0.01402319, 0.05615819, 0.00981484]), 'param_estimator__subsample': masked_array(data=[0.2, 0.30000000000000004, 0.5, 0.8,
                   0.30000000000000004, 0.8, 0.5, 0.6, 0.4, 0.2],
             mask=[False, False

In [186]:
#num_leaves,min_child_samples,learning_rate,n_estimators,max_depth,subsample,colsample_bytree
lgbm9 = LGBM_model(x_train_norm,y_train,307,40,0.0125,150,9,0.2,0.8)

Duration: 0:00:01.700873


In [187]:
y_pred= lgbm9.predict(x_test_norm)
eval_metrics(y_test,y_pred)

MAE:  10.6551
MSE:  225.5176
RMSE:  15.0172
MAPE:  22.6996
Coefficient of determination (R-squared):  0.0999




(10.655053938431044,
 225.5176190034365,
 15.017244054866941,
 22.69956643202355,
 0.0999286033692361)

### Testing on 2018-2019 data

In [188]:
data_test = data_1819.copy()
data_test = feature_pipeline_1(data_test)
ng = data_1819.daily_natural_gas.values
ng_sqrt = np.sqrt(ng)
data_test['ng_sqrt'] = ng_sqrt

data_test = multi_step(data_test,'elec_prices')
data_test = calc_24lags(data_test,'elec_prices')
data_test = calc_24lags(data_test,'ng_sqrt')
print(data_test.shape)
print(data_test.columns)
data_test.head()

(10514, 77)
Index(['applicable_date', 'elec_prices', 'month', 'dayofyear_num',
       'weekofyear_num', 'Hour', 'ng_sqrt', 't+1', 't+2', 't+3', 't+4', 't+5',
       't+6', 't+7', 't+8', 't+9', 't+10', 't+11', 't+12', 't+13', 't+14',
       't+15', 't+16', 't+17', 't+18', 't+19', 't+20', 't+21', 't+22', 't+23',
       'lag_1_elec_prices', 'lag_2_elec_prices', 'lag_3_elec_prices',
       'lag_4_elec_prices', 'lag_5_elec_prices', 'lag_6_elec_prices',
       'lag_7_elec_prices', 'lag_8_elec_prices', 'lag_9_elec_prices',
       'lag_10_elec_prices', 'lag_11_elec_prices', 'lag_12_elec_prices',
       'lag_13_elec_prices', 'lag_14_elec_prices', 'lag_15_elec_prices',
       'lag_16_elec_prices', 'lag_17_elec_prices', 'lag_18_elec_prices',
       'lag_19_elec_prices', 'lag_20_elec_prices', 'lag_21_elec_prices',
       'lag_22_elec_prices', 'lag_23_elec_prices', 'lag_24_elec_prices',
       'lag_1_ng_sqrt', 'lag_2_ng_sqrt', 'lag_3_ng_sqrt', 'lag_4_ng_sqrt',
       'lag_5_ng_sqrt', 'lag_6_ng_sqrt

,applicable_date,elec_prices,month,dayofyear_num,weekofyear_num,Hour,ng_sqrt,t+1,t+2,t+3,...,lag_14_ng_sqrt,lag_15_ng_sqrt,lag_16_ng_sqrt,lag_17_ng_sqrt,lag_18_ng_sqrt,lag_19_ng_sqrt,lag_20_ng_sqrt,lag_21_ng_sqrt,lag_22_ng_sqrt,lag_23_ng_sqrt
0,2018-09-30 22:00:00,71.267,9,273,39,22,1.669179,67.212,60.500,63.682,...,1.669179,1.669179,1.669179,1.669179,1.669179,1.669179,1.669179,1.669179,1.669179,1.669179
1,2018-09-30 23:00:00,67.212,9,273,39,23,1.669179,60.500,63.682,71.617,...,1.669179,1.669179,1.669179,1.669179,1.669179,1.669179,1.669179,1.669179,1.669179,1.669179
2,2018-10-01 00:00:00,60.500,10,274,40,0,1.671852,63.682,71.617,72.855,...,1.669179,1.669179,1.669179,1.669179,1.669179,1.669179,1.669179,1.669179,1.669179,1.669179
3,2018-10-01 01:00:00,63.682,10,274,40,1,1.671852,71.617,72.855,76.937,...,1.669179,1.669179,1.669179,1.669179,1.669179,1.669179,1.669179,1.669179,1.669179,1.669179
4,2018-10-01 02:00:00,71.617,10,274,40,2,1.671852,72.855,76.937,80.480,...,1.669179,1.669179,1.669179,1.669179,1.669179,1.669179,1.669179,1.669179,1.669179,1.669179


In [189]:
#Dropping rows with Nan's
data_test = data_test.dropna(axis=0)
print(data_test.shape)
data_test.isnull().values.any()

(10491, 77)


False

In [190]:
data_test = remove_23rows_hour_col(data_test)
data_test.columns

(438, 76)


Index(['applicable_date', 'elec_prices', 'month', 'dayofyear_num',
       'weekofyear_num', 'ng_sqrt', 't+1', 't+2', 't+3', 't+4', 't+5', 't+6',
       't+7', 't+8', 't+9', 't+10', 't+11', 't+12', 't+13', 't+14', 't+15',
       't+16', 't+17', 't+18', 't+19', 't+20', 't+21', 't+22', 't+23',
       'lag_1_elec_prices', 'lag_2_elec_prices', 'lag_3_elec_prices',
       'lag_4_elec_prices', 'lag_5_elec_prices', 'lag_6_elec_prices',
       'lag_7_elec_prices', 'lag_8_elec_prices', 'lag_9_elec_prices',
       'lag_10_elec_prices', 'lag_11_elec_prices', 'lag_12_elec_prices',
       'lag_13_elec_prices', 'lag_14_elec_prices', 'lag_15_elec_prices',
       'lag_16_elec_prices', 'lag_17_elec_prices', 'lag_18_elec_prices',
       'lag_19_elec_prices', 'lag_20_elec_prices', 'lag_21_elec_prices',
       'lag_22_elec_prices', 'lag_23_elec_prices', 'lag_24_elec_prices',
       'lag_1_ng_sqrt', 'lag_2_ng_sqrt', 'lag_3_ng_sqrt', 'lag_4_ng_sqrt',
       'lag_5_ng_sqrt', 'lag_6_ng_sqrt', 'lag_7_ng_sqrt', 

In [191]:
#selecting columns for X & y
cols = train.columns.to_list()
x_names = cols[2:6]
lags = cols[-47:]
x_names.extend(lags)
y_names = [cols[1]]
time_steps = cols[6:29]
y_names.extend(time_steps)
print(x_names)
print(y_names)

['month', 'dayofyear_num', 'weekofyear_num', 'ng_sqrt', 'lag_1_elec_prices', 'lag_2_elec_prices', 'lag_3_elec_prices', 'lag_4_elec_prices', 'lag_5_elec_prices', 'lag_6_elec_prices', 'lag_7_elec_prices', 'lag_8_elec_prices', 'lag_9_elec_prices', 'lag_10_elec_prices', 'lag_11_elec_prices', 'lag_12_elec_prices', 'lag_13_elec_prices', 'lag_14_elec_prices', 'lag_15_elec_prices', 'lag_16_elec_prices', 'lag_17_elec_prices', 'lag_18_elec_prices', 'lag_19_elec_prices', 'lag_20_elec_prices', 'lag_21_elec_prices', 'lag_22_elec_prices', 'lag_23_elec_prices', 'lag_24_elec_prices', 'lag_1_ng_sqrt', 'lag_2_ng_sqrt', 'lag_3_ng_sqrt', 'lag_4_ng_sqrt', 'lag_5_ng_sqrt', 'lag_6_ng_sqrt', 'lag_7_ng_sqrt', 'lag_8_ng_sqrt', 'lag_9_ng_sqrt', 'lag_10_ng_sqrt', 'lag_11_ng_sqrt', 'lag_12_ng_sqrt', 'lag_13_ng_sqrt', 'lag_14_ng_sqrt', 'lag_15_ng_sqrt', 'lag_16_ng_sqrt', 'lag_17_ng_sqrt', 'lag_18_ng_sqrt', 'lag_19_ng_sqrt', 'lag_20_ng_sqrt', 'lag_21_ng_sqrt', 'lag_22_ng_sqrt', 'lag_23_ng_sqrt']
['elec_prices', 't+1

In [192]:
#GBM
mae,mse,rmse,mape,r2 = testing_runs(10,data_test,gbm9)

Round 1: 
(350, 51) (44, 51) (44, 51)
(350, 24) (44, 24) (44, 24)
MAE:  9.0505
MSE:  172.7878
RMSE:  13.1449
MAPE:  17.2445
Coefficient of determination (R-squared):  0.4904


Round 2: 
(350, 51) (44, 51) (44, 51)
(350, 24) (44, 24) (44, 24)
MAE:  10.7286
MSE:  215.8937
RMSE:  14.6933
MAPE:  20.8633
Coefficient of determination (R-squared):  0.4756


Round 3: 
(350, 51) (44, 51) (44, 51)
(350, 24) (44, 24) (44, 24)
MAE:  11.5352
MSE:  310.5269
RMSE:  17.6218
MAPE:  20.2396
Coefficient of determination (R-squared):  0.5074


Round 4: 
(350, 51) (44, 51) (44, 51)
(350, 24) (44, 24) (44, 24)
MAE:  10.4724
MSE:  242.8466
RMSE:  15.5835
MAPE:  19.4384
Coefficient of determination (R-squared):  0.4469


Round 5: 
(350, 51) (44, 51) (44, 51)
(350, 24) (44, 24) (44, 24)
MAE:  11.6914
MSE:  338.079
RMSE:  18.3869
MAPE:  20.3861
Coefficient of determination (R-squared):  0.3694


Round 6: 
(350, 51) (44, 51) (44, 51)
(350, 24) (44, 24) (44, 24)
MAE:  10.7807
MSE:  327.9479
RMSE:  18.1093
MAPE:  

In [193]:
#XGBM
mae,mse,rmse,mape,r2 = testing_runs(10,data_test,xgb9)

Round 1: 
(350, 51) (44, 51) (44, 51)
(350, 24) (44, 24) (44, 24)
MAE:  10.4442
MSE:  263.7966
RMSE:  16.2418
MAPE:  19.196
Coefficient of determination (R-squared):  0.5113


Round 2: 
(350, 51) (44, 51) (44, 51)
(350, 24) (44, 24) (44, 24)
MAE:  11.8566
MSE:  346.4661
RMSE:  18.6136
MAPE:  21.534
Coefficient of determination (R-squared):  0.3648


Round 3: 
(350, 51) (44, 51) (44, 51)
(350, 24) (44, 24) (44, 24)
MAE:  9.1206
MSE:  217.801
RMSE:  14.7581
MAPE:  16.2949
Coefficient of determination (R-squared):  0.5358


Round 4: 
(350, 51) (44, 51) (44, 51)
(350, 24) (44, 24) (44, 24)
MAE:  11.0293
MSE:  340.0927
RMSE:  18.4416
MAPE:  19.8159
Coefficient of determination (R-squared):  0.4324


Round 5: 
(350, 51) (44, 51) (44, 51)
(350, 24) (44, 24) (44, 24)
MAE:  10.3261
MSE:  263.2885
RMSE:  16.2262
MAPE:  18.1188
Coefficient of determination (R-squared):  0.5091


Round 6: 
(350, 51) (44, 51) (44, 51)
(350, 24) (44, 24) (44, 24)
MAE:  10.911
MSE:  332.1867
RMSE:  18.226
MAPE:  20.0

In [194]:
#light GBM
mae,mse,rmse,mape,r2 = testing_runs(10,data_test,lgbm9)

Round 1: 
(350, 51) (44, 51) (44, 51)
(350, 24) (44, 24) (44, 24)
MAE:  12.6936
MSE:  540.1819
RMSE:  23.2418
MAPE:  20.7621
Coefficient of determination (R-squared):  0.4076


Round 2: 
(350, 51) (44, 51) (44, 51)
(350, 24) (44, 24) (44, 24)
MAE:  10.625
MSE:  261.2585
RMSE:  16.1635
MAPE:  18.6864
Coefficient of determination (R-squared):  0.463


Round 3: 
(350, 51) (44, 51) (44, 51)
(350, 24) (44, 24) (44, 24)
MAE:  10.8755
MSE:  279.3365
RMSE:  16.7134
MAPE:  19.3865
Coefficient of determination (R-squared):  0.4655


Round 4: 
(350, 51) (44, 51) (44, 51)
(350, 24) (44, 24) (44, 24)
MAE:  12.1709
MSE:  417.1829
RMSE:  20.4251
MAPE:  21.2357
Coefficient of determination (R-squared):  0.4086


Round 5: 
(350, 51) (44, 51) (44, 51)
(350, 24) (44, 24) (44, 24)
MAE:  9.9125
MSE:  245.5648
RMSE:  15.6705
MAPE:  18.157
Coefficient of determination (R-squared):  0.3969


Round 6: 
(350, 51) (44, 51) (44, 51)
(350, 24) (44, 24) (44, 24)
MAE:  10.5463
MSE:  247.9739
RMSE:  15.7472
MAPE:  18

<h1 style="text-align:center;font-size:40px;" > Basic time based features + daily_natural_gas (natural gas prices)</h1>

In [147]:
data_1= data.copy()
data_1 = feature_pipeline_1(data_1)
data_1['daily_natural_gas'] = data['daily_natural_gas']
data_1.head()

,applicable_date,elec_prices,month,dayofyear_num,weekofyear_num,Hour,daily_natural_gas
0,2019-01-01 00:00:00,70.28,1,1,1,0,2.61649
1,2019-01-01 01:00:00,65.18,1,1,1,1,2.61649
2,2019-01-01 02:00:00,60.91,1,1,1,2,2.61649
3,2019-01-01 03:00:00,65.16,1,1,1,3,2.61649
4,2019-01-01 04:00:00,65.16,1,1,1,4,2.61649


In [148]:
data_1 = multi_step(data_1,'elec_prices')
data_1 = calc_24lags(data_1,'elec_prices')
data_1 = calc_24lags(data_1,'daily_natural_gas')
print(data_1.shape)
print(data_1.columns)
data_1.head()

(8304, 77)
Index(['applicable_date', 'elec_prices', 'month', 'dayofyear_num',
       'weekofyear_num', 'Hour', 'daily_natural_gas', 't+1', 't+2', 't+3',
       't+4', 't+5', 't+6', 't+7', 't+8', 't+9', 't+10', 't+11', 't+12',
       't+13', 't+14', 't+15', 't+16', 't+17', 't+18', 't+19', 't+20', 't+21',
       't+22', 't+23', 'lag_1_elec_prices', 'lag_2_elec_prices',
       'lag_3_elec_prices', 'lag_4_elec_prices', 'lag_5_elec_prices',
       'lag_6_elec_prices', 'lag_7_elec_prices', 'lag_8_elec_prices',
       'lag_9_elec_prices', 'lag_10_elec_prices', 'lag_11_elec_prices',
       'lag_12_elec_prices', 'lag_13_elec_prices', 'lag_14_elec_prices',
       'lag_15_elec_prices', 'lag_16_elec_prices', 'lag_17_elec_prices',
       'lag_18_elec_prices', 'lag_19_elec_prices', 'lag_20_elec_prices',
       'lag_21_elec_prices', 'lag_22_elec_prices', 'lag_23_elec_prices',
       'lag_24_elec_prices', 'lag_1_daily_natural_gas',
       'lag_2_daily_natural_gas', 'lag_3_daily_natural_gas',
       'l

,applicable_date,elec_prices,month,dayofyear_num,weekofyear_num,Hour,daily_natural_gas,t+1,t+2,t+3,...,lag_14_daily_natural_gas,lag_15_daily_natural_gas,lag_16_daily_natural_gas,lag_17_daily_natural_gas,lag_18_daily_natural_gas,lag_19_daily_natural_gas,lag_20_daily_natural_gas,lag_21_daily_natural_gas,lag_22_daily_natural_gas,lag_23_daily_natural_gas
0,2019-01-01 00:00:00,70.28,1,1,1,0,2.61649,65.18,60.91,65.160,...,2.61649,2.61649,2.61649,2.61649,2.61649,2.61649,2.61649,2.61649,2.61649,2.61649
1,2019-01-01 01:00:00,65.18,1,1,1,1,2.61649,60.91,65.16,65.160,...,2.61649,2.61649,2.61649,2.61649,2.61649,2.61649,2.61649,2.61649,2.61649,2.61649
2,2019-01-01 02:00:00,60.91,1,1,1,2,2.61649,65.16,65.16,68.290,...,2.61649,2.61649,2.61649,2.61649,2.61649,2.61649,2.61649,2.61649,2.61649,2.61649
3,2019-01-01 03:00:00,65.16,1,1,1,3,2.61649,65.16,68.29,63.030,...,2.61649,2.61649,2.61649,2.61649,2.61649,2.61649,2.61649,2.61649,2.61649,2.61649
4,2019-01-01 04:00:00,65.16,1,1,1,4,2.61649,68.29,63.03,51.621,...,2.61649,2.61649,2.61649,2.61649,2.61649,2.61649,2.61649,2.61649,2.61649,2.61649


In [149]:
data_1.isnull().values.any()

True

In [150]:
#Dropping rows with Nan's
data_1 = data_1.dropna(axis=0)
print(data_1.shape)
data_1.isnull().values.any()

(8281, 77)


False

In [151]:
data_1 = remove_23rows_hour_col(data_1)
train,test = train_test_split_tbs(data_1)
train.columns

(345, 76)
(276, 76)
(69, 76)


Index(['applicable_date', 'elec_prices', 'month', 'dayofyear_num',
       'weekofyear_num', 'daily_natural_gas', 't+1', 't+2', 't+3', 't+4',
       't+5', 't+6', 't+7', 't+8', 't+9', 't+10', 't+11', 't+12', 't+13',
       't+14', 't+15', 't+16', 't+17', 't+18', 't+19', 't+20', 't+21', 't+22',
       't+23', 'lag_1_elec_prices', 'lag_2_elec_prices', 'lag_3_elec_prices',
       'lag_4_elec_prices', 'lag_5_elec_prices', 'lag_6_elec_prices',
       'lag_7_elec_prices', 'lag_8_elec_prices', 'lag_9_elec_prices',
       'lag_10_elec_prices', 'lag_11_elec_prices', 'lag_12_elec_prices',
       'lag_13_elec_prices', 'lag_14_elec_prices', 'lag_15_elec_prices',
       'lag_16_elec_prices', 'lag_17_elec_prices', 'lag_18_elec_prices',
       'lag_19_elec_prices', 'lag_20_elec_prices', 'lag_21_elec_prices',
       'lag_22_elec_prices', 'lag_23_elec_prices', 'lag_24_elec_prices',
       'lag_1_daily_natural_gas', 'lag_2_daily_natural_gas',
       'lag_3_daily_natural_gas', 'lag_4_daily_natural_gas',
 

In [152]:
#selecting columns for X & y
cols = train.columns.to_list()
x_names = cols[2:6]
lags = cols[-47:]
x_names.extend(lags)
y_names = [cols[1]]
time_steps = cols[6:29]
y_names.extend(time_steps)
print(x_names)
print(y_names)

['month', 'dayofyear_num', 'weekofyear_num', 'daily_natural_gas', 'lag_1_elec_prices', 'lag_2_elec_prices', 'lag_3_elec_prices', 'lag_4_elec_prices', 'lag_5_elec_prices', 'lag_6_elec_prices', 'lag_7_elec_prices', 'lag_8_elec_prices', 'lag_9_elec_prices', 'lag_10_elec_prices', 'lag_11_elec_prices', 'lag_12_elec_prices', 'lag_13_elec_prices', 'lag_14_elec_prices', 'lag_15_elec_prices', 'lag_16_elec_prices', 'lag_17_elec_prices', 'lag_18_elec_prices', 'lag_19_elec_prices', 'lag_20_elec_prices', 'lag_21_elec_prices', 'lag_22_elec_prices', 'lag_23_elec_prices', 'lag_24_elec_prices', 'lag_1_daily_natural_gas', 'lag_2_daily_natural_gas', 'lag_3_daily_natural_gas', 'lag_4_daily_natural_gas', 'lag_5_daily_natural_gas', 'lag_6_daily_natural_gas', 'lag_7_daily_natural_gas', 'lag_8_daily_natural_gas', 'lag_9_daily_natural_gas', 'lag_10_daily_natural_gas', 'lag_11_daily_natural_gas', 'lag_12_daily_natural_gas', 'lag_13_daily_natural_gas', 'lag_14_daily_natural_gas', 'lag_15_daily_natural_gas', 'lag

In [153]:
x_train, y_train = train[x_names], train[y_names]
x_test, y_test = test[x_names], test[y_names]
print(x_train.shape,y_train.shape)
print(x_test.shape,y_test.shape)

(276, 51) (276, 24)
(69, 51) (69, 24)


In [154]:
x_train_norm,x_test_norm = normalize(x_train,x_test)

In [155]:
GBM_training(x_train_norm,y_train)

Duration: 0:00:11.508486



{'mean_fit_time': array([0.76615744, 0.31885395, 0.25667739, 0.97280583, 0.76886313,
       1.4467463 , 0.41038098, 0.61879241, 0.62382393, 0.23217905]), 'std_fit_time': array([0.03949873, 0.02394988, 0.02117242, 0.05297444, 0.03534312,
       0.03751513, 0.00925794, 0.0200111 , 0.01992543, 0.02504027]), 'mean_score_time': array([0.01250172, 0.01169806, 0.01092789, 0.01191311, 0.00912011,
       0.00937493, 0.00950835, 0.01406326, 0.01121192, 0.01241581]), 'std_score_time': array([0.00625086, 0.00702574, 0.0023666 , 0.00144381, 0.00337006,
       0.0076546 , 0.00777292, 0.00468775, 0.00745538, 0.006213  ]), 'param_estimator__subsample': masked_array(data=[0.6, 0.9, 0.75, 0.75, 0.9, 0.6, 0.85, 0.8, 0.9, 0.75],
             mask=[False, False, False, False, False, False, False, False,
                   False, False],
       fill_value='?',
            dtype=object), 'param_estimator__n_estimators': masked_array(data=[70, 20, 20, 90, 60, 100, 40, 60, 60, 20],


In [156]:
#learning_rate,n_estimators,max_depth,min_samples_split,max_features,min_samples_leaf,subsample
gbm10 = GBM_model(x_train_norm,y_train,0.025,100,13,100,6,10,0.6)

Duration: 0:00:00.825026


In [157]:
y_pred= gbm10.predict(x_test_norm)
eval_metrics(y_test,y_pred)

MAE:  10.5206
MSE:  227.5483
RMSE:  15.0847
MAPE:  22.4132
Coefficient of determination (R-squared):  0.107




(10.52061908424484,
 227.54832163379925,
 15.08470489051076,
 22.413165920021214,
 0.10703463902113143)

In [158]:
XGBM_training(x_train_norm,y_train)

Duration: 0:01:03.523068



{'mean_fit_time': array([13.50811417,  2.81170111,  0.48247795, 11.0685955 , 18.8599231 ,
        2.82639625,  4.20013063,  5.0662184 ,  2.12239621,  8.07180755]), 'std_fit_time': array([0.40985197, 1.01849093, 0.0177599 , 0.48537533, 0.74132262,
       0.20582722, 0.14648628, 0.2082765 , 0.09395178, 0.17802594]), 'mean_score_time': array([0.04496219, 0.02759297, 0.02272189, 0.04261131, 0.05096309,
       0.02649164, 0.0267184 , 0.0358073 , 0.02350535, 0.01946161]), 'std_score_time': array([0.00434793, 0.00161186, 0.00183259, 0.00708942, 0.0094966 ,
       0.00454893, 0.00693651, 0.01088895, 0.00191093, 0.00510895]), 'param_estimator__subsample': masked_array(data=[0.2, 0.4, 0.1, 0.2, 0.5, 0.7000000000000001, 0.8, 0.9,
                   0.8, 0.9],
             mask=[False, False, False, False, False, False, False, False,
                   False, False],
       fill_value='?',
            dtype=object), 'param_estimator__n_estimators': masked_array(data=[40

In [159]:
#learning_rate,n_estimators,max_depth,subsample,colsample_bytree
xgb10 = XGBM_model(x_train_norm,y_train,0.0125,350,3,0.4,0.1)

Duration: 0:00:01.366030


In [160]:
y_pred= xgb10.predict(x_test_norm)
eval_metrics(y_test,y_pred)

MAE:  10.5918
MSE:  229.9936
RMSE:  15.1655
MAPE:  22.5649
Coefficient of determination (R-squared):  0.1205




(10.591821572144825,
 229.99363970073728,
 15.165541193796457,
 22.5648559643467,
 0.120490093388544)

In [161]:
LGBM_training(x_train_norm,y_train)

[LightGBM] [Warning] min_child_samples is set=20, min_child_samples= will be ignored. Current value: min_child_samples=20
[LightGBM] [Warning] Unknown parameter: 60
Duration: 0:00:57.000840



{'mean_fit_time': array([ 1.78428292,  1.72273071, 33.10446379, 10.26829333,  2.84399168,
        4.09787962,  3.36867337,  2.4624629 ,  0.90581124,  2.09008906]), 'std_fit_time': array([0.0766641 , 0.11153216, 1.34561849, 1.63742897, 0.2165776 ,
       0.12960558, 0.17767838, 0.12632941, 0.0338851 , 0.10677964]), 'mean_score_time': array([0.01170106, 0.03411894, 0.05438316, 0.06496727, 0.02430789,
       0.04679625, 0.03017969, 0.0404114 , 0.02720256, 0.01996315]), 'std_score_time': array([0.00419608, 0.00577906, 0.00901378, 0.01618119, 0.00567955,
       0.00703851, 0.00894279, 0.01162147, 0.0235269 , 0.00667264]), 'param_estimator__subsample': masked_array(data=[0.8, 0.7000000000000001, 0.2, 0.8, 0.5, 0.2,
                   0.30000000000000004, 0.8, 0.30000000000000004, 0.9],
             mas

In [162]:
#num_leaves,min_child_samples,learning_rate,n_estimators,max_depth,subsample,colsample_bytree
lgbm10 = LGBM_model(x_train_norm,y_train,19,60,0.0125,450,7,0.2,0.4)

Duration: 0:00:01.187004


In [163]:
y_pred= lgbm10.predict(x_test_norm)
eval_metrics(y_test,y_pred)

MAE:  11.1826
MSE:  242.8725
RMSE:  15.5844
MAPE:  23.8234
Coefficient of determination (R-squared):  0.0325




(11.182559880707378,
 242.8724780141148,
 15.584366461749891,
 23.82336704806736,
 0.0324763398534713)

### Testing on 2018-2019 data

In [164]:
data_test = data_1819.copy()
data_test = feature_pipeline_1(data_test)
data_test['daily_natural_gas'] = data_1819['daily_natural_gas']

data_test = multi_step(data_test,'elec_prices')
data_test = calc_24lags(data_test,'elec_prices')
data_test = calc_24lags(data_test,'daily_natural_gas')
print(data_test.shape)
print(data_test.columns)
data_test.head()

(10514, 77)
Index(['applicable_date', 'elec_prices', 'month', 'dayofyear_num',
       'weekofyear_num', 'Hour', 'daily_natural_gas', 't+1', 't+2', 't+3',
       't+4', 't+5', 't+6', 't+7', 't+8', 't+9', 't+10', 't+11', 't+12',
       't+13', 't+14', 't+15', 't+16', 't+17', 't+18', 't+19', 't+20', 't+21',
       't+22', 't+23', 'lag_1_elec_prices', 'lag_2_elec_prices',
       'lag_3_elec_prices', 'lag_4_elec_prices', 'lag_5_elec_prices',
       'lag_6_elec_prices', 'lag_7_elec_prices', 'lag_8_elec_prices',
       'lag_9_elec_prices', 'lag_10_elec_prices', 'lag_11_elec_prices',
       'lag_12_elec_prices', 'lag_13_elec_prices', 'lag_14_elec_prices',
       'lag_15_elec_prices', 'lag_16_elec_prices', 'lag_17_elec_prices',
       'lag_18_elec_prices', 'lag_19_elec_prices', 'lag_20_elec_prices',
       'lag_21_elec_prices', 'lag_22_elec_prices', 'lag_23_elec_prices',
       'lag_24_elec_prices', 'lag_1_daily_natural_gas',
       'lag_2_daily_natural_gas', 'lag_3_daily_natural_gas',
       '

,applicable_date,elec_prices,month,dayofyear_num,weekofyear_num,Hour,daily_natural_gas,t+1,t+2,t+3,...,lag_14_daily_natural_gas,lag_15_daily_natural_gas,lag_16_daily_natural_gas,lag_17_daily_natural_gas,lag_18_daily_natural_gas,lag_19_daily_natural_gas,lag_20_daily_natural_gas,lag_21_daily_natural_gas,lag_22_daily_natural_gas,lag_23_daily_natural_gas
0,2018-09-30 22:00:00,71.267,9,273,39,22,2.78616,67.212,60.500,63.682,...,2.78616,2.78616,2.78616,2.78616,2.78616,2.78616,2.78616,2.78616,2.78616,2.78616
1,2018-09-30 23:00:00,67.212,9,273,39,23,2.78616,60.500,63.682,71.617,...,2.78616,2.78616,2.78616,2.78616,2.78616,2.78616,2.78616,2.78616,2.78616,2.78616
2,2018-10-01 00:00:00,60.500,10,274,40,0,2.79509,63.682,71.617,72.855,...,2.78616,2.78616,2.78616,2.78616,2.78616,2.78616,2.78616,2.78616,2.78616,2.78616
3,2018-10-01 01:00:00,63.682,10,274,40,1,2.79509,71.617,72.855,76.937,...,2.78616,2.78616,2.78616,2.78616,2.78616,2.78616,2.78616,2.78616,2.78616,2.78616
4,2018-10-01 02:00:00,71.617,10,274,40,2,2.79509,72.855,76.937,80.480,...,2.78616,2.78616,2.78616,2.78616,2.78616,2.78616,2.78616,2.78616,2.78616,2.78616


In [165]:
#Dropping rows with Nan's
data_test = data_test.dropna(axis=0)
print(data_test.shape)
data_test.isnull().values.any()

(10491, 77)


False

In [166]:
data_test = remove_23rows_hour_col(data_test)
data_test.columns

(438, 76)


Index(['applicable_date', 'elec_prices', 'month', 'dayofyear_num',
       'weekofyear_num', 'daily_natural_gas', 't+1', 't+2', 't+3', 't+4',
       't+5', 't+6', 't+7', 't+8', 't+9', 't+10', 't+11', 't+12', 't+13',
       't+14', 't+15', 't+16', 't+17', 't+18', 't+19', 't+20', 't+21', 't+22',
       't+23', 'lag_1_elec_prices', 'lag_2_elec_prices', 'lag_3_elec_prices',
       'lag_4_elec_prices', 'lag_5_elec_prices', 'lag_6_elec_prices',
       'lag_7_elec_prices', 'lag_8_elec_prices', 'lag_9_elec_prices',
       'lag_10_elec_prices', 'lag_11_elec_prices', 'lag_12_elec_prices',
       'lag_13_elec_prices', 'lag_14_elec_prices', 'lag_15_elec_prices',
       'lag_16_elec_prices', 'lag_17_elec_prices', 'lag_18_elec_prices',
       'lag_19_elec_prices', 'lag_20_elec_prices', 'lag_21_elec_prices',
       'lag_22_elec_prices', 'lag_23_elec_prices', 'lag_24_elec_prices',
       'lag_1_daily_natural_gas', 'lag_2_daily_natural_gas',
       'lag_3_daily_natural_gas', 'lag_4_daily_natural_gas',
 

In [167]:
#selecting columns for X & y
cols = train.columns.to_list()
x_names = cols[2:6]
lags = cols[-47:]
x_names.extend(lags)
y_names = [cols[1]]
time_steps = cols[6:29]
y_names.extend(time_steps)
print(x_names)
print(y_names)

['month', 'dayofyear_num', 'weekofyear_num', 'daily_natural_gas', 'lag_1_elec_prices', 'lag_2_elec_prices', 'lag_3_elec_prices', 'lag_4_elec_prices', 'lag_5_elec_prices', 'lag_6_elec_prices', 'lag_7_elec_prices', 'lag_8_elec_prices', 'lag_9_elec_prices', 'lag_10_elec_prices', 'lag_11_elec_prices', 'lag_12_elec_prices', 'lag_13_elec_prices', 'lag_14_elec_prices', 'lag_15_elec_prices', 'lag_16_elec_prices', 'lag_17_elec_prices', 'lag_18_elec_prices', 'lag_19_elec_prices', 'lag_20_elec_prices', 'lag_21_elec_prices', 'lag_22_elec_prices', 'lag_23_elec_prices', 'lag_24_elec_prices', 'lag_1_daily_natural_gas', 'lag_2_daily_natural_gas', 'lag_3_daily_natural_gas', 'lag_4_daily_natural_gas', 'lag_5_daily_natural_gas', 'lag_6_daily_natural_gas', 'lag_7_daily_natural_gas', 'lag_8_daily_natural_gas', 'lag_9_daily_natural_gas', 'lag_10_daily_natural_gas', 'lag_11_daily_natural_gas', 'lag_12_daily_natural_gas', 'lag_13_daily_natural_gas', 'lag_14_daily_natural_gas', 'lag_15_daily_natural_gas', 'lag

In [168]:
#GBM
mae,mse,rmse,mape,r2 = testing_runs(10,data_test,gbm10)

Round 1: 
(350, 51) (44, 51) (44, 51)
(350, 24) (44, 24) (44, 24)
MAE:  11.1277
MSE:  406.5753
RMSE:  20.1637
MAPE:  18.9424
Coefficient of determination (R-squared):  0.4532


Round 2: 
(350, 51) (44, 51) (44, 51)
(350, 24) (44, 24) (44, 24)
MAE:  10.3944
MSE:  241.6535
RMSE:  15.5452
MAPE:  19.4661
Coefficient of determination (R-squared):  0.3765


Round 3: 
(350, 51) (44, 51) (44, 51)
(350, 24) (44, 24) (44, 24)
MAE:  9.3488
MSE:  178.7765
RMSE:  13.3707
MAPE:  17.174
Coefficient of determination (R-squared):  0.3494


Round 4: 
(350, 51) (44, 51) (44, 51)
(350, 24) (44, 24) (44, 24)
MAE:  10.5414
MSE:  291.0011
RMSE:  17.0588
MAPE:  19.3161
Coefficient of determination (R-squared):  0.4124


Round 5: 
(350, 51) (44, 51) (44, 51)
(350, 24) (44, 24) (44, 24)
MAE:  9.5455
MSE:  241.8239
RMSE:  15.5507
MAPE:  16.52
Coefficient of determination (R-squared):  0.4978


Round 6: 
(350, 51) (44, 51) (44, 51)
(350, 24) (44, 24) (44, 24)
MAE:  11.3851
MSE:  409.7697
RMSE:  20.2428
MAPE:  19.

In [169]:
#XGBM
mae,mse,rmse,mape,r2 = testing_runs(10,data_test,xgb10)

Round 1: 
(350, 51) (44, 51) (44, 51)
(350, 24) (44, 24) (44, 24)
MAE:  11.0018
MSE:  266.0996
RMSE:  16.3126
MAPE:  20.0999
Coefficient of determination (R-squared):  0.4441


Round 2: 
(350, 51) (44, 51) (44, 51)
(350, 24) (44, 24) (44, 24)
MAE:  9.8877
MSE:  197.091
RMSE:  14.0389
MAPE:  19.7017
Coefficient of determination (R-squared):  0.3989


Round 3: 
(350, 51) (44, 51) (44, 51)
(350, 24) (44, 24) (44, 24)
MAE:  10.4011
MSE:  241.363
RMSE:  15.5359
MAPE:  18.9509
Coefficient of determination (R-squared):  0.5071


Round 4: 
(350, 51) (44, 51) (44, 51)
(350, 24) (44, 24) (44, 24)
MAE:  8.3297
MSE:  146.3022
RMSE:  12.0955
MAPE:  16.845
Coefficient of determination (R-squared):  0.4492


Round 5: 
(350, 51) (44, 51) (44, 51)
(350, 24) (44, 24) (44, 24)
MAE:  10.9199
MSE:  310.5456
RMSE:  17.6223
MAPE:  19.721
Coefficient of determination (R-squared):  0.5127


Round 6: 
(350, 51) (44, 51) (44, 51)
(350, 24) (44, 24) (44, 24)
MAE:  10.4583
MSE:  289.9372
RMSE:  17.0275
MAPE:  18.6

In [170]:
#light GBM
mae,mse,rmse,mape,r2 = testing_runs(10,data_test,lgbm10)

Round 1: 
(350, 51) (44, 51) (44, 51)
(350, 24) (44, 24) (44, 24)
MAE:  11.0422
MSE:  287.8989
RMSE:  16.9676
MAPE:  20.3924
Coefficient of determination (R-squared):  0.3828


Round 2: 
(350, 51) (44, 51) (44, 51)
(350, 24) (44, 24) (44, 24)
MAE:  10.3497
MSE:  249.2719
RMSE:  15.7883
MAPE:  18.6035
Coefficient of determination (R-squared):  0.5227


Round 3: 
(350, 51) (44, 51) (44, 51)
(350, 24) (44, 24) (44, 24)
MAE:  11.8864
MSE:  386.9976
RMSE:  19.6723
MAPE:  20.7647
Coefficient of determination (R-squared):  0.4072


Round 4: 
(350, 51) (44, 51) (44, 51)
(350, 24) (44, 24) (44, 24)
MAE:  10.4171
MSE:  301.8269
RMSE:  17.3732
MAPE:  18.4915
Coefficient of determination (R-squared):  0.3236


Round 5: 
(350, 51) (44, 51) (44, 51)
(350, 24) (44, 24) (44, 24)
MAE:  11.7598
MSE:  406.0705
RMSE:  20.1512
MAPE:  21.3458
Coefficient of determination (R-squared):  0.3832


Round 6: 
(350, 51) (44, 51) (44, 51)
(350, 24) (44, 24) (44, 24)
MAE:  10.7327
MSE:  219.0973
RMSE:  14.8019
MAPE:

<h1 style="text-align:center;font-size:40px;" > Basic time based features + ng_log (log of natural gas prices)</h1>

In [219]:
data_1= data.copy()
data_1 = feature_pipeline_1(data_1)
ng = data.daily_natural_gas.values
ng_log = np.log(ng)
data_1['ng_log'] = ng_log
data_1.head()

,applicable_date,elec_prices,month,dayofyear_num,weekofyear_num,Hour,ng_log
0,2019-01-01 00:00:00,70.28,1,1,1,0,0.961834
1,2019-01-01 01:00:00,65.18,1,1,1,1,0.961834
2,2019-01-01 02:00:00,60.91,1,1,1,2,0.961834
3,2019-01-01 03:00:00,65.16,1,1,1,3,0.961834
4,2019-01-01 04:00:00,65.16,1,1,1,4,0.961834


In [220]:
data_1 = multi_step(data_1,'elec_prices')
data_1 = calc_24lags(data_1,'elec_prices')
data_1 = calc_24lags(data_1,'ng_log')
print(data_1.shape)
print(data_1.columns)
data_1.head()

(8304, 77)
Index(['applicable_date', 'elec_prices', 'month', 'dayofyear_num',
       'weekofyear_num', 'Hour', 'ng_log', 't+1', 't+2', 't+3', 't+4', 't+5',
       't+6', 't+7', 't+8', 't+9', 't+10', 't+11', 't+12', 't+13', 't+14',
       't+15', 't+16', 't+17', 't+18', 't+19', 't+20', 't+21', 't+22', 't+23',
       'lag_1_elec_prices', 'lag_2_elec_prices', 'lag_3_elec_prices',
       'lag_4_elec_prices', 'lag_5_elec_prices', 'lag_6_elec_prices',
       'lag_7_elec_prices', 'lag_8_elec_prices', 'lag_9_elec_prices',
       'lag_10_elec_prices', 'lag_11_elec_prices', 'lag_12_elec_prices',
       'lag_13_elec_prices', 'lag_14_elec_prices', 'lag_15_elec_prices',
       'lag_16_elec_prices', 'lag_17_elec_prices', 'lag_18_elec_prices',
       'lag_19_elec_prices', 'lag_20_elec_prices', 'lag_21_elec_prices',
       'lag_22_elec_prices', 'lag_23_elec_prices', 'lag_24_elec_prices',
       'lag_1_ng_log', 'lag_2_ng_log', 'lag_3_ng_log', 'lag_4_ng_log',
       'lag_5_ng_log', 'lag_6_ng_log', 'lag_

,applicable_date,elec_prices,month,dayofyear_num,weekofyear_num,Hour,ng_log,t+1,t+2,t+3,...,lag_14_ng_log,lag_15_ng_log,lag_16_ng_log,lag_17_ng_log,lag_18_ng_log,lag_19_ng_log,lag_20_ng_log,lag_21_ng_log,lag_22_ng_log,lag_23_ng_log
0,2019-01-01 00:00:00,70.28,1,1,1,0,0.961834,65.18,60.91,65.160,...,0.961834,0.961834,0.961834,0.961834,0.961834,0.961834,0.961834,0.961834,0.961834,0.961834
1,2019-01-01 01:00:00,65.18,1,1,1,1,0.961834,60.91,65.16,65.160,...,0.961834,0.961834,0.961834,0.961834,0.961834,0.961834,0.961834,0.961834,0.961834,0.961834
2,2019-01-01 02:00:00,60.91,1,1,1,2,0.961834,65.16,65.16,68.290,...,0.961834,0.961834,0.961834,0.961834,0.961834,0.961834,0.961834,0.961834,0.961834,0.961834
3,2019-01-01 03:00:00,65.16,1,1,1,3,0.961834,65.16,68.29,63.030,...,0.961834,0.961834,0.961834,0.961834,0.961834,0.961834,0.961834,0.961834,0.961834,0.961834
4,2019-01-01 04:00:00,65.16,1,1,1,4,0.961834,68.29,63.03,51.621,...,0.961834,0.961834,0.961834,0.961834,0.961834,0.961834,0.961834,0.961834,0.961834,0.961834


In [221]:
data_1.isnull().values.any()

True

In [222]:
#Dropping rows with Nan's
data_1 = data_1.dropna(axis=0)
print(data_1.shape)
data_1.isnull().values.any()

(8281, 77)


False

In [223]:
data_1 = remove_23rows_hour_col(data_1)
train,test = train_test_split_tbs(data_1)
train.columns

(345, 76)
(276, 76)
(69, 76)


Index(['applicable_date', 'elec_prices', 'month', 'dayofyear_num',
       'weekofyear_num', 'ng_log', 't+1', 't+2', 't+3', 't+4', 't+5', 't+6',
       't+7', 't+8', 't+9', 't+10', 't+11', 't+12', 't+13', 't+14', 't+15',
       't+16', 't+17', 't+18', 't+19', 't+20', 't+21', 't+22', 't+23',
       'lag_1_elec_prices', 'lag_2_elec_prices', 'lag_3_elec_prices',
       'lag_4_elec_prices', 'lag_5_elec_prices', 'lag_6_elec_prices',
       'lag_7_elec_prices', 'lag_8_elec_prices', 'lag_9_elec_prices',
       'lag_10_elec_prices', 'lag_11_elec_prices', 'lag_12_elec_prices',
       'lag_13_elec_prices', 'lag_14_elec_prices', 'lag_15_elec_prices',
       'lag_16_elec_prices', 'lag_17_elec_prices', 'lag_18_elec_prices',
       'lag_19_elec_prices', 'lag_20_elec_prices', 'lag_21_elec_prices',
       'lag_22_elec_prices', 'lag_23_elec_prices', 'lag_24_elec_prices',
       'lag_1_ng_log', 'lag_2_ng_log', 'lag_3_ng_log', 'lag_4_ng_log',
       'lag_5_ng_log', 'lag_6_ng_log', 'lag_7_ng_log', 'lag_8_n

In [224]:
#selecting columns for X & y
cols = train.columns.to_list()
x_names = cols[2:6]
lags = cols[-47:]
x_names.extend(lags)
y_names = [cols[1]]
time_steps = cols[6:29]
y_names.extend(time_steps)
print(x_names)
print(y_names)

['month', 'dayofyear_num', 'weekofyear_num', 'ng_log', 'lag_1_elec_prices', 'lag_2_elec_prices', 'lag_3_elec_prices', 'lag_4_elec_prices', 'lag_5_elec_prices', 'lag_6_elec_prices', 'lag_7_elec_prices', 'lag_8_elec_prices', 'lag_9_elec_prices', 'lag_10_elec_prices', 'lag_11_elec_prices', 'lag_12_elec_prices', 'lag_13_elec_prices', 'lag_14_elec_prices', 'lag_15_elec_prices', 'lag_16_elec_prices', 'lag_17_elec_prices', 'lag_18_elec_prices', 'lag_19_elec_prices', 'lag_20_elec_prices', 'lag_21_elec_prices', 'lag_22_elec_prices', 'lag_23_elec_prices', 'lag_24_elec_prices', 'lag_1_ng_log', 'lag_2_ng_log', 'lag_3_ng_log', 'lag_4_ng_log', 'lag_5_ng_log', 'lag_6_ng_log', 'lag_7_ng_log', 'lag_8_ng_log', 'lag_9_ng_log', 'lag_10_ng_log', 'lag_11_ng_log', 'lag_12_ng_log', 'lag_13_ng_log', 'lag_14_ng_log', 'lag_15_ng_log', 'lag_16_ng_log', 'lag_17_ng_log', 'lag_18_ng_log', 'lag_19_ng_log', 'lag_20_ng_log', 'lag_21_ng_log', 'lag_22_ng_log', 'lag_23_ng_log']
['elec_prices', 't+1', 't+2', 't+3', 't+4', 

In [225]:
x_train, y_train = train[x_names], train[y_names]
x_test, y_test = test[x_names], test[y_names]
print(x_train.shape,y_train.shape)
print(x_test.shape,y_test.shape)

(276, 51) (276, 24)
(69, 51) (69, 24)


In [226]:
x_train_norm,x_test_norm = normalize(x_train,x_test)

In [227]:
GBM_training(x_train_norm,y_train)

Duration: 0:00:15.659552



{'mean_fit_time': array([2.14366505, 1.68754616, 1.71187828, 0.59595168, 0.93727164,
       0.50242431, 1.27971473, 0.72133436, 0.86570554, 1.24830515]), 'std_fit_time': array([0.16808762, 0.09876441, 0.12956475, 0.07912449, 0.04247633,
       0.02019175, 0.04623868, 0.0474088 , 0.10010343, 0.14450858]), 'mean_score_time': array([0.02756901, 0.02625201, 0.02211232, 0.02307408, 0.02245715,
       0.02163546, 0.02043703, 0.02576969, 0.02995951, 0.0103744 ]), 'std_score_time': array([0.00472852, 0.00570618, 0.00320827, 0.00364258, 0.00343479,
       0.00340526, 0.0027301 , 0.00547477, 0.0248083 , 0.00470644]), 'param_estimator__subsample': masked_array(data=[0.8, 0.6, 0.5, 0.8, 0.8, 0.6, 0.7, 0.75, 0.7, 0.85],
             mask=[False, False, False, False, False, False, False, False,
                   False, False],
       fill_value='?',
            dtype=object), 'param_estimator__n_estimators': masked_array(data=[80, 70, 60, 20, 40, 20, 60, 30, 30, 70],
   

In [228]:
#learning_rate,n_estimators,max_depth,min_samples_split,max_features,min_samples_leaf,subsample
gbm11 = GBM_model(x_train_norm,y_train,0.1,60,9,100,5,30,0.5)

Duration: 0:00:01.336247


In [229]:
y_pred= gbm11.predict(x_test_norm)
eval_metrics(y_test,y_pred)

MAE:  11.1035
MSE:  247.9246
RMSE:  15.7456
MAPE:  23.655
Coefficient of determination (R-squared):  0.0119




(11.103529402982367,
 247.92455031284734,
 15.74562003583369,
 23.65500022518316,
 0.01185747146317051)

In [230]:
XGBM_training(x_train_norm,y_train)

Duration: 0:01:51.242240



{'mean_fit_time': array([38.80601847,  3.04025838,  9.53548224, 14.31252859, 12.36007867,
       13.34759164,  4.01917565,  5.69254186, 18.78288817,  2.83939476]), 'std_fit_time': array([2.32989639, 0.12892314, 0.63367327, 0.76233518, 0.64661462,
       0.53635185, 0.224566  , 0.19549564, 0.82674689, 0.49962303]), 'mean_score_time': array([0.11764727, 0.05079236, 0.05785904, 0.06730225, 0.06765058,
       0.07316015, 0.05512569, 0.05421138, 0.04722731, 0.04763052]), 'std_score_time': array([0.02826311, 0.01496515, 0.0029874 , 0.01115182, 0.00256341,
       0.00492613, 0.0030028 , 0.00991322, 0.01589737, 0.01850351]), 'param_estimator__subsample': masked_array(data=[0.5, 0.4, 0.30000000000000004, 0.9, 0.2,
                   0.7000000000000001, 0.2, 0.2, 0.30000000000000004, 0.4],
             mask=[False, False, False, False, False, False, False, False,
                   False, False],
       fill_value='?',
            dtype=object), 'param_estimator__n_es

In [231]:
#learning_rate,n_estimators,max_depth,subsample,colsample_bytree
xgb11 = XGBM_model(x_train_norm,y_train,0.025,250,5,0.3,0.6)

Duration: 0:00:02.842989


In [232]:
y_pred= xgb11.predict(x_test_norm)
eval_metrics(y_test,y_pred)

MAE:  11.1067
MSE:  241.2728
RMSE:  15.533
MAPE:  23.6617
Coefficient of determination (R-squared):  0.0611




(11.106661811439887,
 241.27276867128728,
 15.532957499178554,
 23.661673519779754,
 0.061065156052659914)

In [233]:
LGBM_training(x_train_norm,y_train)

[LightGBM] [Warning] min_child_samples is set=20, min_child_samples= will be ignored. Current value: min_child_samples=20
[LightGBM] [Warning] Unknown parameter: 40
Duration: 0:01:09.429492



{'mean_fit_time': array([21.9083158 ,  8.59608071,  5.3775454 ,  1.19510744,  4.07102084,
       14.21815295,  7.82015955,  6.11245623,  2.87805676,  4.22988038]), 'std_fit_time': array([0.89665448, 0.35826665, 0.30093138, 0.12201089, 0.17977363,
       0.95891159, 1.02584433, 0.25959098, 0.26517097, 0.33800623]), 'mean_score_time': array([0.10215104, 0.06243453, 0.07879841, 0.0372478 , 0.04971769,
       0.02852328, 0.08188219, 0.0481847 , 0.03452375, 0.02530339]), 'std_score_time': array([0.02335602, 0.01038466, 0.01602257, 0.04095964, 0.01135308,
       0.00578652, 0.03311475, 0.00835744, 0.02081487, 0.00674884]), 'param_estimator__subsample': masked_array(data=[0.9, 0.9, 0.30000000000000004, 0.8, 0.9, 0.6,
                   0.7000000000000001, 0.6, 0.9, 0.4],
             mask=[False, False,

In [234]:
#num_leaves,min_child_samples,learning_rate,n_estimators,max_depth,subsample,colsample_bytree
lgbm11 = LGBM_model(x_train_norm,y_train,307,40,0.1,50,3,0.8,0.8)

Duration: 0:00:00.758459


In [235]:
y_pred= lgbm11.predict(x_test_norm)
eval_metrics(y_test,y_pred)

MAE:  11.0734
MSE:  236.2628
RMSE:  15.3708
MAPE:  23.5907
Coefficient of determination (R-squared):  0.0548




(11.073361264879352,
 236.26281285964043,
 15.370842945643561,
 23.59072991186922,
 0.05477176357075327)

### Testing on 2018-2019 data

In [236]:
data_test = data_1819.copy()
data_test = feature_pipeline_1(data_test)
ng = data_1819.daily_natural_gas.values
ng_log = np.log(ng)
data_test['ng_log'] = ng_log

data_test = multi_step(data_test,'elec_prices')
data_test = calc_24lags(data_test,'elec_prices')
data_test = calc_24lags(data_test,'ng_log')
print(data_test.shape)
print(data_test.columns)
data_test.head()

(10514, 77)
Index(['applicable_date', 'elec_prices', 'month', 'dayofyear_num',
       'weekofyear_num', 'Hour', 'ng_log', 't+1', 't+2', 't+3', 't+4', 't+5',
       't+6', 't+7', 't+8', 't+9', 't+10', 't+11', 't+12', 't+13', 't+14',
       't+15', 't+16', 't+17', 't+18', 't+19', 't+20', 't+21', 't+22', 't+23',
       'lag_1_elec_prices', 'lag_2_elec_prices', 'lag_3_elec_prices',
       'lag_4_elec_prices', 'lag_5_elec_prices', 'lag_6_elec_prices',
       'lag_7_elec_prices', 'lag_8_elec_prices', 'lag_9_elec_prices',
       'lag_10_elec_prices', 'lag_11_elec_prices', 'lag_12_elec_prices',
       'lag_13_elec_prices', 'lag_14_elec_prices', 'lag_15_elec_prices',
       'lag_16_elec_prices', 'lag_17_elec_prices', 'lag_18_elec_prices',
       'lag_19_elec_prices', 'lag_20_elec_prices', 'lag_21_elec_prices',
       'lag_22_elec_prices', 'lag_23_elec_prices', 'lag_24_elec_prices',
       'lag_1_ng_log', 'lag_2_ng_log', 'lag_3_ng_log', 'lag_4_ng_log',
       'lag_5_ng_log', 'lag_6_ng_log', 'lag

,applicable_date,elec_prices,month,dayofyear_num,weekofyear_num,Hour,ng_log,t+1,t+2,t+3,...,lag_14_ng_log,lag_15_ng_log,lag_16_ng_log,lag_17_ng_log,lag_18_ng_log,lag_19_ng_log,lag_20_ng_log,lag_21_ng_log,lag_22_ng_log,lag_23_ng_log
0,2018-09-30 22:00:00,71.267,9,273,39,22,1.024664,67.212,60.500,63.682,...,1.024664,1.024664,1.024664,1.024664,1.024664,1.024664,1.024664,1.024664,1.024664,1.024664
1,2018-09-30 23:00:00,67.212,9,273,39,23,1.024664,60.500,63.682,71.617,...,1.024664,1.024664,1.024664,1.024664,1.024664,1.024664,1.024664,1.024664,1.024664,1.024664
2,2018-10-01 00:00:00,60.500,10,274,40,0,1.027864,63.682,71.617,72.855,...,1.024664,1.024664,1.024664,1.024664,1.024664,1.024664,1.024664,1.024664,1.024664,1.024664
3,2018-10-01 01:00:00,63.682,10,274,40,1,1.027864,71.617,72.855,76.937,...,1.024664,1.024664,1.024664,1.024664,1.024664,1.024664,1.024664,1.024664,1.024664,1.024664
4,2018-10-01 02:00:00,71.617,10,274,40,2,1.027864,72.855,76.937,80.480,...,1.024664,1.024664,1.024664,1.024664,1.024664,1.024664,1.024664,1.024664,1.024664,1.024664


In [237]:
#Dropping rows with Nan's
data_test = data_test.dropna(axis=0)
print(data_test.shape)
data_test.isnull().values.any()

(10491, 77)


False

In [238]:
data_test = remove_23rows_hour_col(data_test)
data_test.columns

(438, 76)


Index(['applicable_date', 'elec_prices', 'month', 'dayofyear_num',
       'weekofyear_num', 'ng_log', 't+1', 't+2', 't+3', 't+4', 't+5', 't+6',
       't+7', 't+8', 't+9', 't+10', 't+11', 't+12', 't+13', 't+14', 't+15',
       't+16', 't+17', 't+18', 't+19', 't+20', 't+21', 't+22', 't+23',
       'lag_1_elec_prices', 'lag_2_elec_prices', 'lag_3_elec_prices',
       'lag_4_elec_prices', 'lag_5_elec_prices', 'lag_6_elec_prices',
       'lag_7_elec_prices', 'lag_8_elec_prices', 'lag_9_elec_prices',
       'lag_10_elec_prices', 'lag_11_elec_prices', 'lag_12_elec_prices',
       'lag_13_elec_prices', 'lag_14_elec_prices', 'lag_15_elec_prices',
       'lag_16_elec_prices', 'lag_17_elec_prices', 'lag_18_elec_prices',
       'lag_19_elec_prices', 'lag_20_elec_prices', 'lag_21_elec_prices',
       'lag_22_elec_prices', 'lag_23_elec_prices', 'lag_24_elec_prices',
       'lag_1_ng_log', 'lag_2_ng_log', 'lag_3_ng_log', 'lag_4_ng_log',
       'lag_5_ng_log', 'lag_6_ng_log', 'lag_7_ng_log', 'lag_8_n

In [239]:
#selecting columns for X & y
cols = train.columns.to_list()
x_names = cols[2:6]
lags = cols[-47:]
x_names.extend(lags)
y_names = [cols[1]]
time_steps = cols[6:29]
y_names.extend(time_steps)
print(x_names)
print(y_names)

['month', 'dayofyear_num', 'weekofyear_num', 'ng_log', 'lag_1_elec_prices', 'lag_2_elec_prices', 'lag_3_elec_prices', 'lag_4_elec_prices', 'lag_5_elec_prices', 'lag_6_elec_prices', 'lag_7_elec_prices', 'lag_8_elec_prices', 'lag_9_elec_prices', 'lag_10_elec_prices', 'lag_11_elec_prices', 'lag_12_elec_prices', 'lag_13_elec_prices', 'lag_14_elec_prices', 'lag_15_elec_prices', 'lag_16_elec_prices', 'lag_17_elec_prices', 'lag_18_elec_prices', 'lag_19_elec_prices', 'lag_20_elec_prices', 'lag_21_elec_prices', 'lag_22_elec_prices', 'lag_23_elec_prices', 'lag_24_elec_prices', 'lag_1_ng_log', 'lag_2_ng_log', 'lag_3_ng_log', 'lag_4_ng_log', 'lag_5_ng_log', 'lag_6_ng_log', 'lag_7_ng_log', 'lag_8_ng_log', 'lag_9_ng_log', 'lag_10_ng_log', 'lag_11_ng_log', 'lag_12_ng_log', 'lag_13_ng_log', 'lag_14_ng_log', 'lag_15_ng_log', 'lag_16_ng_log', 'lag_17_ng_log', 'lag_18_ng_log', 'lag_19_ng_log', 'lag_20_ng_log', 'lag_21_ng_log', 'lag_22_ng_log', 'lag_23_ng_log']
['elec_prices', 't+1', 't+2', 't+3', 't+4', 

In [240]:
#GBM
mae,mse,rmse,mape,r2 = testing_runs(10,data_test,gbm11)

Round 1: 
(350, 51) (44, 51) (44, 51)
(350, 24) (44, 24) (44, 24)
MAE:  10.5599
MSE:  299.5277
RMSE:  17.3069
MAPE:  18.7183
Coefficient of determination (R-squared):  0.4958


Round 2: 
(350, 51) (44, 51) (44, 51)
(350, 24) (44, 24) (44, 24)
MAE:  11.531
MSE:  356.0132
RMSE:  18.8683
MAPE:  20.2498
Coefficient of determination (R-squared):  0.4565


Round 3: 
(350, 51) (44, 51) (44, 51)
(350, 24) (44, 24) (44, 24)
MAE:  11.5218
MSE:  333.341
RMSE:  18.2576
MAPE:  20.1757
Coefficient of determination (R-squared):  0.4218


Round 4: 
(350, 51) (44, 51) (44, 51)
(350, 24) (44, 24) (44, 24)
MAE:  10.7383
MSE:  323.3112
RMSE:  17.9809
MAPE:  18.8413
Coefficient of determination (R-squared):  0.4075


Round 5: 
(350, 51) (44, 51) (44, 51)
(350, 24) (44, 24) (44, 24)
MAE:  10.796
MSE:  277.2297
RMSE:  16.6502
MAPE:  19.2847
Coefficient of determination (R-squared):  0.3637


Round 6: 
(350, 51) (44, 51) (44, 51)
(350, 24) (44, 24) (44, 24)
MAE:  11.0016
MSE:  283.8815
RMSE:  16.8488
MAPE:  1

In [241]:
#XGBM
mae,mse,rmse,mape,r2 = testing_runs(10,data_test,xgb11)

Round 1: 
(350, 51) (44, 51) (44, 51)
(350, 24) (44, 24) (44, 24)
MAE:  10.8122
MSE:  292.3827
RMSE:  17.0992
MAPE:  19.3067
Coefficient of determination (R-squared):  0.5157


Round 2: 
(350, 51) (44, 51) (44, 51)
(350, 24) (44, 24) (44, 24)
MAE:  11.486
MSE:  300.1518
RMSE:  17.3249
MAPE:  18.7422
Coefficient of determination (R-squared):  0.4082


Round 3: 
(350, 51) (44, 51) (44, 51)
(350, 24) (44, 24) (44, 24)
MAE:  10.8731
MSE:  305.2078
RMSE:  17.4702
MAPE:  19.1986
Coefficient of determination (R-squared):  0.4051


Round 4: 
(350, 51) (44, 51) (44, 51)
(350, 24) (44, 24) (44, 24)
MAE:  10.0353
MSE:  216.868
RMSE:  14.7264
MAPE:  17.8717
Coefficient of determination (R-squared):  0.4272


Round 5: 
(350, 51) (44, 51) (44, 51)
(350, 24) (44, 24) (44, 24)
MAE:  10.4261
MSE:  325.9438
RMSE:  18.0539
MAPE:  19.8506
Coefficient of determination (R-squared):  0.4126


Round 6: 
(350, 51) (44, 51) (44, 51)
(350, 24) (44, 24) (44, 24)
MAE:  9.5934
MSE:  180.8762
RMSE:  13.449
MAPE:  17

In [242]:
#light GBM
mae,mse,rmse,mape,r2 = testing_runs(10,data_test,lgbm11)

Round 1: 
(350, 51) (44, 51) (44, 51)
(350, 24) (44, 24) (44, 24)
MAE:  11.8461
MSE:  369.9126
RMSE:  19.2331
MAPE:  20.6377
Coefficient of determination (R-squared):  0.2451


Round 2: 
(350, 51) (44, 51) (44, 51)
(350, 24) (44, 24) (44, 24)
MAE:  12.2974
MSE:  433.0324
RMSE:  20.8094
MAPE:  21.8984
Coefficient of determination (R-squared):  0.4558


Round 3: 
(350, 51) (44, 51) (44, 51)
(350, 24) (44, 24) (44, 24)
MAE:  10.8686
MSE:  251.0099
RMSE:  15.8433
MAPE:  19.6457
Coefficient of determination (R-squared):  0.3986


Round 4: 
(350, 51) (44, 51) (44, 51)
(350, 24) (44, 24) (44, 24)
MAE:  11.1351
MSE:  293.2718
RMSE:  17.1252
MAPE:  20.7436
Coefficient of determination (R-squared):  0.4573


Round 5: 
(350, 51) (44, 51) (44, 51)
(350, 24) (44, 24) (44, 24)
MAE:  12.8411
MSE:  507.0659
RMSE:  22.5181
MAPE:  21.7011
Coefficient of determination (R-squared):  0.4181


Round 6: 
(350, 51) (44, 51) (44, 51)
(350, 24) (44, 24) (44, 24)
MAE:  11.1172
MSE:  308.0861
RMSE:  17.5524
MAPE:

<h1 style="text-align:center;font-size:40px;" > Basic time based features +  ng_sqr (square of natural gas prices)</h1>

In [195]:
data_1= data.copy()
data_1 = feature_pipeline_1(data_1)
ng = data.daily_natural_gas.values
ng_sqr = ng*ng
data_1['ng_sqr'] = ng_sqr
data_1.head()

,applicable_date,elec_prices,month,dayofyear_num,weekofyear_num,Hour,ng_sqr
0,2019-01-01 00:00:00,70.28,1,1,1,0,6.84602
1,2019-01-01 01:00:00,65.18,1,1,1,1,6.84602
2,2019-01-01 02:00:00,60.91,1,1,1,2,6.84602
3,2019-01-01 03:00:00,65.16,1,1,1,3,6.84602
4,2019-01-01 04:00:00,65.16,1,1,1,4,6.84602


In [196]:
data_1 = multi_step(data_1,'elec_prices')
data_1 = calc_24lags(data_1,'elec_prices')
data_1 = calc_24lags(data_1,'ng_sqr')
print(data_1.shape)
print(data_1.columns)
data_1.head()

(8304, 77)
Index(['applicable_date', 'elec_prices', 'month', 'dayofyear_num',
       'weekofyear_num', 'Hour', 'ng_sqr', 't+1', 't+2', 't+3', 't+4', 't+5',
       't+6', 't+7', 't+8', 't+9', 't+10', 't+11', 't+12', 't+13', 't+14',
       't+15', 't+16', 't+17', 't+18', 't+19', 't+20', 't+21', 't+22', 't+23',
       'lag_1_elec_prices', 'lag_2_elec_prices', 'lag_3_elec_prices',
       'lag_4_elec_prices', 'lag_5_elec_prices', 'lag_6_elec_prices',
       'lag_7_elec_prices', 'lag_8_elec_prices', 'lag_9_elec_prices',
       'lag_10_elec_prices', 'lag_11_elec_prices', 'lag_12_elec_prices',
       'lag_13_elec_prices', 'lag_14_elec_prices', 'lag_15_elec_prices',
       'lag_16_elec_prices', 'lag_17_elec_prices', 'lag_18_elec_prices',
       'lag_19_elec_prices', 'lag_20_elec_prices', 'lag_21_elec_prices',
       'lag_22_elec_prices', 'lag_23_elec_prices', 'lag_24_elec_prices',
       'lag_1_ng_sqr', 'lag_2_ng_sqr', 'lag_3_ng_sqr', 'lag_4_ng_sqr',
       'lag_5_ng_sqr', 'lag_6_ng_sqr', 'lag_

,applicable_date,elec_prices,month,dayofyear_num,weekofyear_num,Hour,ng_sqr,t+1,t+2,t+3,...,lag_14_ng_sqr,lag_15_ng_sqr,lag_16_ng_sqr,lag_17_ng_sqr,lag_18_ng_sqr,lag_19_ng_sqr,lag_20_ng_sqr,lag_21_ng_sqr,lag_22_ng_sqr,lag_23_ng_sqr
0,2019-01-01 00:00:00,70.28,1,1,1,0,6.84602,65.18,60.91,65.160,...,6.84602,6.84602,6.84602,6.84602,6.84602,6.84602,6.84602,6.84602,6.84602,6.84602
1,2019-01-01 01:00:00,65.18,1,1,1,1,6.84602,60.91,65.16,65.160,...,6.84602,6.84602,6.84602,6.84602,6.84602,6.84602,6.84602,6.84602,6.84602,6.84602
2,2019-01-01 02:00:00,60.91,1,1,1,2,6.84602,65.16,65.16,68.290,...,6.84602,6.84602,6.84602,6.84602,6.84602,6.84602,6.84602,6.84602,6.84602,6.84602
3,2019-01-01 03:00:00,65.16,1,1,1,3,6.84602,65.16,68.29,63.030,...,6.84602,6.84602,6.84602,6.84602,6.84602,6.84602,6.84602,6.84602,6.84602,6.84602
4,2019-01-01 04:00:00,65.16,1,1,1,4,6.84602,68.29,63.03,51.621,...,6.84602,6.84602,6.84602,6.84602,6.84602,6.84602,6.84602,6.84602,6.84602,6.84602


In [197]:
data_1.isnull().values.any()

True

In [198]:
#Dropping rows with Nan's
data_1 = data_1.dropna(axis=0)
print(data_1.shape)
data_1.isnull().values.any()

(8281, 77)


False

In [199]:
data_1 = remove_23rows_hour_col(data_1)
train,test = train_test_split_tbs(data_1)
train.columns

(345, 76)
(276, 76)
(69, 76)


Index(['applicable_date', 'elec_prices', 'month', 'dayofyear_num',
       'weekofyear_num', 'ng_sqr', 't+1', 't+2', 't+3', 't+4', 't+5', 't+6',
       't+7', 't+8', 't+9', 't+10', 't+11', 't+12', 't+13', 't+14', 't+15',
       't+16', 't+17', 't+18', 't+19', 't+20', 't+21', 't+22', 't+23',
       'lag_1_elec_prices', 'lag_2_elec_prices', 'lag_3_elec_prices',
       'lag_4_elec_prices', 'lag_5_elec_prices', 'lag_6_elec_prices',
       'lag_7_elec_prices', 'lag_8_elec_prices', 'lag_9_elec_prices',
       'lag_10_elec_prices', 'lag_11_elec_prices', 'lag_12_elec_prices',
       'lag_13_elec_prices', 'lag_14_elec_prices', 'lag_15_elec_prices',
       'lag_16_elec_prices', 'lag_17_elec_prices', 'lag_18_elec_prices',
       'lag_19_elec_prices', 'lag_20_elec_prices', 'lag_21_elec_prices',
       'lag_22_elec_prices', 'lag_23_elec_prices', 'lag_24_elec_prices',
       'lag_1_ng_sqr', 'lag_2_ng_sqr', 'lag_3_ng_sqr', 'lag_4_ng_sqr',
       'lag_5_ng_sqr', 'lag_6_ng_sqr', 'lag_7_ng_sqr', 'lag_8_n

In [200]:
#selecting columns for X & y
cols = train.columns.to_list()
x_names = cols[2:6]
lags = cols[-47:]
x_names.extend(lags)
y_names = [cols[1]]
time_steps = cols[6:29]
y_names.extend(time_steps)
print(x_names)
print(y_names)

['month', 'dayofyear_num', 'weekofyear_num', 'ng_sqr', 'lag_1_elec_prices', 'lag_2_elec_prices', 'lag_3_elec_prices', 'lag_4_elec_prices', 'lag_5_elec_prices', 'lag_6_elec_prices', 'lag_7_elec_prices', 'lag_8_elec_prices', 'lag_9_elec_prices', 'lag_10_elec_prices', 'lag_11_elec_prices', 'lag_12_elec_prices', 'lag_13_elec_prices', 'lag_14_elec_prices', 'lag_15_elec_prices', 'lag_16_elec_prices', 'lag_17_elec_prices', 'lag_18_elec_prices', 'lag_19_elec_prices', 'lag_20_elec_prices', 'lag_21_elec_prices', 'lag_22_elec_prices', 'lag_23_elec_prices', 'lag_24_elec_prices', 'lag_1_ng_sqr', 'lag_2_ng_sqr', 'lag_3_ng_sqr', 'lag_4_ng_sqr', 'lag_5_ng_sqr', 'lag_6_ng_sqr', 'lag_7_ng_sqr', 'lag_8_ng_sqr', 'lag_9_ng_sqr', 'lag_10_ng_sqr', 'lag_11_ng_sqr', 'lag_12_ng_sqr', 'lag_13_ng_sqr', 'lag_14_ng_sqr', 'lag_15_ng_sqr', 'lag_16_ng_sqr', 'lag_17_ng_sqr', 'lag_18_ng_sqr', 'lag_19_ng_sqr', 'lag_20_ng_sqr', 'lag_21_ng_sqr', 'lag_22_ng_sqr', 'lag_23_ng_sqr']
['elec_prices', 't+1', 't+2', 't+3', 't+4', 

In [201]:
x_train, y_train = train[x_names], train[y_names]
x_test, y_test = test[x_names], test[y_names]
print(x_train.shape,y_train.shape)
print(x_test.shape,y_test.shape)

(276, 51) (276, 24)
(69, 51) (69, 24)


In [202]:
x_train_norm,x_test_norm = normalize(x_train,x_test)

In [203]:
GBM_training(x_train_norm,y_train)

Duration: 0:00:18.487829



{'mean_fit_time': array([0.77165861, 1.70039682, 2.24078908, 0.62756102, 1.0601007 ,
       1.05872023, 0.8435302 , 2.81461387, 2.16619589, 1.54814293]), 'std_fit_time': array([0.06373897, 0.10027402, 0.08443415, 0.04947511, 0.02325117,
       0.0358581 , 0.09961871, 0.3334121 , 0.14414143, 0.39797036]), 'mean_score_time': array([0.02187743, 0.02343826, 0.02319119, 0.02352006, 0.02098405,
       0.02144623, 0.02334166, 0.02893953, 0.02190487, 0.01379175]), 'std_score_time': array([0.00258669, 0.00204542, 0.00302263, 0.00609491, 0.00170594,
       0.00280672, 0.00529149, 0.01006973, 0.00387245, 0.00733473]), 'param_estimator__subsample': masked_array(data=[0.9, 0.5, 0.6, 0.8, 0.9, 0.9, 0.9, 0.6, 0.6, 0.9],
             mask=[False, False, False, False, False, False, False, False,
                   False, False],
       fill_value='?',
            dtype=object), 'param_estimator__n_estimators': masked_array(data=[30, 70, 100, 20, 50, 40, 30, 100, 100, 90],
  

In [204]:
#learning_rate,n_estimators,max_depth,min_samples_split,max_features,min_samples_leaf,subsample
gbm12 =  GBM_model(x_train_norm,y_train,0.1,40,9,200,3,10,0.9)

Duration: 0:00:00.974855


In [205]:

y_pred= gbm12.predict(x_test_norm)
eval_metrics(y_test,y_pred)

MAE:  10.8122
MSE:  245.5072
RMSE:  15.6687
MAPE:  23.0343
Coefficient of determination (R-squared):  0.0634




(10.812172322191522,
 245.50720390284334,
 15.668669500083386,
 23.03429201956837,
 0.0634013370062937)

In [206]:
XGBM_training(x_train_norm,y_train)

Duration: 0:01:46.413776



{'mean_fit_time': array([ 8.75407124, 10.04816358,  2.60156271,  8.58075967,  3.53455274,
        3.96417115, 22.29762774, 31.37906446,  3.25258911, 18.2749649 ]), 'std_fit_time': array([0.25675499, 1.83561508, 0.23496753, 0.56456922, 0.33574358,
       0.33285949, 1.77440971, 1.32705165, 0.26611667, 3.1443827 ]), 'mean_score_time': array([0.0794641 , 0.09268155, 0.05991657, 0.06648302, 0.0553566 ,
       0.06124682, 0.06998096, 0.11636391, 0.04503181, 0.04384615]), 'std_score_time': array([0.01450707, 0.01417586, 0.01324536, 0.00683414, 0.00890787,
       0.00974456, 0.01342981, 0.01547443, 0.00430202, 0.0089586 ]), 'param_estimator__subsample': masked_array(data=[0.30000000000000004, 0.30000000000000004, 0.1,
                   0.30000000000000004, 0.1, 0.1, 0.5, 0.6, 0.2, 0.9],
             mask=[False, False, False, False, False, False, False, False,
                   False, False],
       fill_value='?',
            dtype=object), 'param_estimator__n_e

In [207]:
#learning_rate,n_estimators,max_depth,subsample,colsample_bytree
xgb12 = XGBM_model(x_train_norm,y_train,0.05,100,5,0.2,0.6)

Duration: 0:00:01.517330


In [208]:
y_pred= xgb12.predict(x_test_norm)
eval_metrics(y_test,y_pred)

MAE:  11.5299
MSE:  264.4787
RMSE:  16.2628
MAPE:  24.5632
Coefficient of determination (R-squared):  0.0115




(11.529853374324702,
 264.47874581384093,
 16.262802520286623,
 24.563242395046302,
 0.01152656349900343)

In [209]:
LGBM_training(x_train_norm,y_train)

[LightGBM] [Warning] min_child_samples is set=20, min_child_samples= will be ignored. Current value: min_child_samples=20
[LightGBM] [Warning] Unknown parameter: 10
Duration: 0:01:11.643814



{'mean_fit_time': array([10.66818986,  1.21157141,  5.49176745,  7.45422475, 14.07925372,
        4.88064485, 11.2768636 ,  9.21561198, 10.83865325,  2.54762526]), 'std_fit_time': array([1.22037414, 0.26036815, 0.22182586, 0.40232214, 0.77148158,
       0.36415533, 0.63906074, 0.57258442, 0.70487194, 0.22935232]), 'mean_score_time': array([0.06545298, 0.03137856, 0.03506205, 0.0602483 , 0.09102509,
       0.06210546, 0.07515626, 0.09511926, 0.0477524 , 0.04165056]), 'std_score_time': array([0.01074031, 0.02419738, 0.00421107, 0.0072047 , 0.0217734 ,
       0.01018126, 0.01114273, 0.02929279, 0.01073439, 0.01845181]), 'param_estimator__subsample': masked_array(data=[0.9, 0.1, 0.8, 0.1, 0.2, 0.6, 1.0, 0.7000000000000001,
                   0.7000000000000001, 0.2],
             mask=[False, False, 

In [210]:
#num_leaves,min_child_samples,learning_rate,n_estimators,max_depth,subsample,colsample_bytree
lgbm12 = LGBM_model(x_train_norm,y_train,1229,10,0.05,50,3,0.1,1.0)

Duration: 0:00:00.903894


In [211]:
y_pred= lgbm12.predict(x_test_norm)
eval_metrics(y_test,y_pred)

MAE:  10.5045
MSE:  230.5284
RMSE:  15.1832
MAPE:  22.3789
Coefficient of determination (R-squared):  0.1157




(10.50454986433286,
 230.52841691419258,
 15.183162283075044,
 22.378931994316986,
 0.11569605694885281)

### Testing on 2018-2019 data

In [212]:
data_test = data_1819.copy()
data_test = feature_pipeline_1(data_test)
ng = data_1819.daily_natural_gas.values
ng_sqr = ng*ng
data_test['ng_sqr'] = ng_sqr

data_test = multi_step(data_test,'elec_prices')
data_test = calc_24lags(data_test,'elec_prices')
data_test = calc_24lags(data_test,'ng_sqr')
print(data_test.shape)
print(data_test.columns)
data_test.head()

(10514, 77)
Index(['applicable_date', 'elec_prices', 'month', 'dayofyear_num',
       'weekofyear_num', 'Hour', 'ng_sqr', 't+1', 't+2', 't+3', 't+4', 't+5',
       't+6', 't+7', 't+8', 't+9', 't+10', 't+11', 't+12', 't+13', 't+14',
       't+15', 't+16', 't+17', 't+18', 't+19', 't+20', 't+21', 't+22', 't+23',
       'lag_1_elec_prices', 'lag_2_elec_prices', 'lag_3_elec_prices',
       'lag_4_elec_prices', 'lag_5_elec_prices', 'lag_6_elec_prices',
       'lag_7_elec_prices', 'lag_8_elec_prices', 'lag_9_elec_prices',
       'lag_10_elec_prices', 'lag_11_elec_prices', 'lag_12_elec_prices',
       'lag_13_elec_prices', 'lag_14_elec_prices', 'lag_15_elec_prices',
       'lag_16_elec_prices', 'lag_17_elec_prices', 'lag_18_elec_prices',
       'lag_19_elec_prices', 'lag_20_elec_prices', 'lag_21_elec_prices',
       'lag_22_elec_prices', 'lag_23_elec_prices', 'lag_24_elec_prices',
       'lag_1_ng_sqr', 'lag_2_ng_sqr', 'lag_3_ng_sqr', 'lag_4_ng_sqr',
       'lag_5_ng_sqr', 'lag_6_ng_sqr', 'lag

,applicable_date,elec_prices,month,dayofyear_num,weekofyear_num,Hour,ng_sqr,t+1,t+2,t+3,...,lag_14_ng_sqr,lag_15_ng_sqr,lag_16_ng_sqr,lag_17_ng_sqr,lag_18_ng_sqr,lag_19_ng_sqr,lag_20_ng_sqr,lag_21_ng_sqr,lag_22_ng_sqr,lag_23_ng_sqr
0,2018-09-30 22:00:00,71.267,9,273,39,22,7.762688,67.212,60.500,63.682,...,7.762688,7.762688,7.762688,7.762688,7.762688,7.762688,7.762688,7.762688,7.762688,7.762688
1,2018-09-30 23:00:00,67.212,9,273,39,23,7.762688,60.500,63.682,71.617,...,7.762688,7.762688,7.762688,7.762688,7.762688,7.762688,7.762688,7.762688,7.762688,7.762688
2,2018-10-01 00:00:00,60.500,10,274,40,0,7.812528,63.682,71.617,72.855,...,7.762688,7.762688,7.762688,7.762688,7.762688,7.762688,7.762688,7.762688,7.762688,7.762688
3,2018-10-01 01:00:00,63.682,10,274,40,1,7.812528,71.617,72.855,76.937,...,7.762688,7.762688,7.762688,7.762688,7.762688,7.762688,7.762688,7.762688,7.762688,7.762688
4,2018-10-01 02:00:00,71.617,10,274,40,2,7.812528,72.855,76.937,80.480,...,7.762688,7.762688,7.762688,7.762688,7.762688,7.762688,7.762688,7.762688,7.762688,7.762688


In [213]:
#Dropping rows with Nan's
data_test = data_test.dropna(axis=0)
print(data_test.shape)
data_test.isnull().values.any()

(10491, 77)


False

In [214]:
data_test = remove_23rows_hour_col(data_test)
data_test.columns

(438, 76)


Index(['applicable_date', 'elec_prices', 'month', 'dayofyear_num',
       'weekofyear_num', 'ng_sqr', 't+1', 't+2', 't+3', 't+4', 't+5', 't+6',
       't+7', 't+8', 't+9', 't+10', 't+11', 't+12', 't+13', 't+14', 't+15',
       't+16', 't+17', 't+18', 't+19', 't+20', 't+21', 't+22', 't+23',
       'lag_1_elec_prices', 'lag_2_elec_prices', 'lag_3_elec_prices',
       'lag_4_elec_prices', 'lag_5_elec_prices', 'lag_6_elec_prices',
       'lag_7_elec_prices', 'lag_8_elec_prices', 'lag_9_elec_prices',
       'lag_10_elec_prices', 'lag_11_elec_prices', 'lag_12_elec_prices',
       'lag_13_elec_prices', 'lag_14_elec_prices', 'lag_15_elec_prices',
       'lag_16_elec_prices', 'lag_17_elec_prices', 'lag_18_elec_prices',
       'lag_19_elec_prices', 'lag_20_elec_prices', 'lag_21_elec_prices',
       'lag_22_elec_prices', 'lag_23_elec_prices', 'lag_24_elec_prices',
       'lag_1_ng_sqr', 'lag_2_ng_sqr', 'lag_3_ng_sqr', 'lag_4_ng_sqr',
       'lag_5_ng_sqr', 'lag_6_ng_sqr', 'lag_7_ng_sqr', 'lag_8_n

In [215]:
#selecting columns for X & y
cols = train.columns.to_list()
x_names = cols[2:6]
lags = cols[-47:]
x_names.extend(lags)
y_names = [cols[1]]
time_steps = cols[6:29]
y_names.extend(time_steps)
print(x_names)
print(y_names)

['month', 'dayofyear_num', 'weekofyear_num', 'ng_sqr', 'lag_1_elec_prices', 'lag_2_elec_prices', 'lag_3_elec_prices', 'lag_4_elec_prices', 'lag_5_elec_prices', 'lag_6_elec_prices', 'lag_7_elec_prices', 'lag_8_elec_prices', 'lag_9_elec_prices', 'lag_10_elec_prices', 'lag_11_elec_prices', 'lag_12_elec_prices', 'lag_13_elec_prices', 'lag_14_elec_prices', 'lag_15_elec_prices', 'lag_16_elec_prices', 'lag_17_elec_prices', 'lag_18_elec_prices', 'lag_19_elec_prices', 'lag_20_elec_prices', 'lag_21_elec_prices', 'lag_22_elec_prices', 'lag_23_elec_prices', 'lag_24_elec_prices', 'lag_1_ng_sqr', 'lag_2_ng_sqr', 'lag_3_ng_sqr', 'lag_4_ng_sqr', 'lag_5_ng_sqr', 'lag_6_ng_sqr', 'lag_7_ng_sqr', 'lag_8_ng_sqr', 'lag_9_ng_sqr', 'lag_10_ng_sqr', 'lag_11_ng_sqr', 'lag_12_ng_sqr', 'lag_13_ng_sqr', 'lag_14_ng_sqr', 'lag_15_ng_sqr', 'lag_16_ng_sqr', 'lag_17_ng_sqr', 'lag_18_ng_sqr', 'lag_19_ng_sqr', 'lag_20_ng_sqr', 'lag_21_ng_sqr', 'lag_22_ng_sqr', 'lag_23_ng_sqr']
['elec_prices', 't+1', 't+2', 't+3', 't+4', 

In [216]:
#GBM
mae,mse,rmse,mape,r2 = testing_runs(10,data_test,gbm12)

Round 1: 
(350, 51) (44, 51) (44, 51)
(350, 24) (44, 24) (44, 24)
MAE:  11.5929
MSE:  377.6809
RMSE:  19.434
MAPE:  21.1778
Coefficient of determination (R-squared):  0.3302


Round 2: 
(350, 51) (44, 51) (44, 51)
(350, 24) (44, 24) (44, 24)
MAE:  10.7083
MSE:  238.842
RMSE:  15.4545
MAPE:  19.6965
Coefficient of determination (R-squared):  0.365


Round 3: 
(350, 51) (44, 51) (44, 51)
(350, 24) (44, 24) (44, 24)
MAE:  10.4822
MSE:  273.6614
RMSE:  16.5427
MAPE:  19.7208
Coefficient of determination (R-squared):  0.3483


Round 4: 
(350, 51) (44, 51) (44, 51)
(350, 24) (44, 24) (44, 24)
MAE:  11.1422
MSE:  274.5611
RMSE:  16.5699
MAPE:  18.9871
Coefficient of determination (R-squared):  0.4319


Round 5: 
(350, 51) (44, 51) (44, 51)
(350, 24) (44, 24) (44, 24)
MAE:  11.6669
MSE:  381.0528
RMSE:  19.5206
MAPE:  20.0444
Coefficient of determination (R-squared):  0.4056


Round 6: 
(350, 51) (44, 51) (44, 51)
(350, 24) (44, 24) (44, 24)
MAE:  10.685
MSE:  238.9766
RMSE:  15.4589
MAPE:  19

In [217]:
#XGBM
mae,mse,rmse,mape,r2 = testing_runs(10,data_test,xgb12)

Round 1: 
(350, 51) (44, 51) (44, 51)
(350, 24) (44, 24) (44, 24)
MAE:  12.3548
MSE:  373.7569
RMSE:  19.3328
MAPE:  21.2534
Coefficient of determination (R-squared):  0.3016


Round 2: 
(350, 51) (44, 51) (44, 51)
(350, 24) (44, 24) (44, 24)
MAE:  12.1694
MSE:  318.7016
RMSE:  17.8522
MAPE:  20.997
Coefficient of determination (R-squared):  0.3824


Round 3: 
(350, 51) (44, 51) (44, 51)
(350, 24) (44, 24) (44, 24)
MAE:  9.4482
MSE:  213.5397
RMSE:  14.613
MAPE:  17.2904
Coefficient of determination (R-squared):  0.56


Round 4: 
(350, 51) (44, 51) (44, 51)
(350, 24) (44, 24) (44, 24)
MAE:  10.372
MSE:  252.4316
RMSE:  15.8881
MAPE:  19.2535
Coefficient of determination (R-squared):  0.3328


Round 5: 
(350, 51) (44, 51) (44, 51)
(350, 24) (44, 24) (44, 24)
MAE:  11.3482
MSE:  270.3726
RMSE:  16.443
MAPE:  22.169
Coefficient of determination (R-squared):  0.5045


Round 6: 
(350, 51) (44, 51) (44, 51)
(350, 24) (44, 24) (44, 24)
MAE:  9.7303
MSE:  202.7855
RMSE:  14.2403
MAPE:  18.3553

In [218]:
#light GBM
mae,mse,rmse,mape,r2 = testing_runs(10,data_test,lgbm12)

Round 1: 
(350, 51) (44, 51) (44, 51)
(350, 24) (44, 24) (44, 24)
MAE:  11.1697
MSE:  276.1182
RMSE:  16.6168
MAPE:  19.7748
Coefficient of determination (R-squared):  0.4004


Round 2: 
(350, 51) (44, 51) (44, 51)
(350, 24) (44, 24) (44, 24)
MAE:  10.4903
MSE:  295.2291
RMSE:  17.1822
MAPE:  19.3281
Coefficient of determination (R-squared):  0.4564


Round 3: 
(350, 51) (44, 51) (44, 51)
(350, 24) (44, 24) (44, 24)
MAE:  9.5835
MSE:  171.0735
RMSE:  13.0795
MAPE:  18.7187
Coefficient of determination (R-squared):  0.3663


Round 4: 
(350, 51) (44, 51) (44, 51)
(350, 24) (44, 24) (44, 24)
MAE:  10.4547
MSE:  227.9379
RMSE:  15.0976
MAPE:  18.9312
Coefficient of determination (R-squared):  0.3838


Round 5: 
(350, 51) (44, 51) (44, 51)
(350, 24) (44, 24) (44, 24)
MAE:  11.7626
MSE:  316.5741
RMSE:  17.7925
MAPE:  22.0517
Coefficient of determination (R-squared):  0.2166


Round 6: 
(350, 51) (44, 51) (44, 51)
(350, 24) (44, 24) (44, 24)
MAE:  13.4341
MSE:  477.7313
RMSE:  21.8571
MAPE: 

<h1 style="text-align:center;font-size:40px;" > Basic time based features + wdsp_d (windspeeds in Dublin county) </h1>

In [243]:
data_1= data.copy()
data_1 = feature_pipeline_1(data_1)
data_1['wdsp_d'] = data['wdsp_d']
data_1.head()

,applicable_date,elec_prices,month,dayofyear_num,weekofyear_num,Hour,wdsp_d
0,2019-01-01 00:00:00,70.28,1,1,1,0,4.626
1,2019-01-01 01:00:00,65.18,1,1,1,1,4.626
2,2019-01-01 02:00:00,60.91,1,1,1,2,4.112
3,2019-01-01 03:00:00,65.16,1,1,1,3,4.112
4,2019-01-01 04:00:00,65.16,1,1,1,4,5.140


In [244]:
data_1 = multi_step(data_1,'elec_prices')
data_1 = calc_24lags(data_1,'elec_prices')
data_1 = calc_24lags(data_1,'wdsp_d')
print(data_1.shape)
print(data_1.columns)
data_1.head()

(8304, 77)
Index(['applicable_date', 'elec_prices', 'month', 'dayofyear_num',
       'weekofyear_num', 'Hour', 'wdsp_d', 't+1', 't+2', 't+3', 't+4', 't+5',
       't+6', 't+7', 't+8', 't+9', 't+10', 't+11', 't+12', 't+13', 't+14',
       't+15', 't+16', 't+17', 't+18', 't+19', 't+20', 't+21', 't+22', 't+23',
       'lag_1_elec_prices', 'lag_2_elec_prices', 'lag_3_elec_prices',
       'lag_4_elec_prices', 'lag_5_elec_prices', 'lag_6_elec_prices',
       'lag_7_elec_prices', 'lag_8_elec_prices', 'lag_9_elec_prices',
       'lag_10_elec_prices', 'lag_11_elec_prices', 'lag_12_elec_prices',
       'lag_13_elec_prices', 'lag_14_elec_prices', 'lag_15_elec_prices',
       'lag_16_elec_prices', 'lag_17_elec_prices', 'lag_18_elec_prices',
       'lag_19_elec_prices', 'lag_20_elec_prices', 'lag_21_elec_prices',
       'lag_22_elec_prices', 'lag_23_elec_prices', 'lag_24_elec_prices',
       'lag_1_wdsp_d', 'lag_2_wdsp_d', 'lag_3_wdsp_d', 'lag_4_wdsp_d',
       'lag_5_wdsp_d', 'lag_6_wdsp_d', 'lag_

,applicable_date,elec_prices,month,dayofyear_num,weekofyear_num,Hour,wdsp_d,t+1,t+2,t+3,...,lag_14_wdsp_d,lag_15_wdsp_d,lag_16_wdsp_d,lag_17_wdsp_d,lag_18_wdsp_d,lag_19_wdsp_d,lag_20_wdsp_d,lag_21_wdsp_d,lag_22_wdsp_d,lag_23_wdsp_d
0,2019-01-01 00:00:00,70.28,1,1,1,0,4.626,65.18,60.91,65.160,...,4.626,4.626,4.626,4.626,4.626,4.626,4.626,4.626,4.626,4.626
1,2019-01-01 01:00:00,65.18,1,1,1,1,4.626,60.91,65.16,65.160,...,4.626,4.626,4.626,4.626,4.626,4.626,4.626,4.626,4.626,4.626
2,2019-01-01 02:00:00,60.91,1,1,1,2,4.112,65.16,65.16,68.290,...,4.626,4.626,4.626,4.626,4.626,4.626,4.626,4.626,4.626,4.626
3,2019-01-01 03:00:00,65.16,1,1,1,3,4.112,65.16,68.29,63.030,...,4.626,4.626,4.626,4.626,4.626,4.626,4.626,4.626,4.626,4.626
4,2019-01-01 04:00:00,65.16,1,1,1,4,5.140,68.29,63.03,51.621,...,4.626,4.626,4.626,4.626,4.626,4.626,4.626,4.626,4.626,4.626


In [245]:
data_1.isnull().values.any()

True

In [246]:
#Dropping rows with Nan's
data_1 = data_1.dropna(axis=0)
print(data_1.shape)
data_1.isnull().values.any()

(8281, 77)


False

In [247]:
data_1 = remove_23rows_hour_col(data_1)
train,test = train_test_split_tbs(data_1)
train.columns

(345, 76)
(276, 76)
(69, 76)


Index(['applicable_date', 'elec_prices', 'month', 'dayofyear_num',
       'weekofyear_num', 'wdsp_d', 't+1', 't+2', 't+3', 't+4', 't+5', 't+6',
       't+7', 't+8', 't+9', 't+10', 't+11', 't+12', 't+13', 't+14', 't+15',
       't+16', 't+17', 't+18', 't+19', 't+20', 't+21', 't+22', 't+23',
       'lag_1_elec_prices', 'lag_2_elec_prices', 'lag_3_elec_prices',
       'lag_4_elec_prices', 'lag_5_elec_prices', 'lag_6_elec_prices',
       'lag_7_elec_prices', 'lag_8_elec_prices', 'lag_9_elec_prices',
       'lag_10_elec_prices', 'lag_11_elec_prices', 'lag_12_elec_prices',
       'lag_13_elec_prices', 'lag_14_elec_prices', 'lag_15_elec_prices',
       'lag_16_elec_prices', 'lag_17_elec_prices', 'lag_18_elec_prices',
       'lag_19_elec_prices', 'lag_20_elec_prices', 'lag_21_elec_prices',
       'lag_22_elec_prices', 'lag_23_elec_prices', 'lag_24_elec_prices',
       'lag_1_wdsp_d', 'lag_2_wdsp_d', 'lag_3_wdsp_d', 'lag_4_wdsp_d',
       'lag_5_wdsp_d', 'lag_6_wdsp_d', 'lag_7_wdsp_d', 'lag_8_w

In [248]:
#selecting columns for X & y
cols = train.columns.to_list()
x_names = cols[2:6]
lags = cols[-47:]
x_names.extend(lags)
y_names = [cols[1]]
time_steps = cols[6:29]
y_names.extend(time_steps)
print(x_names)
print(y_names)

['month', 'dayofyear_num', 'weekofyear_num', 'wdsp_d', 'lag_1_elec_prices', 'lag_2_elec_prices', 'lag_3_elec_prices', 'lag_4_elec_prices', 'lag_5_elec_prices', 'lag_6_elec_prices', 'lag_7_elec_prices', 'lag_8_elec_prices', 'lag_9_elec_prices', 'lag_10_elec_prices', 'lag_11_elec_prices', 'lag_12_elec_prices', 'lag_13_elec_prices', 'lag_14_elec_prices', 'lag_15_elec_prices', 'lag_16_elec_prices', 'lag_17_elec_prices', 'lag_18_elec_prices', 'lag_19_elec_prices', 'lag_20_elec_prices', 'lag_21_elec_prices', 'lag_22_elec_prices', 'lag_23_elec_prices', 'lag_24_elec_prices', 'lag_1_wdsp_d', 'lag_2_wdsp_d', 'lag_3_wdsp_d', 'lag_4_wdsp_d', 'lag_5_wdsp_d', 'lag_6_wdsp_d', 'lag_7_wdsp_d', 'lag_8_wdsp_d', 'lag_9_wdsp_d', 'lag_10_wdsp_d', 'lag_11_wdsp_d', 'lag_12_wdsp_d', 'lag_13_wdsp_d', 'lag_14_wdsp_d', 'lag_15_wdsp_d', 'lag_16_wdsp_d', 'lag_17_wdsp_d', 'lag_18_wdsp_d', 'lag_19_wdsp_d', 'lag_20_wdsp_d', 'lag_21_wdsp_d', 'lag_22_wdsp_d', 'lag_23_wdsp_d']
['elec_prices', 't+1', 't+2', 't+3', 't+4', 

In [249]:
x_train, y_train = train[x_names], train[y_names]
x_test, y_test = test[x_names], test[y_names]
print(x_train.shape,y_train.shape)
print(x_test.shape,y_test.shape)

(276, 51) (276, 24)
(69, 51) (69, 24)


In [250]:
x_train_norm,x_test_norm = normalize(x_train,x_test)

In [251]:
GBM_training(x_train_norm,y_train)

Duration: 0:00:10.150515



{'mean_fit_time': array([0.54386334, 0.64656918, 0.64701445, 0.71381295, 1.45684385,
       1.042992  , 1.09164934, 0.45914531, 0.97054098, 0.72120554]), 'std_fit_time': array([0.03118892, 0.01263549, 0.01690935, 0.05587808, 0.15399407,
       0.1148525 , 0.08687333, 0.00933301, 0.03311725, 0.09622836]), 'mean_score_time': array([0.0110147 , 0.01193154, 0.01077001, 0.01201448, 0.01330841,
       0.01271393, 0.01253543, 0.01304617, 0.01208925, 0.00807941]), 'std_score_time': array([0.00134327, 0.00143977, 0.00112467, 0.00128971, 0.00103278,
       0.00175047, 0.00203564, 0.00376933, 0.00098624, 0.00310611]), 'param_estimator__subsample': masked_array(data=[0.9, 0.85, 0.85, 0.75, 0.8, 0.6, 0.9, 0.7, 0.8, 0.8],
             mask=[False, False, False, False, False, False, False, False,
                   False, False],
       fill_value='?',
            dtype=object), 'param_estimator__n_estimators': masked_array(data=[30, 60, 60, 50, 80, 90, 100, 40, 90, 80],
 

In [252]:
#def GBM_model(X,y,learning_rate,n_estimators,max_depth,min_samples_split,max_features,min_samples_leaf,subsample):
gbm13 = GBM_model(x_train_norm,y_train,0.05,50,5,100,2,10,0.75)

Duration: 0:00:00.355186


In [253]:
y_pred= gbm13.predict(x_test_norm)
eval_metrics(y_test,y_pred)

MAE:  10.5196
MSE:  230.2973
RMSE:  15.1756
MAPE:  22.411
Coefficient of determination (R-squared):  0.109




(10.519621188954517,
 230.29732317544858,
 15.175550177026485,
 22.41104000019333,
 0.10902050484735426)

In [254]:
XGBM_training(x_train_norm,y_train)

Duration: 0:00:43.178453



{'mean_fit_time': array([ 5.11619184,  3.21311707,  0.66177368,  4.57754033,  4.58872738,
        1.56083004,  2.98711925, 13.56801488,  7.37287357,  2.90808809]), 'std_fit_time': array([0.39086596, 0.89921152, 0.04276687, 0.18446204, 0.28087484,
       0.11356382, 0.14545009, 0.58746793, 0.30689519, 0.30582528]), 'mean_score_time': array([0.03481109, 0.0277287 , 0.0267415 , 0.02890325, 0.02873361,
       0.02981932, 0.03014138, 0.03801935, 0.02766366, 0.02468297]), 'std_score_time': array([0.00970268, 0.00540836, 0.00623814, 0.00451767, 0.00317069,
       0.00350383, 0.00301819, 0.00777829, 0.00629175, 0.00733354]), 'param_estimator__subsample': masked_array(data=[0.30000000000000004, 0.2, 0.5, 0.4, 0.5, 0.4, 0.2, 0.6,
                   0.8, 0.4],
             mask=[False, False, False, False, False, False, False, False,
                   False, False],
       fill_value='?',
            dtype=object), 'param_estimator__n_estimators': masked_array(data=[1

In [255]:
# def XGBM_model(X,y,learning_rate,n_estimators,max_depth,subsample,colsample_bytree):
xgb13 = XGBM_model(x_train_norm,y_train,0.0125,250,3,0.5,0.7)

Duration: 0:00:01.313370


In [256]:
y_pred= xgb13.predict(x_test_norm)
eval_metrics(y_test,y_pred)

MAE:  10.5755
MSE:  240.9269
RMSE:  15.5218
MAPE:  22.5301
Coefficient of determination (R-squared):  0.1186




(10.575500528977688,
 240.92687758582383,
 15.521819403208628,
 22.53008555344563,
 0.11855084986271762)

In [257]:
LGBM_training(x_train_norm,y_train)

[LightGBM] [Warning] min_child_samples is set=20, min_child_samples= will be ignored. Current value: min_child_samples=20
[LightGBM] [Warning] Unknown parameter: 60
Duration: 0:00:36.673579



{'mean_fit_time': array([5.31263189, 7.10381653, 2.42699034, 4.12696216, 5.4851567 ,
       1.6126509 , 4.92320726, 2.34119241, 4.57185481, 1.8521455 ]), 'std_fit_time': array([0.32652626, 0.59741807, 0.38277351, 0.28509823, 0.21628822,
       0.09322275, 0.16889309, 0.16464809, 0.24461048, 0.17566822]), 'mean_score_time': array([0.02659166, 0.05878329, 0.02057021, 0.0507503 , 0.04585519,
       0.02226007, 0.04898195, 0.02255116, 0.05039225, 0.02282093]), 'std_score_time': array([0.0038673 , 0.00991842, 0.0070498 , 0.01970275, 0.00426871,
       0.0137151 , 0.01572771, 0.002907  , 0.01383485, 0.00733652]), 'param_estimator__subsample': masked_array(data=[0.1, 0.9, 0.8, 0.30000000000000004, 0.6, 0.9, 0.5, 0.5,
                   0.2, 0.2],
             mask=[False, False, False, False, False, Fal

In [258]:
#def LGBM_model(X,y,num_leaves,min_child_samples,learning_rate,n_estimators,max_depth,subsample,colsample_bytree)
lgbm13 = LGBM_model(x_train_norm,y_train,4915,60,0.05,100,7,0.9,0.9)

Duration: 0:00:00.462245


In [259]:
y_pred= lgbm13.predict(x_test_norm)
eval_metrics(y_test,y_pred)

MAE:  11.1505
MSE:  241.4947
RMSE:  15.5401
MAPE:  23.755
Coefficient of determination (R-squared):  0.0457




(11.15047767843595,
 241.49469765626745,
 15.540099666870463,
 23.75501900534941,
 0.0456990722938587)

### Testing on 2018-2019 data

In [260]:
data_test = data_1819.copy()
data_test = feature_pipeline_1(data_test)
data_test['wdsp_d']= data_1819['wdsp_d']
data_test = multi_step(data_test,'elec_prices')
data_test = calc_24lags(data_test,'elec_prices')
data_test = calc_24lags(data_test,'wdsp_d')
print(data_test.shape)
print(data_test.columns)
data_test.head()

(10514, 77)
Index(['applicable_date', 'elec_prices', 'month', 'dayofyear_num',
       'weekofyear_num', 'Hour', 'wdsp_d', 't+1', 't+2', 't+3', 't+4', 't+5',
       't+6', 't+7', 't+8', 't+9', 't+10', 't+11', 't+12', 't+13', 't+14',
       't+15', 't+16', 't+17', 't+18', 't+19', 't+20', 't+21', 't+22', 't+23',
       'lag_1_elec_prices', 'lag_2_elec_prices', 'lag_3_elec_prices',
       'lag_4_elec_prices', 'lag_5_elec_prices', 'lag_6_elec_prices',
       'lag_7_elec_prices', 'lag_8_elec_prices', 'lag_9_elec_prices',
       'lag_10_elec_prices', 'lag_11_elec_prices', 'lag_12_elec_prices',
       'lag_13_elec_prices', 'lag_14_elec_prices', 'lag_15_elec_prices',
       'lag_16_elec_prices', 'lag_17_elec_prices', 'lag_18_elec_prices',
       'lag_19_elec_prices', 'lag_20_elec_prices', 'lag_21_elec_prices',
       'lag_22_elec_prices', 'lag_23_elec_prices', 'lag_24_elec_prices',
       'lag_1_wdsp_d', 'lag_2_wdsp_d', 'lag_3_wdsp_d', 'lag_4_wdsp_d',
       'lag_5_wdsp_d', 'lag_6_wdsp_d', 'lag

,applicable_date,elec_prices,month,dayofyear_num,weekofyear_num,Hour,wdsp_d,t+1,t+2,t+3,...,lag_14_wdsp_d,lag_15_wdsp_d,lag_16_wdsp_d,lag_17_wdsp_d,lag_18_wdsp_d,lag_19_wdsp_d,lag_20_wdsp_d,lag_21_wdsp_d,lag_22_wdsp_d,lag_23_wdsp_d
0,2018-09-30 22:00:00,71.267,9,273,39,22,5.140,67.212,60.500,63.682,...,5.14,5.14,5.14,5.14,5.14,5.14,5.14,5.14,5.14,5.14
1,2018-09-30 23:00:00,67.212,9,273,39,23,5.140,60.500,63.682,71.617,...,5.14,5.14,5.14,5.14,5.14,5.14,5.14,5.14,5.14,5.14
2,2018-10-01 00:00:00,60.500,10,274,40,0,5.140,63.682,71.617,72.855,...,5.14,5.14,5.14,5.14,5.14,5.14,5.14,5.14,5.14,5.14
3,2018-10-01 01:00:00,63.682,10,274,40,1,5.654,71.617,72.855,76.937,...,5.14,5.14,5.14,5.14,5.14,5.14,5.14,5.14,5.14,5.14
4,2018-10-01 02:00:00,71.617,10,274,40,2,4.626,72.855,76.937,80.480,...,5.14,5.14,5.14,5.14,5.14,5.14,5.14,5.14,5.14,5.14


In [261]:
#Dropping rows with Nan's
data_test = data_test.dropna(axis=0)
print(data_test.shape)
data_test.isnull().values.any()

(10491, 77)


False

In [262]:
data_test = remove_23rows_hour_col(data_test)
data_test.columns

(438, 76)


Index(['applicable_date', 'elec_prices', 'month', 'dayofyear_num',
       'weekofyear_num', 'wdsp_d', 't+1', 't+2', 't+3', 't+4', 't+5', 't+6',
       't+7', 't+8', 't+9', 't+10', 't+11', 't+12', 't+13', 't+14', 't+15',
       't+16', 't+17', 't+18', 't+19', 't+20', 't+21', 't+22', 't+23',
       'lag_1_elec_prices', 'lag_2_elec_prices', 'lag_3_elec_prices',
       'lag_4_elec_prices', 'lag_5_elec_prices', 'lag_6_elec_prices',
       'lag_7_elec_prices', 'lag_8_elec_prices', 'lag_9_elec_prices',
       'lag_10_elec_prices', 'lag_11_elec_prices', 'lag_12_elec_prices',
       'lag_13_elec_prices', 'lag_14_elec_prices', 'lag_15_elec_prices',
       'lag_16_elec_prices', 'lag_17_elec_prices', 'lag_18_elec_prices',
       'lag_19_elec_prices', 'lag_20_elec_prices', 'lag_21_elec_prices',
       'lag_22_elec_prices', 'lag_23_elec_prices', 'lag_24_elec_prices',
       'lag_1_wdsp_d', 'lag_2_wdsp_d', 'lag_3_wdsp_d', 'lag_4_wdsp_d',
       'lag_5_wdsp_d', 'lag_6_wdsp_d', 'lag_7_wdsp_d', 'lag_8_w

In [263]:
#selecting columns for X & y
cols = train.columns.to_list()
x_names = cols[2:6]
lags = cols[-47:]
x_names.extend(lags)
y_names = [cols[1]]
time_steps = cols[6:29]
y_names.extend(time_steps)
print(x_names)
print(y_names)

['month', 'dayofyear_num', 'weekofyear_num', 'wdsp_d', 'lag_1_elec_prices', 'lag_2_elec_prices', 'lag_3_elec_prices', 'lag_4_elec_prices', 'lag_5_elec_prices', 'lag_6_elec_prices', 'lag_7_elec_prices', 'lag_8_elec_prices', 'lag_9_elec_prices', 'lag_10_elec_prices', 'lag_11_elec_prices', 'lag_12_elec_prices', 'lag_13_elec_prices', 'lag_14_elec_prices', 'lag_15_elec_prices', 'lag_16_elec_prices', 'lag_17_elec_prices', 'lag_18_elec_prices', 'lag_19_elec_prices', 'lag_20_elec_prices', 'lag_21_elec_prices', 'lag_22_elec_prices', 'lag_23_elec_prices', 'lag_24_elec_prices', 'lag_1_wdsp_d', 'lag_2_wdsp_d', 'lag_3_wdsp_d', 'lag_4_wdsp_d', 'lag_5_wdsp_d', 'lag_6_wdsp_d', 'lag_7_wdsp_d', 'lag_8_wdsp_d', 'lag_9_wdsp_d', 'lag_10_wdsp_d', 'lag_11_wdsp_d', 'lag_12_wdsp_d', 'lag_13_wdsp_d', 'lag_14_wdsp_d', 'lag_15_wdsp_d', 'lag_16_wdsp_d', 'lag_17_wdsp_d', 'lag_18_wdsp_d', 'lag_19_wdsp_d', 'lag_20_wdsp_d', 'lag_21_wdsp_d', 'lag_22_wdsp_d', 'lag_23_wdsp_d']
['elec_prices', 't+1', 't+2', 't+3', 't+4', 

In [264]:
#GBM
mae,mse,rmse,mape,r2 = testing_runs(10,data_test,gbm13)

Round 1: 
(350, 51) (44, 51) (44, 51)
(350, 24) (44, 24) (44, 24)
MAE:  10.0316
MSE:  265.3898
RMSE:  16.2908
MAPE:  19.0943
Coefficient of determination (R-squared):  0.4687


Round 2: 
(350, 51) (44, 51) (44, 51)
(350, 24) (44, 24) (44, 24)
MAE:  11.3311
MSE:  384.6116
RMSE:  19.6115
MAPE:  20.1815
Coefficient of determination (R-squared):  0.3948


Round 3: 
(350, 51) (44, 51) (44, 51)
(350, 24) (44, 24) (44, 24)
MAE:  12.2487
MSE:  353.1572
RMSE:  18.7925
MAPE:  21.3012
Coefficient of determination (R-squared):  0.3343


Round 4: 
(350, 51) (44, 51) (44, 51)
(350, 24) (44, 24) (44, 24)
MAE:  10.9824
MSE:  267.5916
RMSE:  16.3582
MAPE:  19.5984
Coefficient of determination (R-squared):  0.4232


Round 5: 
(350, 51) (44, 51) (44, 51)
(350, 24) (44, 24) (44, 24)
MAE:  8.9322
MSE:  175.1699
RMSE:  13.2352
MAPE:  17.0815
Coefficient of determination (R-squared):  0.4381


Round 6: 
(350, 51) (44, 51) (44, 51)
(350, 24) (44, 24) (44, 24)
MAE:  11.516
MSE:  317.0506
RMSE:  17.8059
MAPE:  

In [265]:
#XGBM
mae,mse,rmse,mape,r2 = testing_runs(10,data_test,xgb13)

Round 1: 
(350, 51) (44, 51) (44, 51)
(350, 24) (44, 24) (44, 24)
MAE:  12.0092
MSE:  389.5891
RMSE:  19.738
MAPE:  20.8912
Coefficient of determination (R-squared):  0.4535


Round 2: 
(350, 51) (44, 51) (44, 51)
(350, 24) (44, 24) (44, 24)
MAE:  9.9953
MSE:  217.6463
RMSE:  14.7528
MAPE:  17.9321
Coefficient of determination (R-squared):  0.2298


Round 3: 
(350, 51) (44, 51) (44, 51)
(350, 24) (44, 24) (44, 24)
MAE:  10.5758
MSE:  320.0226
RMSE:  17.8892
MAPE:  19.2587
Coefficient of determination (R-squared):  0.5338


Round 4: 
(350, 51) (44, 51) (44, 51)
(350, 24) (44, 24) (44, 24)
MAE:  12.6034
MSE:  450.0106
RMSE:  21.2135
MAPE:  22.0167
Coefficient of determination (R-squared):  0.3051


Round 5: 
(350, 51) (44, 51) (44, 51)
(350, 24) (44, 24) (44, 24)
MAE:  9.498
MSE:  222.2189
RMSE:  14.907
MAPE:  18.481
Coefficient of determination (R-squared):  0.5778


Round 6: 
(350, 51) (44, 51) (44, 51)
(350, 24) (44, 24) (44, 24)
MAE:  10.1275
MSE:  203.6776
RMSE:  14.2716
MAPE:  17.7

In [266]:
#light GBM
mae,mse,rmse,mape,r2 = testing_runs(10,data_test,lgbm13)

Round 1: 
(350, 51) (44, 51) (44, 51)
(350, 24) (44, 24) (44, 24)
MAE:  12.0308
MSE:  310.3486
RMSE:  17.6167
MAPE:  21.6229
Coefficient of determination (R-squared):  0.3752


Round 2: 
(350, 51) (44, 51) (44, 51)
(350, 24) (44, 24) (44, 24)
MAE:  12.6965
MSE:  428.9704
RMSE:  20.7116
MAPE:  23.2517
Coefficient of determination (R-squared):  0.3351


Round 3: 
(350, 51) (44, 51) (44, 51)
(350, 24) (44, 24) (44, 24)
MAE:  10.9663
MSE:  301.9585
RMSE:  17.377
MAPE:  21.2135
Coefficient of determination (R-squared):  0.4708


Round 4: 
(350, 51) (44, 51) (44, 51)
(350, 24) (44, 24) (44, 24)
MAE:  12.155
MSE:  483.5352
RMSE:  21.9894
MAPE:  19.7587
Coefficient of determination (R-squared):  0.5117


Round 5: 
(350, 51) (44, 51) (44, 51)
(350, 24) (44, 24) (44, 24)
MAE:  10.5813
MSE:  325.897
RMSE:  18.0526
MAPE:  19.8948
Coefficient of determination (R-squared):  0.4528


Round 6: 
(350, 51) (44, 51) (44, 51)
(350, 24) (44, 24) (44, 24)
MAE:  10.8509
MSE:  296.4566
RMSE:  17.2179
MAPE:  1

In [267]:
import pickle
# save the model to disk
pickle.dump(gbm9, open('GBM_basic_ng_sqrt_model.sav', 'wb'))
pickle.dump(gbm10, open('GBM_basic_daily_natural_gas_model.sav', 'wb'))
pickle.dump(gbm11, open('GBM_basic_ng_log_model.sav', 'wb'))
pickle.dump(gbm12, open('GBM_basic_ng_sqr_model.sav', 'wb'))
pickle.dump(gbm13, open('GBM_basic_wdsp_d_model.sav', 'wb'))

pickle.dump(xgb9, open('XGBM_basic_ng_sqrt_model.sav', 'wb'))
pickle.dump(xgb10, open('XGBM_basic_daily_natural_gas_model.sav', 'wb'))
pickle.dump(xgb11, open('XGBM_basic_ng_log_model.sav', 'wb'))
pickle.dump(xgb12, open('XGBM_basic_ng_sqr_model.sav', 'wb'))
pickle.dump(xgb13, open('XGBM_basic_wdsp_d_model.sav', 'wb'))

pickle.dump(lgbm9, open('LightGBM_basic_ng_sqrt_model.sav', 'wb'))
pickle.dump(lgbm10, open('LightGBM_basic_daily_natural_gas_model.sav', 'wb'))
pickle.dump(lgbm11, open('LightGBM_basic_ng_log_model.sav', 'wb'))
pickle.dump(lgbm12, open('LightGBM_basic_ng_sqr_model.sav', 'wb'))
pickle.dump(lgbm13, open('LightGBM_basic_wdsp_d_model.sav', 'wb'))
#gbm = pickle.load(open('GBMtimefeatures1_model.sav', 'rb'))